# Define Library

In [1]:
# %% [markdown]
# # Jupyter Notebook Loading Header
#
# This is a custom loading header for Jupyter Notebooks in Visual Studio Code.
# It includes common imports and settings to get you started quickly.
# %% [markdown]
## Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from google.cloud import bigquery
from google.cloud import storage
import os
import tempfile
import time
from datetime import datetime
import uuid
import joblib
import uuid

import gcsfs
import duckdb as dd
import pickle
import joblib
from typing import Union
import io

path = r'C:\Users\Dwaipayan\AppData\Roaming\gcloud\legacy_credentials\dchakroborti@tonikbank.com\adc.json'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = path
client = bigquery.Client(project='prj-prod-dataplatform')
os.environ["GOOGLE_CLOUD_PROJECT"] = "prj-prod-dataplatform"
# %% [markdown]
## Configure Settings
# Set options or configurations as needed
pd.set_option('display.max_columns', None)
pd.set_option("Display.max_rows", 100)

# Function

## expand_calc_features

In [2]:
import pandas as pd
import json

def expand_calc_features(df):
    """
    Expand the calcFeatures JSON column into separate columns and return the complete DataFrame.
    
    Parameters:
    df (pd.DataFrame): Input DataFrame with calcFeatures column containing JSON data
    
    Returns:
    pd.DataFrame: Expanded DataFrame with all original columns plus JSON features as separate columns
    """
    
    # Make a copy to avoid modifying the original DataFrame
    df_expanded = df.copy()
    
    # Parse the calcFeatures JSON column
    calc_features_list = []
    
    for idx, calc_features_str in enumerate(df['calcFeatures']):
        try:
            # Parse the JSON string
            features_dict = json.loads(calc_features_str.replace("'", '"'))  # Replace single quotes with double quotes for valid JSON
            calc_features_list.append(features_dict)
        except (json.JSONDecodeError, AttributeError) as e:
            # If parsing fails, create an empty dict and print warning
            print(f"Warning: Could not parse calcFeatures at index {idx}: {e}")
            calc_features_list.append({})
    
    # Create DataFrame from the parsed JSON data
    calc_features_df = pd.DataFrame(calc_features_list)
    
    # Add prefix to JSON-derived columns to avoid conflicts
    calc_features_df = calc_features_df.add_prefix('calc_')
    
    # Reset index to ensure proper alignment
    df_expanded = df_expanded.reset_index(drop=True)
    calc_features_df = calc_features_df.reset_index(drop=True)
    
    # Combine original DataFrame with expanded calcFeatures
    result_df = pd.concat([df_expanded, calc_features_df], axis=1)
    
    return result_df


## expand_calc_features_robust

In [3]:
import pandas as pd
import json

def expand_calc_features_robust(df):
    """
    Expand the calcFeatures JSON column into separate columns with better error handling.
    
    Parameters:
    df (pd.DataFrame): Input DataFrame with calcFeatures column containing JSON data
    
    Returns:
    pd.DataFrame: Expanded DataFrame with all original columns plus JSON features as separate columns
    """
    
    # Make a copy to avoid modifying the original DataFrame
    df_expanded = df.copy()
    
    # Parse the calcFeatures JSON column
    calc_features_data = []
    
    for idx, row in df.iterrows():
        calc_features_str = row['calcFeatures']
        
        if pd.isna(calc_features_str) or calc_features_str == '':
            calc_features_data.append({})
            continue
            
        try:
            # Clean the string and parse JSON
            cleaned_str = calc_features_str.replace("'", '"').replace('None', 'null').replace('True', 'true').replace('False', 'false')
            features_dict = json.loads(cleaned_str)
            calc_features_data.append(features_dict)
        except Exception as e:
            print(f"Warning: Could not parse calcFeatures at index {idx}: {e}")
            print(f"Problematic string: {calc_features_str[:100]}...")  # Print first 100 chars
            calc_features_data.append({})
    
    # Create DataFrame from the parsed JSON data
    calc_features_df = pd.DataFrame(calc_features_data)
    
    # Add prefix to JSON-derived columns to avoid conflicts with existing columns
    calc_features_df = calc_features_df.add_prefix('feat_')
    
    # Combine DataFrames
    result_df = pd.concat([df_expanded, calc_features_df], axis=1)
    
    print(f"Original DataFrame shape: {df.shape}")
    print(f"Expanded DataFrame shape: {result_df.shape}")
    print(f"Added {len(calc_features_df.columns)} new columns from calcFeatures")
    
    return result_df

## PSI Functions new

In [4]:
# import pandas as pd
# import numpy as np
# from typing import List, Dict, Tuple
# import warnings
# warnings.filterwarnings('ignore')

# def identify_feature_types(df: pd.DataFrame, feature_list: List[str]) -> Dict[str, List[str]]:
#     """
#     Identify categorical and numerical features from the feature list.
    
#     Parameters:
#     -----------
#     df : pd.DataFrame
#         Input dataframe
#     feature_list : List[str]
#         List of features to classify
    
#     Returns:
#     --------
#     Dict with 'categorical' and 'numerical' keys containing respective feature lists
#     """
#     categorical_features = []
#     numerical_features = []
    
#     for feature in feature_list:
#         if feature not in df.columns:
#             print(f"Warning: Feature '{feature}' not found in dataframe")
#             continue
            
#         # Check if feature is numeric
#         if pd.api.types.is_numeric_dtype(df[feature]):
#             # If unique values are less than 15 and all integers, treat as categorical
#             unique_vals = df[feature].nunique()
#             if unique_vals < 15 and df[feature].dropna().apply(lambda x: x == int(x) if isinstance(x, (int, float)) else False).all():
#                 categorical_features.append(feature)
#             else:
#                 numerical_features.append(feature)
#         else:
#             categorical_features.append(feature)
    
#     return {
#         'categorical': categorical_features,
#         'numerical': numerical_features
#     }


# def create_bins_for_features(df: pd.DataFrame, 
#                              numerical_features: List[str],
#                              categorical_features: List[str],
#                              base_month: str,
#                              month_col: str = 'Application_month') -> Dict:
#     """
#     Create bins for numerical features (deciles) and categorical features (top 6 + others).
    
#     Parameters:
#     -----------
#     df : pd.DataFrame
#         Input dataframe
#     numerical_features : List[str]
#         List of numerical features
#     categorical_features : List[str]
#         List of categorical features
#     base_month : str
#         Base month to determine binning strategy
#     month_col : str
#         Name of month column
    
#     Returns:
#     --------
#     Dictionary containing binning information for each feature
#     """
#     base_df = df[df[month_col] == base_month].copy()
#     binning_info = {}
    
#     # Create bins for numerical features (deciles)
#     for feature in numerical_features:
#         valid_data = base_df[feature].dropna()
        
#         if len(valid_data) == 0:
#             binning_info[feature] = {'type': 'numerical', 'bins': None}
#             continue
        
#         # Create decile bins
#         try:
#             bins = np.percentile(valid_data, np.arange(0, 101, 10))
#             # Remove duplicates and sort
#             bins = np.unique(bins)
#             binning_info[feature] = {
#                 'type': 'numerical',
#                 'bins': bins
#             }
#         except:
#             binning_info[feature] = {'type': 'numerical', 'bins': None}
    
#     # Create bins for categorical features (top 6 + others)
#     for feature in categorical_features:
#         value_counts = base_df[feature].value_counts()
#         top_6 = value_counts.nlargest(6).index.tolist()
        
#         binning_info[feature] = {
#             'type': 'categorical',
#             'top_categories': top_6
#         }
    
#     return binning_info


# def apply_binning(df: pd.DataFrame, 
#                   feature: str, 
#                   binning_info: Dict) -> pd.Series:
#     """
#     Apply binning to a feature based on binning information.
    
#     Parameters:
#     -----------
#     df : pd.DataFrame
#         Input dataframe
#     feature : str
#         Feature name
#     binning_info : Dict
#         Binning information for the feature
    
#     Returns:
#     --------
#     pd.Series with binned values
#     """
#     if binning_info['type'] == 'numerical':
#         if binning_info['bins'] is None:
#             return pd.Series(['Missing'] * len(df), index=df.index)
        
#         bins = binning_info['bins']
#         labels = [f"Bin_{i+1}" for i in range(len(bins)-1)]
        
#         binned = pd.cut(df[feature], 
#                        bins=bins, 
#                        labels=labels, 
#                        include_lowest=True,
#                        duplicates='drop')
        
#         # Handle nulls - convert to string and then replace
#         binned = binned.astype(str)
#         binned[df[feature].isna()] = 'Missing'
        
#         return binned
    
#     else:  # categorical
#         top_cats = binning_info['top_categories']
        
#         # Convert categorical to object if needed to avoid category errors
#         if pd.api.types.is_categorical_dtype(df[feature]):
#             feature_data = df[feature].astype(str)
#         else:
#             feature_data = df[feature].astype(str)
        
#         # Replace NaN string representation with 'Missing'
#         feature_data = feature_data.replace('nan', 'Missing')
        
#         # Apply binning logic
#         binned = feature_data.apply(lambda x: x if x in top_cats else ('Missing' if x == 'Missing' else 'Others'))
        
#         return binned


# def calculate_psi(expected_pct: pd.Series, 
#                   actual_pct: pd.Series, 
#                   epsilon: float = 0.0001) -> float:
#     """
#     Calculate Population Stability Index.
    
#     Parameters:
#     -----------
#     expected_pct : pd.Series
#         Expected (baseline) percentages
#     actual_pct : pd.Series
#         Actual percentages
#     epsilon : float
#         Small value to avoid log(0)
    
#     Returns:
#     --------
#     PSI value
#     """
#     # Align indices
#     all_bins = expected_pct.index.union(actual_pct.index)
#     expected_pct = expected_pct.reindex(all_bins, fill_value=0)
#     actual_pct = actual_pct.reindex(all_bins, fill_value=0)
    
#     # Add epsilon to avoid log(0)
#     expected_pct = expected_pct + epsilon
#     actual_pct = actual_pct + epsilon
    
#     # Calculate PSI
#     psi_value = np.sum((actual_pct - expected_pct) * np.log(actual_pct / expected_pct))
    
#     return psi_value


# def calculate_month_on_month_psi(df: pd.DataFrame,
#                                  feature_list: List[str],
#                                  segment_columns: List[str],
#                                  month_col: str = 'Application_month') -> pd.DataFrame:
#     """
#     Calculate month-on-month PSI for each feature, overall and by segments.
#     Each row shows PSI comparing first month vs each subsequent month.
    
#     Parameters:
#     -----------
#     df : pd.DataFrame
#         Input dataframe
#     feature_list : List[str]
#         List of features to calculate PSI for
#     segment_columns : List[str]
#         List of segment columns
#     month_col : str
#         Name of month column
    
#     Returns:
#     --------
#     pd.DataFrame with PSI values with one row per feature-month-segment combination
#     """
#     # Create a copy to avoid modifying original
#     df = df.copy()
    
#     # Identify feature types
#     feature_types = identify_feature_types(df, feature_list)
    
#     # Get sorted months
#     months = sorted(df[month_col].unique())
#     base_month = months[0]
    
#     # Create binning strategy based on base month
#     binning_info = create_bins_for_features(
#         df, 
#         feature_types['numerical'],
#         feature_types['categorical'],
#         base_month,
#         month_col
#     )
    
#     results = []
    
#     # Calculate overall PSI
#     for feature in feature_list:
#         if feature not in df.columns:
#             continue
        
#         # Apply binning
#         df[f'{feature}_binned'] = apply_binning(df, feature, binning_info[feature])
        
#         # Get base month distribution
#         base_dist = df[df[month_col] == base_month][f'{feature}_binned'].value_counts(normalize=True)
        
#         # For each month (including base month for reference)
#         for month in months:
#             actual_dist = df[df[month_col] == month][f'{feature}_binned'].value_counts(normalize=True)
#             psi_value = calculate_psi(base_dist, actual_dist) if month != base_month else 0.0
            
#             results.append({
#                 'Feature': feature,
#                 'Feature_Type': binning_info[feature]['type'],
#                 'Segment_Column': 'Overall',
#                 'Segment_Value': 'All',
#                 'Month': f"{month}",
#                 'Base_Month': base_month,
#                 'Current_Month': month,
#                 'PSI': psi_value
#             })
    
#     # Calculate PSI by segments
#     for segment_col in segment_columns:
#         if segment_col not in df.columns:
#             continue
        
#         segments = df[segment_col].dropna().unique()
        
#         for segment_val in segments:
#             segment_df = df[df[segment_col] == segment_val]
            
#             for feature in feature_list:
#                 if feature not in df.columns:
#                     continue
                
#                 # Get base month distribution for segment
#                 base_segment = segment_df[segment_df[month_col] == base_month]
#                 if len(base_segment) == 0:
#                     continue
                
#                 base_dist = base_segment[f'{feature}_binned'].value_counts(normalize=True)
                
#                 # For each month (including base month for reference)
#                 for month in months:
#                     actual_segment = segment_df[segment_df[month_col] == month]
#                     if len(actual_segment) == 0:
#                         continue
                    
#                     actual_dist = actual_segment[f'{feature}_binned'].value_counts(normalize=True)
#                     psi_value = calculate_psi(base_dist, actual_dist) if month != base_month else 0.0
                    
#                     results.append({
#                         'Feature': feature,
#                         'Feature_Type': binning_info[feature]['type'],
#                         'Segment_Column': segment_col,
#                         'Segment_Value': segment_val,
#                         'Month': f"{month}",
#                         'Base_Month': base_month,
#                         'Current_Month': month,
#                         'PSI': psi_value
#                     })
    
#     return pd.DataFrame(results)


# def calculate_bin_level_psi(df: pd.DataFrame,
#                             feature_list: List[str],
#                             segment_columns: List[str],
#                             month_col: str = 'Application_month') -> pd.DataFrame:
#     """
#     Calculate bin-level PSI for each feature, overall and by segments.
#     Each row shows bin-level PSI comparing first month vs each month.
    
#     Parameters:
#     -----------
#     df : pd.DataFrame
#         Input dataframe
#     feature_list : List[str]
#         List of features to calculate PSI for
#     segment_columns : List[str]
#         List of segment columns
#     month_col : str
#         Name of month column
    
#     Returns:
#     --------
#     pd.DataFrame with bin-level PSI details
#     """
#     # Create a copy to avoid modifying original
#     df = df.copy()
    
#     # Identify feature types
#     feature_types = identify_feature_types(df, feature_list)
    
#     # Get sorted months
#     months = sorted(df[month_col].unique())
#     base_month = months[0]
    
#     # Create binning strategy based on base month
#     binning_info = create_bins_for_features(
#         df, 
#         feature_types['numerical'],
#         feature_types['categorical'],
#         base_month,
#         month_col
#     )
    
#     results = []
#     epsilon = 0.0001
    
#     # Calculate overall bin-level PSI
#     for feature in feature_list:
#         if feature not in df.columns:
#             continue
        
#         # Apply binning
#         df[f'{feature}_binned'] = apply_binning(df, feature, binning_info[feature])
        
#         # Get base month distribution
#         base_dist = df[df[month_col] == base_month][f'{feature}_binned'].value_counts(normalize=True)
        
#         # Calculate bin-level PSI for each month (including base month)
#         for month in months:
#             actual_dist = df[df[month_col] == month][f'{feature}_binned'].value_counts(normalize=True)
            
#             # Get all bins
#             all_bins = base_dist.index.union(actual_dist.index)
            
#             for bin_name in all_bins:
#                 expected_pct = base_dist.get(bin_name, 0) + epsilon
#                 actual_pct = actual_dist.get(bin_name, 0) + epsilon
                
#                 bin_psi = (actual_pct - expected_pct) * np.log(actual_pct / expected_pct) if month != base_month else 0.0
                
#                 results.append({
#                     'Feature': feature,
#                     'Feature_Type': binning_info[feature]['type'],
#                     'Segment_Column': 'Overall',
#                     'Segment_Value': 'All',
#                     'Month': f"{month}",
#                     'Base_Month': base_month,
#                     'Current_Month': month,
#                     'Bin': bin_name,
#                     'Base_Percentage': (base_dist.get(bin_name, 0) * 100),
#                     'Actual_Percentage': (actual_dist.get(bin_name, 0) * 100),
#                     'Bin_PSI': bin_psi
#                 })
    
#     # Calculate bin-level PSI by segments
#     for segment_col in segment_columns:
#         if segment_col not in df.columns:
#             continue
        
#         segments = df[segment_col].dropna().unique()
        
#         for segment_val in segments:
#             segment_df = df[df[segment_col] == segment_val]
            
#             for feature in feature_list:
#                 if feature not in df.columns:
#                     continue
                
#                 # Get base month distribution for segment
#                 base_segment = segment_df[segment_df[month_col] == base_month]
#                 if len(base_segment) == 0:
#                     continue
                
#                 base_dist = base_segment[f'{feature}_binned'].value_counts(normalize=True)
                
#                 # Calculate bin-level PSI for each month (including base month)
#                 for month in months:
#                     actual_segment = segment_df[segment_df[month_col] == month]
#                     if len(actual_segment) == 0:
#                         continue
                    
#                     actual_dist = actual_segment[f'{feature}_binned'].value_counts(normalize=True)
                    
#                     # Get all bins
#                     all_bins = base_dist.index.union(actual_dist.index)
                    
#                     for bin_name in all_bins:
#                         expected_pct = base_dist.get(bin_name, 0) + epsilon
#                         actual_pct = actual_dist.get(bin_name, 0) + epsilon
                        
#                         bin_psi = (actual_pct - expected_pct) * np.log(actual_pct / expected_pct) if month != base_month else 0.0
                        
#                         results.append({
#                             'Feature': feature,
#                             'Feature_Type': binning_info[feature]['type'],
#                             'Segment_Column': segment_col,
#                             'Segment_Value': segment_val,
#                             'Month': f"{month}",
#                             'Base_Month': base_month,
#                             'Current_Month': month,
#                             'Bin': bin_name,
#                             'Base_Percentage': (base_dist.get(bin_name, 0) * 100),
#                             'Actual_Percentage': (actual_dist.get(bin_name, 0) * 100),
#                             'Bin_PSI': bin_psi
#                         })
    
#     return pd.DataFrame(results)

In [5]:
import pandas as pd
import numpy as np
from typing import List, Dict, Tuple
import warnings
warnings.filterwarnings('ignore')

def identify_feature_types(df: pd.DataFrame, feature_list: List[str]) -> Dict[str, List[str]]:
    """
    Identify categorical and numerical features from the feature list.
    
    Parameters:
    -----------
    df : pd.DataFrame
        Input dataframe
    feature_list : List[str]
        List of features to classify
    
    Returns:
    --------
    Dict with 'categorical' and 'numerical' keys containing respective feature lists
    """
    categorical_features = []
    numerical_features = []
    
    for feature in feature_list:
        if feature not in df.columns:
            print(f"Warning: Feature '{feature}' not found in dataframe")
            continue
            
        # Check if feature is numeric
        if pd.api.types.is_numeric_dtype(df[feature]):
            # If unique values are less than 15 and all integers, treat as categorical
            unique_vals = df[feature].nunique()
            if unique_vals < 15 and df[feature].dropna().apply(lambda x: x == int(x) if isinstance(x, (int, float)) else False).all():
                categorical_features.append(feature)
            else:
                numerical_features.append(feature)
        else:
            categorical_features.append(feature)
    
    return {
        'categorical': categorical_features,
        'numerical': numerical_features
    }


def create_bins_for_features(df: pd.DataFrame, 
                             numerical_features: List[str],
                             categorical_features: List[str],
                             base_month: str,
                             month_col: str = 'Application_month') -> Dict:
    """
    Create bins for numerical features (deciles) and categorical features (top 6 + others).
    
    Parameters:
    -----------
    df : pd.DataFrame
        Input dataframe
    numerical_features : List[str]
        List of numerical features
    categorical_features : List[str]
        List of categorical features
    base_month : str
        Base month to determine binning strategy
    month_col : str
        Name of month column
    
    Returns:
    --------
    Dictionary containing binning information for each feature
    """
    base_df = df[df[month_col] == base_month].copy()
    binning_info = {}
    
    # Create bins for numerical features (deciles)
    for feature in numerical_features:
        valid_data = base_df[feature].dropna()
        
        if len(valid_data) == 0:
            binning_info[feature] = {'type': 'numerical', 'bins': None, 'bin_ranges': {}}
            continue
        
        # Create decile bins
        try:
            bins = np.percentile(valid_data, np.arange(0, 101, 10))
            # Remove duplicates and sort
            bins = np.unique(bins)
            
            # Create bin ranges dictionary
            bin_ranges = {}
            for i in range(len(bins)-1):
                bin_name = f"Bin_{i+1}"
                bin_ranges[bin_name] = {
                    'min': bins[i],
                    'max': bins[i+1],
                    'range_str': f"[{bins[i]:.2f}, {bins[i+1]:.2f}]"
                }
            
            binning_info[feature] = {
                'type': 'numerical',
                'bins': bins,
                'bin_ranges': bin_ranges
            }
        except:
            binning_info[feature] = {'type': 'numerical', 'bins': None, 'bin_ranges': {}}
    
    # Create bins for categorical features (top 6 + others)
    for feature in categorical_features:
        value_counts = base_df[feature].value_counts()
        top_6 = value_counts.nlargest(6).index.tolist()
        
        binning_info[feature] = {
            'type': 'categorical',
            'top_categories': top_6,
            'bin_ranges': {}  # No ranges for categorical
        }
    
    return binning_info


def apply_binning(df: pd.DataFrame, 
                  feature: str, 
                  binning_info: Dict) -> pd.Series:
    """
    Apply binning to a feature based on binning information.
    
    Parameters:
    -----------
    df : pd.DataFrame
        Input dataframe
    feature : str
        Feature name
    binning_info : Dict
        Binning information for the feature
    
    Returns:
    --------
    pd.Series with binned values
    """
    if binning_info['type'] == 'numerical':
        if binning_info['bins'] is None:
            return pd.Series(['Missing'] * len(df), index=df.index)
        
        bins = binning_info['bins']
        labels = [f"Bin_{i+1}" for i in range(len(bins)-1)]
        
        binned = pd.cut(df[feature], 
                       bins=bins, 
                       labels=labels, 
                       include_lowest=True,
                       duplicates='drop')
        
        # Handle nulls - convert to string and then replace
        binned = binned.astype(str)
        binned[df[feature].isna()] = 'Missing'
        
        return binned
    
    else:  # categorical
        top_cats = binning_info['top_categories']
        
        # Convert categorical to object if needed to avoid category errors
        if pd.api.types.is_categorical_dtype(df[feature]):
            feature_data = df[feature].astype(str)
        else:
            feature_data = df[feature].astype(str)
        
        # Replace NaN string representation with 'Missing'
        feature_data = feature_data.replace('nan', 'Missing')
        
        # Apply binning logic
        binned = feature_data.apply(lambda x: x if x in top_cats else ('Missing' if x == 'Missing' else 'Others'))
        
        return binned


def calculate_psi(expected_pct: pd.Series, 
                  actual_pct: pd.Series, 
                  epsilon: float = 0.0001) -> float:
    """
    Calculate Population Stability Index.
    
    Parameters:
    -----------
    expected_pct : pd.Series
        Expected (baseline) percentages
    actual_pct : pd.Series
        Actual percentages
    epsilon : float
        Small value to avoid log(0)
    
    Returns:
    --------
    PSI value
    """
    # Align indices
    all_bins = expected_pct.index.union(actual_pct.index)
    expected_pct = expected_pct.reindex(all_bins, fill_value=0)
    actual_pct = actual_pct.reindex(all_bins, fill_value=0)
    
    # Add epsilon to avoid log(0)
    expected_pct = expected_pct + epsilon
    actual_pct = actual_pct + epsilon
    
    # Calculate PSI
    psi_value = np.sum((actual_pct - expected_pct) * np.log(actual_pct / expected_pct))
    
    return psi_value


def calculate_month_on_month_psi(df: pd.DataFrame,
                                 feature_list: List[str],
                                 segment_columns: List[str],
                                 month_col: str = 'Application_month') -> pd.DataFrame:
    """
    Calculate month-on-month PSI for each feature, overall and by segments.
    Each row shows PSI comparing first month vs each subsequent month.
    
    Parameters:
    -----------
    df : pd.DataFrame
        Input dataframe
    feature_list : List[str]
        List of features to calculate PSI for
    segment_columns : List[str]
        List of segment columns
    month_col : str
        Name of month column
    
    Returns:
    --------
    pd.DataFrame with PSI values with one row per feature-month-segment combination
    """
    # Create a copy to avoid modifying original
    df = df.copy()
    
    # Identify feature types
    feature_types = identify_feature_types(df, feature_list)
    
    # Get sorted months
    months = sorted(df[month_col].unique())
    base_month = months[0]
    
    # Create binning strategy based on base month
    binning_info = create_bins_for_features(
        df, 
        feature_types['numerical'],
        feature_types['categorical'],
        base_month,
        month_col
    )
    
    results = []
    
    # Calculate overall PSI
    for feature in feature_list:
        if feature not in df.columns:
            continue
        
        # Apply binning
        df[f'{feature}_binned'] = apply_binning(df, feature, binning_info[feature])
        
        # Get base month distribution
        base_dist = df[df[month_col] == base_month][f'{feature}_binned'].value_counts(normalize=True)
        
        # For each month (including base month for reference)
        for month in months:
            actual_dist = df[df[month_col] == month][f'{feature}_binned'].value_counts(normalize=True)
            psi_value = calculate_psi(base_dist, actual_dist) if month != base_month else 0.0
            
            results.append({
                'Feature': feature,
                'Feature_Type': binning_info[feature]['type'],
                'Segment_Column': 'Overall',
                'Segment_Value': 'All',
                'Month': f"{month}",
                'Base_Month': base_month,
                'Current_Month': month,
                'PSI': psi_value
            })
    
    # Calculate PSI by segments
    for segment_col in segment_columns:
        if segment_col not in df.columns:
            continue
        
        segments = df[segment_col].dropna().unique()
        
        for segment_val in segments:
            segment_df = df[df[segment_col] == segment_val]
            
            for feature in feature_list:
                if feature not in df.columns:
                    continue
                
                # Get base month distribution for segment
                base_segment = segment_df[segment_df[month_col] == base_month]
                if len(base_segment) == 0:
                    continue
                
                base_dist = base_segment[f'{feature}_binned'].value_counts(normalize=True)
                
                # For each month (including base month for reference)
                for month in months:
                    actual_segment = segment_df[segment_df[month_col] == month]
                    if len(actual_segment) == 0:
                        continue
                    
                    actual_dist = actual_segment[f'{feature}_binned'].value_counts(normalize=True)
                    psi_value = calculate_psi(base_dist, actual_dist) if month != base_month else 0.0
                    
                    results.append({
                        'Feature': feature,
                        'Feature_Type': binning_info[feature]['type'],
                        'Segment_Column': segment_col,
                        'Segment_Value': segment_val,
                        'Month': f"{month}",
                        'Base_Month': base_month,
                        'Current_Month': month,
                        'PSI': psi_value
                    })
    
    return pd.DataFrame(results)


def calculate_bin_level_psi(df: pd.DataFrame,
                            feature_list: List[str],
                            segment_columns: List[str],
                            month_col: str = 'Application_month') -> pd.DataFrame:
    """
    Calculate bin-level PSI for each feature, overall and by segments.
    Each row shows bin-level PSI comparing first month vs each month.
    
    Parameters:
    -----------
    df : pd.DataFrame
        Input dataframe
    feature_list : List[str]
        List of features to calculate PSI for
    segment_columns : List[str]
        List of segment columns
    month_col : str
        Name of month column
    
    Returns:
    --------
    pd.DataFrame with bin-level PSI details including bin ranges
    """
    # Create a copy to avoid modifying original
    df = df.copy()
    
    # Identify feature types
    feature_types = identify_feature_types(df, feature_list)
    
    # Get sorted months
    months = sorted(df[month_col].unique())
    base_month = months[0]
    
    # Create binning strategy based on base month
    binning_info = create_bins_for_features(
        df, 
        feature_types['numerical'],
        feature_types['categorical'],
        base_month,
        month_col
    )
    
    results = []
    epsilon = 0.0001
    
    # Calculate overall bin-level PSI
    for feature in feature_list:
        if feature not in df.columns:
            continue
        
        # Apply binning
        df[f'{feature}_binned'] = apply_binning(df, feature, binning_info[feature])
        
        # Get base month distribution
        base_dist = df[df[month_col] == base_month][f'{feature}_binned'].value_counts(normalize=True)
        
        # Calculate bin-level PSI for each month (including base month)
        for month in months:
            actual_dist = df[df[month_col] == month][f'{feature}_binned'].value_counts(normalize=True)
            
            # Get all bins
            all_bins = base_dist.index.union(actual_dist.index)
            
            for bin_name in all_bins:
                expected_pct = base_dist.get(bin_name, 0) + epsilon
                actual_pct = actual_dist.get(bin_name, 0) + epsilon
                
                bin_psi = (actual_pct - expected_pct) * np.log(actual_pct / expected_pct) if month != base_month else 0.0
                
                # Get bin range information
                bin_ranges = binning_info[feature]['bin_ranges']
                if bin_name in bin_ranges:
                    bin_min = bin_ranges[bin_name]['min']
                    bin_max = bin_ranges[bin_name]['max']
                    bin_range = bin_ranges[bin_name]['range_str']
                else:
                    # For categorical or special bins (Missing, Others)
                    bin_min = None
                    bin_max = None
                    bin_range = bin_name
                
                results.append({
                    'Feature': feature,
                    'Feature_Type': binning_info[feature]['type'],
                    'Segment_Column': 'Overall',
                    'Segment_Value': 'All',
                    'Month': f"{month}",
                    'Base_Month': base_month,
                    'Current_Month': month,
                    'Bin': bin_name,
                    'Bin_Range': bin_range,
                    'Bin_Min': bin_min,
                    'Bin_Max': bin_max,
                    'Base_Percentage': (base_dist.get(bin_name, 0) * 100),
                    'Actual_Percentage': (actual_dist.get(bin_name, 0) * 100),
                    'Bin_PSI': bin_psi
                })
    
    # Calculate bin-level PSI by segments
    for segment_col in segment_columns:
        if segment_col not in df.columns:
            continue
        
        segments = df[segment_col].dropna().unique()
        
        for segment_val in segments:
            segment_df = df[df[segment_col] == segment_val]
            
            for feature in feature_list:
                if feature not in df.columns:
                    continue
                
                # Get base month distribution for segment
                base_segment = segment_df[segment_df[month_col] == base_month]
                if len(base_segment) == 0:
                    continue
                
                base_dist = base_segment[f'{feature}_binned'].value_counts(normalize=True)
                
                # Calculate bin-level PSI for each month (including base month)
                for month in months:
                    actual_segment = segment_df[segment_df[month_col] == month]
                    if len(actual_segment) == 0:
                        continue
                    
                    actual_dist = actual_segment[f'{feature}_binned'].value_counts(normalize=True)
                    
                    # Get all bins
                    all_bins = base_dist.index.union(actual_dist.index)
                    
                    for bin_name in all_bins:
                        expected_pct = base_dist.get(bin_name, 0) + epsilon
                        actual_pct = actual_dist.get(bin_name, 0) + epsilon
                        
                        bin_psi = (actual_pct - expected_pct) * np.log(actual_pct / expected_pct) if month != base_month else 0.0
                        
                        # Get bin range information
                        bin_ranges = binning_info[feature]['bin_ranges']
                        if bin_name in bin_ranges:
                            bin_min = bin_ranges[bin_name]['min']
                            bin_max = bin_ranges[bin_name]['max']
                            bin_range = bin_ranges[bin_name]['range_str']
                        else:
                            # For categorical or special bins (Missing, Others)
                            bin_min = None
                            bin_max = None
                            bin_range = bin_name
                        
                        results.append({
                            'Feature': feature,
                            'Feature_Type': binning_info[feature]['type'],
                            'Segment_Column': segment_col,
                            'Segment_Value': segment_val,
                            'Month': f"{month}",
                            'Base_Month': base_month,
                            'Current_Month': month,
                            'Bin': bin_name,
                            'Bin_Range': bin_range,
                            'Bin_Min': bin_min,
                            'Bin_Max': bin_max,
                            'Base_Percentage': (base_dist.get(bin_name, 0) * 100),
                            'Actual_Percentage': (actual_dist.get(bin_name, 0) * 100),
                            'Bin_PSI': bin_psi
                        })
    
    return pd.DataFrame(results)

## PSI function old

In [6]:
# import pandas as pd
# import numpy as np
# from typing import List, Dict, Tuple
# import warnings
# warnings.filterwarnings('ignore')

# def identify_feature_types(df: pd.DataFrame, feature_list: List[str]) -> Dict[str, List[str]]:
#     """
#     Identify categorical and numerical features from the feature list.
    
#     Parameters:
#     -----------
#     df : pd.DataFrame
#         Input dataframe
#     feature_list : List[str]
#         List of features to classify
    
#     Returns:
#     --------
#     Dict with 'categorical' and 'numerical' keys containing respective feature lists
#     """
#     categorical_features = []
#     numerical_features = []
    
#     for feature in feature_list:
#         if feature not in df.columns:
#             print(f"Warning: Feature '{feature}' not found in dataframe")
#             continue
            
#         # Check if feature is numeric
#         if pd.api.types.is_numeric_dtype(df[feature]):
#             # If unique values are less than 15 and all integers, treat as categorical
#             unique_vals = df[feature].nunique()
#             if unique_vals < 15 and df[feature].dropna().apply(lambda x: x == int(x) if isinstance(x, (int, float)) else False).all():
#                 categorical_features.append(feature)
#             else:
#                 numerical_features.append(feature)
#         else:
#             categorical_features.append(feature)
    
#     return {
#         'categorical': categorical_features,
#         'numerical': numerical_features
#     }


# def create_bins_for_features(df: pd.DataFrame, 
#                              numerical_features: List[str],
#                              categorical_features: List[str],
#                              base_month: str,
#                              month_col: str = 'Application_month') -> Dict:
#     """
#     Create bins for numerical features (deciles) and categorical features (top 6 + others).
    
#     Parameters:
#     -----------
#     df : pd.DataFrame
#         Input dataframe
#     numerical_features : List[str]
#         List of numerical features
#     categorical_features : List[str]
#         List of categorical features
#     base_month : str
#         Base month to determine binning strategy
#     month_col : str
#         Name of month column
    
#     Returns:
#     --------
#     Dictionary containing binning information for each feature
#     """
#     base_df = df[df[month_col] == base_month].copy()
#     binning_info = {}
    
#     # Create bins for numerical features (deciles)
#     for feature in numerical_features:
#         valid_data = base_df[feature].dropna()
        
#         if len(valid_data) == 0:
#             binning_info[feature] = {'type': 'numerical', 'bins': None}
#             continue
        
#         # Create decile bins
#         try:
#             bins = np.percentile(valid_data, np.arange(0, 101, 10))
#             # Remove duplicates and sort
#             bins = np.unique(bins)
#             binning_info[feature] = {
#                 'type': 'numerical',
#                 'bins': bins
#             }
#         except:
#             binning_info[feature] = {'type': 'numerical', 'bins': None}
    
#     # Create bins for categorical features (top 6 + others)
#     for feature in categorical_features:
#         value_counts = base_df[feature].value_counts()
#         top_6 = value_counts.nlargest(6).index.tolist()
        
#         binning_info[feature] = {
#             'type': 'categorical',
#             'top_categories': top_6
#         }
    
#     return binning_info


# def apply_binning(df: pd.DataFrame, 
#                   feature: str, 
#                   binning_info: Dict) -> pd.Series:
#     """
#     Apply binning to a feature based on binning information.
    
#     Parameters:
#     -----------
#     df : pd.DataFrame
#         Input dataframe
#     feature : str
#         Feature name
#     binning_info : Dict
#         Binning information for the feature
    
#     Returns:
#     --------
#     pd.Series with binned values
#     """
#     if binning_info['type'] == 'numerical':
#         if binning_info['bins'] is None:
#             return pd.Series(['Missing'] * len(df), index=df.index)
        
#         bins = binning_info['bins']
#         labels = [f"Bin_{i+1}" for i in range(len(bins)-1)]
        
#         binned = pd.cut(df[feature], 
#                        bins=bins, 
#                        labels=labels, 
#                        include_lowest=True,
#                        duplicates='drop')
        
#         # Handle nulls
#         binned = binned.astype(str)
#         binned[df[feature].isna()] = 'Missing'
        
#         return binned
    
#     else:  # categorical
#         top_cats = binning_info['top_categories']
#         binned = df[feature].apply(lambda x: x if x in top_cats else 'Others')
#         binned = binned.fillna('Missing')
        
#         return binned


# def calculate_psi(expected_pct: pd.Series, 
#                   actual_pct: pd.Series, 
#                   epsilon: float = 0.0001) -> float:
#     """
#     Calculate Population Stability Index.
    
#     Parameters:
#     -----------
#     expected_pct : pd.Series
#         Expected (baseline) percentages
#     actual_pct : pd.Series
#         Actual percentages
#     epsilon : float
#         Small value to avoid log(0)
    
#     Returns:
#     --------
#     PSI value
#     """
#     # Align indices
#     all_bins = expected_pct.index.union(actual_pct.index)
#     expected_pct = expected_pct.reindex(all_bins, fill_value=0)
#     actual_pct = actual_pct.reindex(all_bins, fill_value=0)
    
#     # Add epsilon to avoid log(0)
#     expected_pct = expected_pct + epsilon
#     actual_pct = actual_pct + epsilon
    
#     # Calculate PSI
#     psi_value = np.sum((actual_pct - expected_pct) * np.log(actual_pct / expected_pct))
    
#     return psi_value


# def calculate_month_on_month_psi(df: pd.DataFrame,
#                                  feature_list: List[str],
#                                  segment_columns: List[str],
#                                  month_col: str = 'Application_month') -> pd.DataFrame:
#     """
#     Calculate month-on-month PSI for each feature, overall and by segments.
#     Each row shows PSI comparing first month vs each subsequent month.
    
#     Parameters:
#     -----------
#     df : pd.DataFrame
#         Input dataframe
#     feature_list : List[str]
#         List of features to calculate PSI for
#     segment_columns : List[str]
#         List of segment columns
#     month_col : str
#         Name of month column
    
#     Returns:
#     --------
#     pd.DataFrame with PSI values with one row per feature-month-segment combination
#     """
#     # Identify feature types
#     feature_types = identify_feature_types(df, feature_list)
    
#     # Get sorted months
#     months = sorted(df[month_col].unique())
#     base_month = months[0]
    
#     # Create binning strategy based on base month
#     binning_info = create_bins_for_features(
#         df, 
#         feature_types['numerical'],
#         feature_types['categorical'],
#         base_month,
#         month_col
#     )
    
#     results = []
    
#     # Calculate overall PSI
#     for feature in feature_list:
#         if feature not in df.columns:
#             continue
        
#         # Apply binning
#         df[f'{feature}_binned'] = apply_binning(df, feature, binning_info[feature])
        
#         # Get base month distribution
#         base_dist = df[df[month_col] == base_month][f'{feature}_binned'].value_counts(normalize=True)
        
#         # For each month (including base month for reference)
#         for month in months:
#             actual_dist = df[df[month_col] == month][f'{feature}_binned'].value_counts(normalize=True)
#             psi_value = calculate_psi(base_dist, actual_dist) if month != base_month else 0.0
            
#             results.append({
#                 'Feature': feature,
#                 'Feature_Type': binning_info[feature]['type'],
#                 'Segment_Column': 'Overall',
#                 'Segment_Value': 'All',
#                 'Month': f"{month}",
#                 'Base_Month': base_month,
#                 'Current_Month': month,
#                 'PSI': psi_value
#             })
    
#     # Calculate PSI by segments
#     for segment_col in segment_columns:
#         if segment_col not in df.columns:
#             continue
        
#         segments = df[segment_col].dropna().unique()
        
#         for segment_val in segments:
#             segment_df = df[df[segment_col] == segment_val]
            
#             for feature in feature_list:
#                 if feature not in df.columns:
#                     continue
                
#                 # Get base month distribution for segment
#                 base_segment = segment_df[segment_df[month_col] == base_month]
#                 if len(base_segment) == 0:
#                     continue
                
#                 base_dist = base_segment[f'{feature}_binned'].value_counts(normalize=True)
                
#                 # For each month (including base month for reference)
#                 for month in months:
#                     actual_segment = segment_df[segment_df[month_col] == month]
#                     if len(actual_segment) == 0:
#                         continue
                    
#                     actual_dist = actual_segment[f'{feature}_binned'].value_counts(normalize=True)
#                     psi_value = calculate_psi(base_dist, actual_dist) if month != base_month else 0.0
                    
#                     results.append({
#                         'Feature': feature,
#                         'Feature_Type': binning_info[feature]['type'],
#                         'Segment_Column': segment_col,
#                         'Segment_Value': segment_val,
#                         'Month': f"{month}",
#                         'Base_Month': base_month,
#                         'Current_Month': month,
#                         'PSI': psi_value
#                     })
    
#     # Clean up temporary columns
#     for feature in feature_list:
#         if f'{feature}_binned' in df.columns:
#             df.drop(f'{feature}_binned', axis=1, inplace=True)
    
#     return pd.DataFrame(results)


# def calculate_bin_level_psi(df: pd.DataFrame,
#                             feature_list: List[str],
#                             segment_columns: List[str],
#                             month_col: str = 'Application_month') -> pd.DataFrame:
#     """
#     Calculate bin-level PSI for each feature, overall and by segments.
#     Each row shows bin-level PSI comparing first month vs each month.
    
#     Parameters:
#     -----------
#     df : pd.DataFrame
#         Input dataframe
#     feature_list : List[str]
#         List of features to calculate PSI for
#     segment_columns : List[str]
#         List of segment columns
#     month_col : str
#         Name of month column
    
#     Returns:
#     --------
#     pd.DataFrame with bin-level PSI details
#     """
#     # Identify feature types
#     feature_types = identify_feature_types(df, feature_list)
    
#     # Get sorted months
#     months = sorted(df[month_col].unique())
#     base_month = months[0]
    
#     # Create binning strategy based on base month
#     binning_info = create_bins_for_features(
#         df, 
#         feature_types['numerical'],
#         feature_types['categorical'],
#         base_month,
#         month_col
#     )
    
#     results = []
#     epsilon = 0.0001
    
#     # Calculate overall bin-level PSI
#     for feature in feature_list:
#         if feature not in df.columns:
#             continue
        
#         # Apply binning
#         df[f'{feature}_binned'] = apply_binning(df, feature, binning_info[feature])
        
#         # Get base month distribution
#         base_dist = df[df[month_col] == base_month][f'{feature}_binned'].value_counts(normalize=True)
        
#         # Calculate bin-level PSI for each month (including base month)
#         for month in months:
#             actual_dist = df[df[month_col] == month][f'{feature}_binned'].value_counts(normalize=True)
            
#             # Get all bins
#             all_bins = base_dist.index.union(actual_dist.index)
            
#             for bin_name in all_bins:
#                 expected_pct = base_dist.get(bin_name, 0) + epsilon
#                 actual_pct = actual_dist.get(bin_name, 0) + epsilon
                
#                 bin_psi = (actual_pct - expected_pct) * np.log(actual_pct / expected_pct) if month != base_month else 0.0
                
#                 results.append({
#                     'Feature': feature,
#                     'Feature_Type': binning_info[feature]['type'],
#                     'Segment_Column': 'Overall',
#                     'Segment_Value': 'All',
#                     'Month': f"{month}",
#                     'Base_Month': base_month,
#                     'Current_Month': month,
#                     'Bin': bin_name,
#                     'Base_Percentage': (base_dist.get(bin_name, 0) * 100),
#                     'Actual_Percentage': (actual_dist.get(bin_name, 0) * 100),
#                     'Bin_PSI': bin_psi
#                 })
    
#     # Calculate bin-level PSI by segments
#     for segment_col in segment_columns:
#         if segment_col not in df.columns:
#             continue
        
#         segments = df[segment_col].dropna().unique()
        
#         for segment_val in segments:
#             segment_df = df[df[segment_col] == segment_val]
            
#             for feature in feature_list:
#                 if feature not in df.columns:
#                     continue
                
#                 # Get base month distribution for segment
#                 base_segment = segment_df[segment_df[month_col] == base_month]
#                 if len(base_segment) == 0:
#                     continue
                
#                 base_dist = base_segment[f'{feature}_binned'].value_counts(normalize=True)
                
#                 # Calculate bin-level PSI for each month (including base month)
#                 for month in months:
#                     actual_segment = segment_df[segment_df[month_col] == month]
#                     if len(actual_segment) == 0:
#                         continue
                    
#                     actual_dist = actual_segment[f'{feature}_binned'].value_counts(normalize=True)
                    
#                     # Get all bins
#                     all_bins = base_dist.index.union(actual_dist.index)
                    
#                     for bin_name in all_bins:
#                         expected_pct = base_dist.get(bin_name, 0) + epsilon
#                         actual_pct = actual_dist.get(bin_name, 0) + epsilon
                        
#                         bin_psi = (actual_pct - expected_pct) * np.log(actual_pct / expected_pct) if month != base_month else 0.0
                        
#                         results.append({
#                             'Feature': feature,
#                             'Feature_Type': binning_info[feature]['type'],
#                             'Segment_Column': segment_col,
#                             'Segment_Value': segment_val,
#                             'Month': f"{base_month} vs {month}",
#                             'Base_Month': base_month,
#                             'Current_Month': month,
#                             'Bin': bin_name,
#                             'Base_Percentage': (base_dist.get(bin_name, 0) * 100),
#                             'Actual_Percentage': (actual_dist.get(bin_name, 0) * 100),
#                             'Bin_PSI': bin_psi
#                         })
    
#     # Clean up temporary columns
#     for feature in feature_list:
#         if f'{feature}_binned' in df.columns:
#             df.drop(f'{feature}_binned', axis=1, inplace=True)
    
#     return pd.DataFrame(results)




In [7]:
# sq = """drop table if exists prj-prod-dataplatform.dap_ds_poweruser_playground.temp_csi_new_monitoring_data;"""

# job = client.query(sq)
# job.result()  # Wait for job to complete
# print(f"Table  prj-prod-dataplatform.dap_ds_poweruser_playground.temp_csi_new_monitoring_data dropped successfully.")

# Queries

## Alpha-Cash-CIC-Model (All Trench)

## Trench 1

In [8]:
sq = r"""  
WITH parsed as (
  select customerId, digitalLoanAccountId,modelDisplayName,modelVersionId,start_time,end_time,prediction,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
--REPLACE(REPLACE(prediction, "'", '"'), "None", "null") AS prediction_clean
FROM `prj-prod-dataplatform.audit_balance.ml_model_run_details`
where modelDisplayName = 'Alpha-Cash-CIC-Model'
),
latest_request as (
select * from parsed
QUALIFY ROW_NUMBER() OVER (PARTITION BY customerId, digitalLoanAccountId,modelDisplayName ORDER BY start_time DESC ) = 1),

model_run as (
select customerId,digitalLoanAccountId,modelName, publish_time,requestPayload as requestPayload_clean
--REPLACE(REPLACE(requestPayload, "'", '"'), "None", "null") AS requestPayload_clean
from `prj-prod-dataplatform.audit_balance.ml_request_details` 
WHERE modelName = 'Alpha-Cash-Model-response'
QUALIFY ROW_NUMBER() OVER (PARTITION BY customerId, digitalLoanAccountId,modelName ORDER BY publish_time DESC ) = 1),
base as (
select * from (
  select 
 r.customerId,
 r.digitalLoanAccountId,
 r.prediction,
 r.start_time,
 r.end_time,
 r.modelDisplayName,
 r.modelVersionId,
 loanmaster.new_loan_type,
 loanmaster.gender,
 'Cash' as product,
 'Alpha-Cash-CIC-Model_All_Trench' Model_Name,
  REGEXP_EXTRACT(m.requestPayload_clean, r"osType[:=]['\"]?([^'\"]+)['\"]?") AS osType,
  REGEXP_EXTRACT(m.requestPayload_clean, r"loanType[:=]['\"]?([^'\"]+)['\"]?") AS loanType,
  REGEXP_EXTRACT(m.requestPayload_clean, r"trenchCategory[:=]['\"]?([^'\"]+)['\"]?") AS trenchCategory,
   SAFE_CAST(REGEXP_EXTRACT(m.requestPayload_clean, r"aStackScore[:= ]([0-9\.]+)") AS FLOAT64) AS aStackScore,
  SAFE_CAST(REGEXP_EXTRACT(m.requestPayload_clean, r"aCicScore[:= ]([0-9\.]+)") AS FLOAT64) AS aCicScore,
  calcFeatures,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  r.start_time) AS appln_submit_datetime,
  loanmaster.disbursementDateTime,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  r.start_time)) as Application_month, 
FROM latest_request r
left join model_run m
on r.digitalLoanAccountId = m.digitalLoanAccountId 
left join risk_credit_mis.loan_master_table loanmaster 
  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
qualify row_number() over (partition by r.customerId,r.digitalLoanAccountId order by r.start_time desc) = 1
 )
)
select * from base where trenchCategory = 'Trench 1';
"""
dfd = client.query(sq).to_dataframe()
dfd.head()

,customerId,digitalLoanAccountId,prediction,start_time,end_time,modelDisplayName,modelVersionId,new_loan_type,gender,product,Model_Name,osType,loanType,trenchCategory,aStackScore,aCicScore,calcFeatures,appln_submit_datetime,disbursementDateTime,Application_month
0,3709317,bb7415c6-63f5-4798-94bd-178a2eddecae,0.42580333948299337,2025-09-27 12:30:54.283162,2025-09-27 12:30:54.453205,Alpha-Cash-CIC-Model,v1,Quick,M,Cash,Alpha-Cash-CIC-Model_All_Trench,android,Quick,Trench 1,0.287180,0.425803,"{""cic_max_age_all_contracts_snapshot"": 3004.0,...",2025-09-27 20:30:38,2025-09-27 21:02:46,2025-09
1,3717047,5e293883-ab08-4e9b-8abd-5a82f864a1c0,0.38931915993332805,2025-10-01 08:06:25.190052,2025-10-01 08:06:25.769685,Alpha-Cash-CIC-Model,v1,Quick,F,Cash,Alpha-Cash-CIC-Model_All_Trench,android,Quick,Trench 1,0.505667,0.389319,"{""cic_max_age_all_contracts_snapshot"": 731.0, ...",2025-10-01 16:06:11,NaT,2025-10
2,3729123,baf85e10-ac20-4757-98ff-83e3181bbb4b,0.761000545430151,2025-10-07 04:42:00.815129,2025-10-07 04:42:00.994801,Alpha-Cash-CIC-Model,v1,Quick,F,Cash,Alpha-Cash-CIC-Model_All_Trench,android,Quick,Trench 1,0.815492,0.761001,"{""cic_max_age_all_contracts_snapshot"": 366, ""c...",2025-10-07 12:41:48,NaT,2025-10
3,3730693,0ecdf071-af0b-4078-956e-42133d56adb8,0.5033200545833454,2025-10-08 00:12:14.151145,2025-10-08 00:12:14.267878,Alpha-Cash-CIC-Model,v1,Quick,F,Cash,Alpha-Cash-CIC-Model_All_Trench,android,Quick,Trench 1,0.572349,0.503320,"{""cic_max_age_all_contracts_snapshot"": 370, ""c...",2025-10-08 08:12:03,NaT,2025-10
4,3734878,693bd415-b8f1-4f4b-80e8-c6201da55e60,0.5919196080408091,2025-10-10 05:32:46.387835,2025-10-10 05:32:46.490734,Alpha-Cash-CIC-Model,v1,Quick,M,Cash,Alpha-Cash-CIC-Model_All_Trench,android,Quick,Trench 1,0.624343,0.591920,"{""cic_max_age_all_contracts_snapshot"": 274, ""c...",2025-10-10 13:32:36,NaT,2025-10


### expand df

In [9]:
# Expand the calcFeatures column
expanded_df = expand_calc_features(dfd)

# Display the result
print(f"Original columns: {dfd.shape[1]}")
print(f"Expanded columns: {expanded_df.shape[1]}")
expanded_df.head()


Original columns: 20
Expanded columns: 33


,customerId,digitalLoanAccountId,prediction,start_time,end_time,modelDisplayName,modelVersionId,new_loan_type,gender,product,Model_Name,osType,loanType,trenchCategory,aStackScore,aCicScore,calcFeatures,appln_submit_datetime,disbursementDateTime,Application_month,calc_cic_max_age_all_contracts_snapshot,calc_cic_ratio_overdue_contracts_to_granted_contracts,calc_cic_ScoreRange,calc_cic_ln_loan_level_user_type,calc_cic_has_ever_been_overdue,calc_cic_latest_granted_contract_overdue_flag,calc_cic_ratio_closed_over_new_granted_cnt_24M,calc_cic_ratio_risky_contracts_to_granted_contracts,calc_cic_Short_and_Term_Loans_granted_contracts_cnt_24M,calc_cic_flg_zero_non_granted_ever,calc_cic_Personal_Loans_granted_contracts_amt_24M,calc_cic_CreditAvgCreditLimit,calc_cic_flg_zero_granted_ever
0,3709317,bb7415c6-63f5-4798-94bd-178a2eddecae,0.42580333948299337,2025-09-27 12:30:54.283162,2025-09-27 12:30:54.453205,Alpha-Cash-CIC-Model,v1,Quick,M,Cash,Alpha-Cash-CIC-Model_All_Trench,android,Quick,Trench 1,0.287180,0.425803,"{""cic_max_age_all_contracts_snapshot"": 3004.0,...",2025-09-27 20:30:38,2025-09-27 21:02:46,2025-09,3004.0,0.032258,Unknown,2_New Applicant,1.0,0.0,0.612903,0.000000,14.0,0,57787.0,92400.0,0
1,3717047,5e293883-ab08-4e9b-8abd-5a82f864a1c0,0.38931915993332805,2025-10-01 08:06:25.190052,2025-10-01 08:06:25.769685,Alpha-Cash-CIC-Model,v1,Quick,F,Cash,Alpha-Cash-CIC-Model_All_Trench,android,Quick,Trench 1,0.505667,0.389319,"{""cic_max_age_all_contracts_snapshot"": 731.0, ...",2025-10-01 16:06:11,NaT,2025-10,731.0,0.000000,Bi,2_New Applicant,0.0,0.0,1.064748,0.000000,80.0,0,352631.0,0.0,0
2,3729123,baf85e10-ac20-4757-98ff-83e3181bbb4b,0.761000545430151,2025-10-07 04:42:00.815129,2025-10-07 04:42:00.994801,Alpha-Cash-CIC-Model,v1,Quick,F,Cash,Alpha-Cash-CIC-Model_All_Trench,android,Quick,Trench 1,0.815492,0.761001,"{""cic_max_age_all_contracts_snapshot"": 366, ""c...",2025-10-07 12:41:48,NaT,2025-10,366.0,0.619048,Unknown,2_New Applicant,1.0,1.0,2.125000,0.619048,4.0,0,9120.0,0.0,0
3,3730693,0ecdf071-af0b-4078-956e-42133d56adb8,0.5033200545833454,2025-10-08 00:12:14.151145,2025-10-08 00:12:14.267878,Alpha-Cash-CIC-Model,v1,Quick,F,Cash,Alpha-Cash-CIC-Model_All_Trench,android,Quick,Trench 1,0.572349,0.503320,"{""cic_max_age_all_contracts_snapshot"": 370, ""c...",2025-10-08 08:12:03,NaT,2025-10,370.0,0.000000,Ai,2_New Applicant,0.0,0.0,0.750000,0.000000,NaN,1,NaN,0.0,0
4,3734878,693bd415-b8f1-4f4b-80e8-c6201da55e60,0.5919196080408091,2025-10-10 05:32:46.387835,2025-10-10 05:32:46.490734,Alpha-Cash-CIC-Model,v1,Quick,M,Cash,Alpha-Cash-CIC-Model_All_Trench,android,Quick,Trench 1,0.624343,0.591920,"{""cic_max_age_all_contracts_snapshot"": 274, ""c...",2025-10-10 13:32:36,NaT,2025-10,274.0,0.000000,Unknown,2_New Applicant,0.0,0.0,NaN,0.333333,NaN,0,NaN,0.0,0


In [10]:
# sq = r"""  
# WITH parsed as (
#   select customerId, digitalLoanAccountId,modelDisplayName,modelVersionId,start_time,end_time,prediction,
# REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
# --REPLACE(REPLACE(prediction, "'", '"'), "None", "null") AS prediction_clean
# FROM `prj-prod-dataplatform.audit_balance.ml_model_run_details`
# where modelDisplayName = 'Alpha-Cash-CIC-Model'
# ),

# latest_request as (
# select * from parsed
# QUALIFY ROW_NUMBER() OVER (PARTITION BY customerId, digitalLoanAccountId,modelDisplayName ORDER BY start_time DESC ) = 1),

# model_run as (
# select customerId,digitalLoanAccountId,modelName, publish_time,requestPayload as requestPayload_clean
# --REPLACE(REPLACE(requestPayload, "'", '"'), "None", "null") AS requestPayload_clean
# from `prj-prod-dataplatform.audit_balance.ml_request_details` 
# WHERE modelName = 'Alpha-Cash-Model-response'
# QUALIFY ROW_NUMBER() OVER (PARTITION BY customerId, digitalLoanAccountId,modelName ORDER BY publish_time DESC ) = 1),
# base as (
# select * from (
#   select 
#  r.customerId,
#  r.digitalLoanAccountId,
#  r.prediction,
#  r.start_time,
#  r.end_time,
#  r.modelDisplayName,
#  r.modelVersionId,
#  loanmaster.new_loan_type,
#  loanmaster.gender,
#   REGEXP_EXTRACT(m.requestPayload_clean, r"osType[:=]['\"]?([^'\"]+)['\"]?") AS osType,
#   REGEXP_EXTRACT(m.requestPayload_clean, r"loanType[:=]['\"]?([^'\"]+)['\"]?") AS loanType,
#   REGEXP_EXTRACT(m.requestPayload_clean, r"trenchCategory[:=]['\"]?([^'\"]+)['\"]?") AS trenchCategory,
#    SAFE_CAST(REGEXP_EXTRACT(m.requestPayload_clean, r"aStackScore[:= ]([0-9\.]+)") AS FLOAT64) AS aStackScore,
#   SAFE_CAST(REGEXP_EXTRACT(m.requestPayload_clean, r"aCicScore[:= ]([0-9\.]+)") AS FLOAT64) AS aCicScore,
#   --  Alpha CIC Score Model Features for Trench 1
#   SAFE_CAST(JSON_VALUE(r.calcFeatures, "$.cic_max_age_all_contracts_snapshot") AS INT64) AS cic_max_age_all_contracts_snapshot,
#   SAFE_CAST(JSON_VALUE(r.calcFeatures, "$.cic_ratio_overdue_contracts_to_granted_contracts") AS FLOAT64) AS cic_ratio_overdue_contracts_to_granted_contracts,
#   JSON_VALUE(r.calcFeatures, "$.cic_ScoreRange") AS cic_ScoreRange,
#   JSON_VALUE(r.calcFeatures, "$.cic_ln_loan_level_user_type") AS cic_ln_loan_level_user_type,
#   JSON_VALUE(r.calcFeatures, "$.cic_has_ever_been_overdue") AS cic_has_ever_been_overdue,
#   JSON_VALUE(r.calcFeatures, "$.cic_latest_granted_contract_overdue_flag") AS cic_latest_granted_contract_overdue_flag,
#   JSON_VALUE(r.calcFeatures, "$.cic_ratio_closed_over_new_granted_cnt_24M") AS cic_ratio_closed_over_new_granted_cnt_24M,
#   JSON_VALUE(r.calcFeatures, "$.cic_ratio_risky_contracts_to_granted_contracts") AS cic_ratio_risky_contracts_to_granted_contracts,
#   JSON_VALUE(r.calcFeatures, "$.cic_Short_and_Term_Loans_granted_contracts_cnt_24M") AS cic_Short_and_Term_Loans_granted_contracts_cnt_24M,
#   JSON_VALUE(r.calcFeatures, "$.cic_flg_zero_non_granted_ever") AS cic_flg_zero_non_granted_ever,
#   JSON_VALUE(r.calcFeatures, "$.cic_Personal_Loans_granted_contracts_amt_24M") AS cic_Personal_Loans_granted_contracts_amt_24M,
#   JSON_VALUE(r.calcFeatures, "$.cic_CreditAvgCreditLimit") AS cic_CreditAvgCreditLimit,
#   JSON_VALUE(r.calcFeatures, "$.cic_flg_zero_granted_ever") AS cic_flg_zero_granted_ever,
#   coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  r.start_time) AS appln_submit_datetime,
#   loanmaster.disbursementDateTime,
#   format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  r.start_time)) as Application_month, 
# FROM latest_request r
# left join model_run m
# on r.digitalLoanAccountId = m.digitalLoanAccountId 
# left join risk_credit_mis.loan_master_table loanmaster 
#   ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
#  )
# where trenchCategory = 'Trench 1'
# )
# select *, case when appln_submit_datetime <= '2025-09-30' then 'Train' else 'Test' end dataselection from base
# ;
# """

# df = client.query(sq).to_dataframe()
# df.head()

### copy to df

In [11]:
df = expanded_df.drop(columns=['calcFeatures']).copy()

### Drop table temp_csi_new_monitoring_data

In [12]:
sq = """drop table if exists prj-prod-dataplatform.dap_ds_poweruser_playground.temp_csi_new_monitoring_data;"""

job = client.query(sq)
job.result()  # Wait for job to complete
print(f"Table  prj-prod-dataplatform.dap_ds_poweruser_playground.temp_csi_new_monitoring_data dropped successfully.")

sq = """drop table if exists prj-prod-dataplatform.dap_ds_poweruser_playground.temp_csi_new_monitoring_data_feature_bin_level;"""

job = client.query(sq)
job.result()  # Wait for job to complete
print(f"Table  prj-prod-dataplatform.dap_ds_poweruser_playground.temp_csi_new_monitoring_data_feature_bin_level dropped successfully.")

Table  prj-prod-dataplatform.dap_ds_poweruser_playground.temp_csi_new_monitoring_data dropped successfully.
Table  prj-prod-dataplatform.dap_ds_poweruser_playground.temp_csi_new_monitoring_data_feature_bin_level dropped successfully.


### Run the PSI calculation

In [13]:
df = df.copy()

# Define feature list
feature_list = ['aCicScore',
     'calc_cic_max_age_all_contracts_snapshot',
       'calc_cic_ratio_overdue_contracts_to_granted_contracts',
       'calc_cic_ScoreRange', 'calc_cic_ln_loan_level_user_type',
       'calc_cic_has_ever_been_overdue',
       'calc_cic_latest_granted_contract_overdue_flag',
       'calc_cic_ratio_closed_over_new_granted_cnt_24M',
       'calc_cic_ratio_risky_contracts_to_granted_contracts',
       'calc_cic_Short_and_Term_Loans_granted_contracts_cnt_24M',
       'calc_cic_flg_zero_non_granted_ever',
       'calc_cic_Personal_Loans_granted_contracts_amt_24M',
       'calc_cic_CreditAvgCreditLimit', 'calc_cic_flg_zero_granted_ever'
]

# Define segment columns
segment_columns = ['new_loan_type', 'gender', 'osType', 'trenchCategory', 'product']
# Calculate month-on-month PSI
psi_results = calculate_month_on_month_psi(df, feature_list, segment_columns)
psi_results['modelDisplayName'] = df['modelDisplayName'].iloc[0]
psi_results['Model_Name'] = df['Model_Name'].iloc[0]
psi_results['modelVersionId'] = df['modelVersionId'].iloc[0]
psi_results['trenchCategory'] = df['trenchCategory'].iloc[0]
psi_results = psi_results[[ 'modelDisplayName', 'Model_Name',  'modelVersionId', 'trenchCategory','Feature', 'Feature_Type', 'Segment_Column', 'Segment_Value', 'Month',
       'Base_Month', 'Current_Month', 'PSI']].copy()

# Calculate bin-level PSI
bin_psi_results = calculate_bin_level_psi(df, feature_list, segment_columns)
bin_psi_results['modelDisplayName'] = df['modelDisplayName'].iloc[0]
bin_psi_results['Model_Name'] = df['Model_Name'].iloc[0]
bin_psi_results['modelVersionId'] = df['modelVersionId'].iloc[0]
bin_psi_results['trenchCategory'] = df['trenchCategory'].iloc[0]
bin_psi_results = bin_psi_results[[ 'modelDisplayName', 'Model_Name',  'modelVersionId', 'trenchCategory','Feature', 'Feature_Type', 'Segment_Column', 'Segment_Value', 'Month',
       'Base_Month', 'Current_Month', 'Bin', 'Bin_Range', 'Bin_Min', 'Bin_Max', 'Base_Percentage',
       'Actual_Percentage', 'Bin_PSI']].copy()

# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.temp_csi_new_monitoring_data"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_TRUNCATE",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(psi_results, table_id, job_config=job_config)
job.result()  # Wait for the job to complete


# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.temp_csi_new_monitoring_data_feature_bin_level"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_TRUNCATE",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(bin_psi_results, table_id, job_config=job_config)
job.result()  # Wait for the job to complete

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=6e97fc97-dc95-4e09-bf26-265dee795cdf>

In [14]:
bin_psi_results.columns

Index(['modelDisplayName', 'Model_Name', 'modelVersionId', 'trenchCategory',
       'Feature', 'Feature_Type', 'Segment_Column', 'Segment_Value', 'Month',
       'Base_Month', 'Current_Month', 'Bin', 'Bin_Range', 'Bin_Min', 'Bin_Max',
       'Base_Percentage', 'Actual_Percentage', 'Bin_PSI'],
      dtype='object')

## Trench 2

In [15]:
sq = r"""  
WITH parsed as (
  select customerId, digitalLoanAccountId,modelDisplayName,modelVersionId,start_time,end_time,prediction,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
--REPLACE(REPLACE(prediction, "'", '"'), "None", "null") AS prediction_clean
FROM `prj-prod-dataplatform.audit_balance.ml_model_run_details`
where modelDisplayName = 'Alpha-Cash-CIC-Model'
),
latest_request as (
select * from parsed
QUALIFY ROW_NUMBER() OVER (PARTITION BY customerId, digitalLoanAccountId,modelDisplayName ORDER BY start_time DESC ) = 1),

model_run as (
select customerId,digitalLoanAccountId,modelName, publish_time,requestPayload as requestPayload_clean
--REPLACE(REPLACE(requestPayload, "'", '"'), "None", "null") AS requestPayload_clean
from `prj-prod-dataplatform.audit_balance.ml_request_details` 
WHERE modelName = 'Alpha-Cash-Model-response'
QUALIFY ROW_NUMBER() OVER (PARTITION BY customerId, digitalLoanAccountId,modelName ORDER BY publish_time DESC ) = 1),
base as (
select * from (
  select 
 r.customerId,
 r.digitalLoanAccountId,
 r.prediction,
 r.start_time,
 r.end_time,
 r.modelDisplayName,
 r.modelVersionId,
 loanmaster.new_loan_type,
 loanmaster.gender,
 'Cash' as product,
 'Alpha-Cash-CIC-Model_All_Trench' Model_Name,
  REGEXP_EXTRACT(m.requestPayload_clean, r"osType[:=]['\"]?([^'\"]+)['\"]?") AS osType,
  REGEXP_EXTRACT(m.requestPayload_clean, r"loanType[:=]['\"]?([^'\"]+)['\"]?") AS loanType,
  REGEXP_EXTRACT(m.requestPayload_clean, r"trenchCategory[:=]['\"]?([^'\"]+)['\"]?") AS trenchCategory,
   SAFE_CAST(REGEXP_EXTRACT(m.requestPayload_clean, r"aStackScore[:= ]([0-9\.]+)") AS FLOAT64) AS aStackScore,
  SAFE_CAST(REGEXP_EXTRACT(m.requestPayload_clean, r"aCicScore[:= ]([0-9\.]+)") AS FLOAT64) AS aCicScore,
  calcFeatures,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  r.start_time) AS appln_submit_datetime,
  loanmaster.disbursementDateTime,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  r.start_time)) as Application_month, 
FROM latest_request r
left join model_run m
on r.digitalLoanAccountId = m.digitalLoanAccountId 
left join risk_credit_mis.loan_master_table loanmaster 
  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
qualify row_number() over (partition by r.customerId,r.digitalLoanAccountId order by r.start_time desc) = 1
 )
)
select * from base where trenchCategory = 'Trench 2';
"""
dfd = client.query(sq).to_dataframe()
dfd.head()

,customerId,digitalLoanAccountId,prediction,start_time,end_time,modelDisplayName,modelVersionId,new_loan_type,gender,product,Model_Name,osType,loanType,trenchCategory,aStackScore,aCicScore,calcFeatures,appln_submit_datetime,disbursementDateTime,Application_month
0,2910987,4082a166-e7c6-4861-a952-b894475c3599,0.5306874629307167,2025-09-29 07:28:43.706389,2025-09-29 07:28:43.821449,Alpha-Cash-CIC-Model,v1,Quick,F,Cash,Alpha-Cash-CIC-Model_All_Trench,android,Quick,Trench 2,0.647732,0.530687,"{""cic_max_age_all_contracts_snapshot"": 126, ""c...",2025-09-29 15:28:32,NaT,2025-09
1,3432426,a1feddec-f31a-43d3-a7d1-0f1002668abe,0.36567619380284216,2025-10-08 07:03:45.184793,2025-10-08 07:03:45.340379,Alpha-Cash-CIC-Model,v1,Quick,F,Cash,Alpha-Cash-CIC-Model_All_Trench,ios,Quick,Trench 2,0.191612,0.365676,"{""cic_max_age_all_contracts_snapshot"": 2170.0,...",2025-10-08 15:03:22,2025-10-08 21:40:28,2025-10
2,3482360,8ee3c952-7c16-4f8e-a9a8-4dc6f0eab1d5,0.6283851891024527,2025-10-09 11:13:43.019993,2025-10-09 11:13:43.399833,Alpha-Cash-CIC-Model,v1,Quick,F,Cash,Alpha-Cash-CIC-Model_All_Trench,android,Quick,Trench 2,0.572483,0.628385,"{""cic_max_age_all_contracts_snapshot"": 570.0, ...",2025-10-09 19:13:30,NaT,2025-10
3,3524672,ff9f5d93-edfa-434b-b3fe-9eb6fa4239b4,0.4427772244336582,2025-10-10 23:14:26.005026,2025-10-10 23:14:26.105788,Alpha-Cash-CIC-Model,v1,Quick,F,Cash,Alpha-Cash-CIC-Model_All_Trench,android,Quick,Trench 2,0.481568,0.442777,"{""cic_max_age_all_contracts_snapshot"": 374.0, ...",2025-10-11 07:14:15,2025-10-11 09:02:47,2025-10
4,3592340,010b98b6-a8ce-4a5b-b314-212430210d68,0.461272749697465,2025-10-09 03:38:30.315501,2025-10-09 03:38:30.446838,Alpha-Cash-CIC-Model,v1,Quick,M,Cash,Alpha-Cash-CIC-Model_All_Trench,android,Quick,Trench 2,0.309645,0.461273,"{""cic_max_age_all_contracts_snapshot"": 211, ""c...",2025-10-09 11:38:04,2025-10-10 16:41:46,2025-10


### expand df

In [16]:
# Expand the calcFeatures column
expanded_df = expand_calc_features(dfd)

# Display the result
print(f"Original columns: {dfd.shape[1]}")
print(f"Expanded columns: {expanded_df.shape[1]}")
expanded_df.head()


Original columns: 20
Expanded columns: 33


,customerId,digitalLoanAccountId,prediction,start_time,end_time,modelDisplayName,modelVersionId,new_loan_type,gender,product,Model_Name,osType,loanType,trenchCategory,aStackScore,aCicScore,calcFeatures,appln_submit_datetime,disbursementDateTime,Application_month,calc_cic_max_age_all_contracts_snapshot,calc_cic_ratio_overdue_contracts_to_granted_contracts,calc_cic_ScoreRange,calc_cic_ln_loan_level_user_type,calc_cic_has_ever_been_overdue,calc_cic_latest_granted_contract_overdue_flag,calc_cic_ratio_closed_over_new_granted_cnt_24M,calc_cic_ratio_risky_contracts_to_granted_contracts,calc_cic_Short_and_Term_Loans_granted_contracts_cnt_24M,calc_cic_flg_zero_non_granted_ever,calc_cic_Personal_Loans_granted_contracts_amt_24M,calc_cic_CreditAvgCreditLimit,calc_cic_flg_zero_granted_ever
0,2910987,4082a166-e7c6-4861-a952-b894475c3599,0.5306874629307167,2025-09-29 07:28:43.706389,2025-09-29 07:28:43.821449,Alpha-Cash-CIC-Model,v1,Quick,F,Cash,Alpha-Cash-CIC-Model_All_Trench,android,Quick,Trench 2,0.647732,0.530687,"{""cic_max_age_all_contracts_snapshot"": 126, ""c...",2025-09-29 15:28:32,NaT,2025-09,126.0,0.000000,Ai,2_New Applicant,0.0,0.0,1.000000,0.100000,NaN,0,20570.0,0.0,0
1,3432426,a1feddec-f31a-43d3-a7d1-0f1002668abe,0.36567619380284216,2025-10-08 07:03:45.184793,2025-10-08 07:03:45.340379,Alpha-Cash-CIC-Model,v1,Quick,F,Cash,Alpha-Cash-CIC-Model_All_Trench,ios,Quick,Trench 2,0.191612,0.365676,"{""cic_max_age_all_contracts_snapshot"": 2170.0,...",2025-10-08 15:03:22,2025-10-08 21:40:28,2025-10,2170.0,0.000000,Hi,2_New Applicant,0.0,0.0,0.900000,0.000000,NaN,0,25660.0,180000.0,0
2,3482360,8ee3c952-7c16-4f8e-a9a8-4dc6f0eab1d5,0.6283851891024527,2025-10-09 11:13:43.019993,2025-10-09 11:13:43.399833,Alpha-Cash-CIC-Model,v1,Quick,F,Cash,Alpha-Cash-CIC-Model_All_Trench,android,Quick,Trench 2,0.572483,0.628385,"{""cic_max_age_all_contracts_snapshot"": 570.0, ...",2025-10-09 19:13:30,NaT,2025-10,570.0,0.604938,NH_Gi,2_New Applicant,1.0,0.0,1.100000,0.592593,40.0,0,3531.0,0.0,0
3,3524672,ff9f5d93-edfa-434b-b3fe-9eb6fa4239b4,0.4427772244336582,2025-10-10 23:14:26.005026,2025-10-10 23:14:26.105788,Alpha-Cash-CIC-Model,v1,Quick,F,Cash,Alpha-Cash-CIC-Model_All_Trench,android,Quick,Trench 2,0.481568,0.442777,"{""cic_max_age_all_contracts_snapshot"": 374.0, ...",2025-10-11 07:14:15,2025-10-11 09:02:47,2025-10,374.0,0.000000,NH_Fi,2_New Applicant,0.0,0.0,1.000000,0.000000,NaN,1,NaN,146000.0,0
4,3592340,010b98b6-a8ce-4a5b-b314-212430210d68,0.461272749697465,2025-10-09 03:38:30.315501,2025-10-09 03:38:30.446838,Alpha-Cash-CIC-Model,v1,Quick,M,Cash,Alpha-Cash-CIC-Model_All_Trench,android,Quick,Trench 2,0.309645,0.461273,"{""cic_max_age_all_contracts_snapshot"": 211, ""c...",2025-10-09 11:38:04,2025-10-10 16:41:46,2025-10,211.0,0.000000,Ei,2_New Applicant,0.0,0.0,0.833333,0.000000,NaN,0,29331.0,0.0,0


In [17]:
# sq = r"""  
# WITH parsed as (
#   select customerId, digitalLoanAccountId,modelDisplayName,modelVersionId,start_time,end_time,prediction,
# REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
# --REPLACE(REPLACE(prediction, "'", '"'), "None", "null") AS prediction_clean
# FROM `prj-prod-dataplatform.audit_balance.ml_model_run_details`
# where modelDisplayName = 'Alpha-Cash-CIC-Model'
# ),

# latest_request as (
# select * from parsed
# QUALIFY ROW_NUMBER() OVER (PARTITION BY customerId, digitalLoanAccountId,modelDisplayName ORDER BY start_time DESC ) = 1),

# model_run as (
# select customerId,digitalLoanAccountId,modelName, publish_time,requestPayload as requestPayload_clean
# --REPLACE(REPLACE(requestPayload, "'", '"'), "None", "null") AS requestPayload_clean
# from `prj-prod-dataplatform.audit_balance.ml_request_details` 
# WHERE modelName = 'Alpha-Cash-Model-response'
# QUALIFY ROW_NUMBER() OVER (PARTITION BY customerId, digitalLoanAccountId,modelName ORDER BY publish_time DESC ) = 1),
# base as (
# select * from (
#   select 
#  r.customerId,
#  r.digitalLoanAccountId,
#  r.prediction,
#  r.start_time,
#  r.end_time,
#  r.modelDisplayName,
#  r.modelVersionId,
#  loanmaster.new_loan_type,
#  loanmaster.gender,
#   REGEXP_EXTRACT(m.requestPayload_clean, r"osType[:=]['\"]?([^'\"]+)['\"]?") AS osType,
#   REGEXP_EXTRACT(m.requestPayload_clean, r"loanType[:=]['\"]?([^'\"]+)['\"]?") AS loanType,
#   REGEXP_EXTRACT(m.requestPayload_clean, r"trenchCategory[:=]['\"]?([^'\"]+)['\"]?") AS trenchCategory,
#    SAFE_CAST(REGEXP_EXTRACT(m.requestPayload_clean, r"aStackScore[:= ]([0-9\.]+)") AS FLOAT64) AS aStackScore,
#   SAFE_CAST(REGEXP_EXTRACT(m.requestPayload_clean, r"aCicScore[:= ]([0-9\.]+)") AS FLOAT64) AS aCicScore,
#   --  Alpha CIC Score Model Features for Trench 1
#   SAFE_CAST(JSON_VALUE(r.calcFeatures, "$.cic_max_age_all_contracts_snapshot") AS INT64) AS cic_max_age_all_contracts_snapshot,
#   SAFE_CAST(JSON_VALUE(r.calcFeatures, "$.cic_ratio_overdue_contracts_to_granted_contracts") AS FLOAT64) AS cic_ratio_overdue_contracts_to_granted_contracts,
#   JSON_VALUE(r.calcFeatures, "$.cic_ScoreRange") AS cic_ScoreRange,
#   JSON_VALUE(r.calcFeatures, "$.cic_ln_loan_level_user_type") AS cic_ln_loan_level_user_type,
#   JSON_VALUE(r.calcFeatures, "$.cic_has_ever_been_overdue") AS cic_has_ever_been_overdue,
#   JSON_VALUE(r.calcFeatures, "$.cic_latest_granted_contract_overdue_flag") AS cic_latest_granted_contract_overdue_flag,
#   JSON_VALUE(r.calcFeatures, "$.cic_ratio_closed_over_new_granted_cnt_24M") AS cic_ratio_closed_over_new_granted_cnt_24M,
#   JSON_VALUE(r.calcFeatures, "$.cic_ratio_risky_contracts_to_granted_contracts") AS cic_ratio_risky_contracts_to_granted_contracts,
#   JSON_VALUE(r.calcFeatures, "$.cic_Short_and_Term_Loans_granted_contracts_cnt_24M") AS cic_Short_and_Term_Loans_granted_contracts_cnt_24M,
#   JSON_VALUE(r.calcFeatures, "$.cic_flg_zero_non_granted_ever") AS cic_flg_zero_non_granted_ever,
#   JSON_VALUE(r.calcFeatures, "$.cic_Personal_Loans_granted_contracts_amt_24M") AS cic_Personal_Loans_granted_contracts_amt_24M,
#   JSON_VALUE(r.calcFeatures, "$.cic_CreditAvgCreditLimit") AS cic_CreditAvgCreditLimit,
#   JSON_VALUE(r.calcFeatures, "$.cic_flg_zero_granted_ever") AS cic_flg_zero_granted_ever,
#   coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  r.start_time) AS appln_submit_datetime,
#   loanmaster.disbursementDateTime,
#   format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  r.start_time)) as Application_month, 
# FROM latest_request r
# left join model_run m
# on r.digitalLoanAccountId = m.digitalLoanAccountId 
# left join risk_credit_mis.loan_master_table loanmaster 
#   ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
#  )
# where trenchCategory = 'Trench 1'
# )
# select *, case when appln_submit_datetime <= '2025-09-30' then 'Train' else 'Test' end dataselection from base
# ;
# """

# df = client.query(sq).to_dataframe()
# df.head()

### copy to df

In [18]:
df = expanded_df.drop(columns=['calcFeatures']).copy()

### Run the PSI calculation

In [19]:
df = df.copy()

# Define feature list
feature_list = ['aCicScore',
     'calc_cic_max_age_all_contracts_snapshot',
       'calc_cic_ratio_overdue_contracts_to_granted_contracts',
       'calc_cic_ScoreRange', 'calc_cic_ln_loan_level_user_type',
       'calc_cic_has_ever_been_overdue',
       'calc_cic_latest_granted_contract_overdue_flag',
       'calc_cic_ratio_closed_over_new_granted_cnt_24M',
       'calc_cic_ratio_risky_contracts_to_granted_contracts',
       'calc_cic_Short_and_Term_Loans_granted_contracts_cnt_24M',
       'calc_cic_flg_zero_non_granted_ever',
       'calc_cic_Personal_Loans_granted_contracts_amt_24M',
       'calc_cic_CreditAvgCreditLimit', 'calc_cic_flg_zero_granted_ever'
]

# Define segment columns
segment_columns = ['new_loan_type', 'gender', 'osType', 'trenchCategory', 'product']
# Calculate month-on-month PSI
psi_results = calculate_month_on_month_psi(df, feature_list, segment_columns)
psi_results['modelDisplayName'] = df['modelDisplayName'].iloc[0]
psi_results['Model_Name'] = df['Model_Name'].iloc[0]
psi_results['modelVersionId'] = df['modelVersionId'].iloc[0]
psi_results['trenchCategory'] = df['trenchCategory'].iloc[0]
psi_results = psi_results[[ 'modelDisplayName', 'Model_Name',  'modelVersionId', 'trenchCategory','Feature', 'Feature_Type', 'Segment_Column', 'Segment_Value', 'Month',
       'Base_Month', 'Current_Month', 'PSI']].copy()

# Calculate bin-level PSI
bin_psi_results = calculate_bin_level_psi(df, feature_list, segment_columns)
bin_psi_results['modelDisplayName'] = df['modelDisplayName'].iloc[0]
bin_psi_results['Model_Name'] = df['Model_Name'].iloc[0]
bin_psi_results['modelVersionId'] = df['modelVersionId'].iloc[0]
bin_psi_results['trenchCategory'] = df['trenchCategory'].iloc[0]
bin_psi_results = bin_psi_results[[ 'modelDisplayName', 'Model_Name',  'modelVersionId', 'trenchCategory','Feature', 'Feature_Type', 'Segment_Column', 'Segment_Value', 'Month',
       'Base_Month', 'Current_Month', 'Bin', 'Bin_Range', 'Bin_Min', 'Bin_Max', 'Base_Percentage',
       'Actual_Percentage', 'Bin_PSI']].copy()

# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.temp_csi_new_monitoring_data"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(psi_results, table_id, job_config=job_config)
job.result()  # Wait for the job to complete


# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.temp_csi_new_monitoring_data_feature_bin_level"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(bin_psi_results, table_id, job_config=job_config)
job.result()  # Wait for the job to complete

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=6ed9a1da-b864-43c7-8f00-481d7cba1b88>

## Trench 3

In [20]:
sq = r"""  
WITH parsed as (
  select customerId, digitalLoanAccountId,modelDisplayName,modelVersionId,start_time,end_time,prediction,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
--REPLACE(REPLACE(prediction, "'", '"'), "None", "null") AS prediction_clean
FROM `prj-prod-dataplatform.audit_balance.ml_model_run_details`
where modelDisplayName = 'Alpha-Cash-CIC-Model'
),
latest_request as (
select * from parsed
QUALIFY ROW_NUMBER() OVER (PARTITION BY customerId, digitalLoanAccountId,modelDisplayName ORDER BY start_time DESC ) = 1),

model_run as (
select customerId,digitalLoanAccountId,modelName, publish_time,requestPayload as requestPayload_clean
--REPLACE(REPLACE(requestPayload, "'", '"'), "None", "null") AS requestPayload_clean
from `prj-prod-dataplatform.audit_balance.ml_request_details` 
WHERE modelName = 'Alpha-Cash-Model-response'
QUALIFY ROW_NUMBER() OVER (PARTITION BY customerId, digitalLoanAccountId,modelName ORDER BY publish_time DESC ) = 1),
base as (
select * from (
  select 
 r.customerId,
 r.digitalLoanAccountId,
 r.prediction,
 r.start_time,
 r.end_time,
 r.modelDisplayName,
 r.modelVersionId,
 loanmaster.new_loan_type,
 loanmaster.gender,
 'Cash' as product,
 'Alpha-Cash-CIC-Model_All_Trench' Model_Name,
  REGEXP_EXTRACT(m.requestPayload_clean, r"osType[:=]['\"]?([^'\"]+)['\"]?") AS osType,
  REGEXP_EXTRACT(m.requestPayload_clean, r"loanType[:=]['\"]?([^'\"]+)['\"]?") AS loanType,
  REGEXP_EXTRACT(m.requestPayload_clean, r"trenchCategory[:=]['\"]?([^'\"]+)['\"]?") AS trenchCategory,
   SAFE_CAST(REGEXP_EXTRACT(m.requestPayload_clean, r"aStackScore[:= ]([0-9\.]+)") AS FLOAT64) AS aStackScore,
  SAFE_CAST(REGEXP_EXTRACT(m.requestPayload_clean, r"aCicScore[:= ]([0-9\.]+)") AS FLOAT64) AS aCicScore,
  calcFeatures,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  r.start_time) AS appln_submit_datetime,
  loanmaster.disbursementDateTime,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  r.start_time)) as Application_month, 
FROM latest_request r
left join model_run m
on r.digitalLoanAccountId = m.digitalLoanAccountId 
left join risk_credit_mis.loan_master_table loanmaster 
  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
qualify row_number() over (partition by r.customerId,r.digitalLoanAccountId order by r.start_time desc) = 1
 )
)
select * from base where trenchCategory = 'Trench 3';
"""
dfd = client.query(sq).to_dataframe()
dfd.head()

,customerId,digitalLoanAccountId,prediction,start_time,end_time,modelDisplayName,modelVersionId,new_loan_type,gender,product,Model_Name,osType,loanType,trenchCategory,aStackScore,aCicScore,calcFeatures,appln_submit_datetime,disbursementDateTime,Application_month
0,3335859,53b06e45-e8bc-465e-ad30-4aec9af33333,0.5533750345569043,2025-10-09 04:16:34.220082,2025-10-09 04:16:34.367033,Alpha-Cash-CIC-Model,v1,Quick,M,Cash,Alpha-Cash-CIC-Model_All_Trench,ios,Quick,Trench 3,0.549099,0.553375,"{""cic_max_age_all_contracts_snapshot"": 556, ""c...",2025-10-09 12:16:24,NaT,2025-10
1,3311248,342a8542-2fe1-41d9-a11c-9a2272c2f5da,0.3759479301029923,2025-10-10 09:46:00.296484,2025-10-10 09:46:00.750937,Alpha-Cash-CIC-Model,v1,Quick,F,Cash,Alpha-Cash-CIC-Model_All_Trench,ios,Quick,Trench 3,0.412758,0.375948,"{""cic_max_age_all_contracts_snapshot"": 428.0, ...",2025-10-10 17:45:42,2025-10-10 18:10:24,2025-10
2,1119019,8ce7a733-521d-4d65-b9d0-e12de2cc375b,0.35492268789828746,2025-10-10 20:19:36.365308,2025-10-10 20:19:36.513105,Alpha-Cash-CIC-Model,v1,Quick,M,Cash,Alpha-Cash-CIC-Model_All_Trench,android,Quick,Trench 3,0.464307,0.354923,"{""cic_max_age_all_contracts_snapshot"": 1873, ""...",2025-10-11 04:19:23,NaT,2025-10
3,3345325,49627b46-d418-4903-9fb5-bf15ea74495d,0.40525888332083804,2025-10-09 19:14:46.850439,2025-10-09 19:14:46.972052,Alpha-Cash-CIC-Model,v1,Quick,F,Cash,Alpha-Cash-CIC-Model_All_Trench,ios,Quick,Trench 3,0.412175,0.405259,"{""cic_max_age_all_contracts_snapshot"": 187, ""c...",2025-10-10 03:14:38,2025-10-10 09:38:07,2025-10
4,3415580,2e3e64b2-fc63-4a5c-8a98-7227a7f018b9,0.40525888332083804,2025-10-11 02:09:44.999498,2025-10-11 02:09:45.087998,Alpha-Cash-CIC-Model,v1,Quick,F,Cash,Alpha-Cash-CIC-Model_All_Trench,ios,Quick,Trench 3,0.400289,0.405259,"{""cic_max_age_all_contracts_snapshot"": 153, ""c...",2025-10-11 10:09:31,2025-10-11 10:18:18,2025-10


### expand df

In [21]:
# Expand the calcFeatures column
expanded_df = expand_calc_features(dfd)

# Display the result
print(f"Original columns: {dfd.shape[1]}")
print(f"Expanded columns: {expanded_df.shape[1]}")
expanded_df.head()


Original columns: 20
Expanded columns: 33


,customerId,digitalLoanAccountId,prediction,start_time,end_time,modelDisplayName,modelVersionId,new_loan_type,gender,product,Model_Name,osType,loanType,trenchCategory,aStackScore,aCicScore,calcFeatures,appln_submit_datetime,disbursementDateTime,Application_month,calc_cic_max_age_all_contracts_snapshot,calc_cic_ratio_overdue_contracts_to_granted_contracts,calc_cic_ScoreRange,calc_cic_ln_loan_level_user_type,calc_cic_has_ever_been_overdue,calc_cic_latest_granted_contract_overdue_flag,calc_cic_ratio_closed_over_new_granted_cnt_24M,calc_cic_ratio_risky_contracts_to_granted_contracts,calc_cic_Short_and_Term_Loans_granted_contracts_cnt_24M,calc_cic_flg_zero_non_granted_ever,calc_cic_Personal_Loans_granted_contracts_amt_24M,calc_cic_CreditAvgCreditLimit,calc_cic_flg_zero_granted_ever
0,3335859,53b06e45-e8bc-465e-ad30-4aec9af33333,0.5533750345569043,2025-10-09 04:16:34.220082,2025-10-09 04:16:34.367033,Alpha-Cash-CIC-Model,v1,Quick,M,Cash,Alpha-Cash-CIC-Model_All_Trench,ios,Quick,Trench 3,0.549099,0.553375,"{""cic_max_age_all_contracts_snapshot"": 556, ""c...",2025-10-09 12:16:24,NaT,2025-10,556.0,0.571429,Bi,1_Repeat Applicant,1.0,0.0,2.000000,0.142857,NaN,0,NaN,0,0
1,3311248,342a8542-2fe1-41d9-a11c-9a2272c2f5da,0.3759479301029923,2025-10-10 09:46:00.296484,2025-10-10 09:46:00.750937,Alpha-Cash-CIC-Model,v1,Quick,F,Cash,Alpha-Cash-CIC-Model_All_Trench,ios,Quick,Trench 3,0.412758,0.375948,"{""cic_max_age_all_contracts_snapshot"": 428.0, ...",2025-10-10 17:45:42,2025-10-10 18:10:24,2025-10,428.0,0.000000,NH_Gi,1_Repeat Applicant,0.0,0.0,0.843931,0.000000,112.0,0,82215.0,0,0
2,1119019,8ce7a733-521d-4d65-b9d0-e12de2cc375b,0.35492268789828746,2025-10-10 20:19:36.365308,2025-10-10 20:19:36.513105,Alpha-Cash-CIC-Model,v1,Quick,M,Cash,Alpha-Cash-CIC-Model_All_Trench,android,Quick,Trench 3,0.464307,0.354923,"{""cic_max_age_all_contracts_snapshot"": 1873, ""...",2025-10-11 04:19:23,NaT,2025-10,1873.0,0.000000,Unknown,1_Repeat Applicant,0.0,0.0,1.250000,0.000000,NaN,0,26200.0,0,0
3,3345325,49627b46-d418-4903-9fb5-bf15ea74495d,0.40525888332083804,2025-10-09 19:14:46.850439,2025-10-09 19:14:46.972052,Alpha-Cash-CIC-Model,v1,Quick,F,Cash,Alpha-Cash-CIC-Model_All_Trench,ios,Quick,Trench 3,0.412175,0.405259,"{""cic_max_age_all_contracts_snapshot"": 187, ""c...",2025-10-10 03:14:38,2025-10-10 09:38:07,2025-10,187.0,0.000000,Ai,1_Repeat Applicant,0.0,0.0,1.000000,0.000000,NaN,1,NaN,0,0
4,3415580,2e3e64b2-fc63-4a5c-8a98-7227a7f018b9,0.40525888332083804,2025-10-11 02:09:44.999498,2025-10-11 02:09:45.087998,Alpha-Cash-CIC-Model,v1,Quick,F,Cash,Alpha-Cash-CIC-Model_All_Trench,ios,Quick,Trench 3,0.400289,0.405259,"{""cic_max_age_all_contracts_snapshot"": 153, ""c...",2025-10-11 10:09:31,2025-10-11 10:18:18,2025-10,153.0,0.000000,Ai,1_Repeat Applicant,0.0,0.0,1.000000,0.000000,NaN,1,NaN,0,0


In [22]:
# sq = r"""  
# WITH parsed as (
#   select customerId, digitalLoanAccountId,modelDisplayName,modelVersionId,start_time,end_time,prediction,
# REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
# --REPLACE(REPLACE(prediction, "'", '"'), "None", "null") AS prediction_clean
# FROM `prj-prod-dataplatform.audit_balance.ml_model_run_details`
# where modelDisplayName = 'Alpha-Cash-CIC-Model'
# ),

# latest_request as (
# select * from parsed
# QUALIFY ROW_NUMBER() OVER (PARTITION BY customerId, digitalLoanAccountId,modelDisplayName ORDER BY start_time DESC ) = 1),

# model_run as (
# select customerId,digitalLoanAccountId,modelName, publish_time,requestPayload as requestPayload_clean
# --REPLACE(REPLACE(requestPayload, "'", '"'), "None", "null") AS requestPayload_clean
# from `prj-prod-dataplatform.audit_balance.ml_request_details` 
# WHERE modelName = 'Alpha-Cash-Model-response'
# QUALIFY ROW_NUMBER() OVER (PARTITION BY customerId, digitalLoanAccountId,modelName ORDER BY publish_time DESC ) = 1),
# base as (
# select * from (
#   select 
#  r.customerId,
#  r.digitalLoanAccountId,
#  r.prediction,
#  r.start_time,
#  r.end_time,
#  r.modelDisplayName,
#  r.modelVersionId,
#  loanmaster.new_loan_type,
#  loanmaster.gender,
#   REGEXP_EXTRACT(m.requestPayload_clean, r"osType[:=]['\"]?([^'\"]+)['\"]?") AS osType,
#   REGEXP_EXTRACT(m.requestPayload_clean, r"loanType[:=]['\"]?([^'\"]+)['\"]?") AS loanType,
#   REGEXP_EXTRACT(m.requestPayload_clean, r"trenchCategory[:=]['\"]?([^'\"]+)['\"]?") AS trenchCategory,
#    SAFE_CAST(REGEXP_EXTRACT(m.requestPayload_clean, r"aStackScore[:= ]([0-9\.]+)") AS FLOAT64) AS aStackScore,
#   SAFE_CAST(REGEXP_EXTRACT(m.requestPayload_clean, r"aCicScore[:= ]([0-9\.]+)") AS FLOAT64) AS aCicScore,
#   --  Alpha CIC Score Model Features for Trench 1
#   SAFE_CAST(JSON_VALUE(r.calcFeatures, "$.cic_max_age_all_contracts_snapshot") AS INT64) AS cic_max_age_all_contracts_snapshot,
#   SAFE_CAST(JSON_VALUE(r.calcFeatures, "$.cic_ratio_overdue_contracts_to_granted_contracts") AS FLOAT64) AS cic_ratio_overdue_contracts_to_granted_contracts,
#   JSON_VALUE(r.calcFeatures, "$.cic_ScoreRange") AS cic_ScoreRange,
#   JSON_VALUE(r.calcFeatures, "$.cic_ln_loan_level_user_type") AS cic_ln_loan_level_user_type,
#   JSON_VALUE(r.calcFeatures, "$.cic_has_ever_been_overdue") AS cic_has_ever_been_overdue,
#   JSON_VALUE(r.calcFeatures, "$.cic_latest_granted_contract_overdue_flag") AS cic_latest_granted_contract_overdue_flag,
#   JSON_VALUE(r.calcFeatures, "$.cic_ratio_closed_over_new_granted_cnt_24M") AS cic_ratio_closed_over_new_granted_cnt_24M,
#   JSON_VALUE(r.calcFeatures, "$.cic_ratio_risky_contracts_to_granted_contracts") AS cic_ratio_risky_contracts_to_granted_contracts,
#   JSON_VALUE(r.calcFeatures, "$.cic_Short_and_Term_Loans_granted_contracts_cnt_24M") AS cic_Short_and_Term_Loans_granted_contracts_cnt_24M,
#   JSON_VALUE(r.calcFeatures, "$.cic_flg_zero_non_granted_ever") AS cic_flg_zero_non_granted_ever,
#   JSON_VALUE(r.calcFeatures, "$.cic_Personal_Loans_granted_contracts_amt_24M") AS cic_Personal_Loans_granted_contracts_amt_24M,
#   JSON_VALUE(r.calcFeatures, "$.cic_CreditAvgCreditLimit") AS cic_CreditAvgCreditLimit,
#   JSON_VALUE(r.calcFeatures, "$.cic_flg_zero_granted_ever") AS cic_flg_zero_granted_ever,
#   coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  r.start_time) AS appln_submit_datetime,
#   loanmaster.disbursementDateTime,
#   format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  r.start_time)) as Application_month, 
# FROM latest_request r
# left join model_run m
# on r.digitalLoanAccountId = m.digitalLoanAccountId 
# left join risk_credit_mis.loan_master_table loanmaster 
#   ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
#  )
# where trenchCategory = 'Trench 1'
# )
# select *, case when appln_submit_datetime <= '2025-09-30' then 'Train' else 'Test' end dataselection from base
# ;
# """

# df = client.query(sq).to_dataframe()
# df.head()

### copy to df

In [23]:
df = expanded_df.drop(columns=['calcFeatures']).copy()

### Run the PSI calculation

In [24]:
df = df.copy()

# Define feature list
feature_list = ['aCicScore',
     'calc_cic_max_age_all_contracts_snapshot',
       'calc_cic_ratio_overdue_contracts_to_granted_contracts',
       'calc_cic_ScoreRange', 'calc_cic_ln_loan_level_user_type',
       'calc_cic_has_ever_been_overdue',
       'calc_cic_latest_granted_contract_overdue_flag',
       'calc_cic_ratio_closed_over_new_granted_cnt_24M',
       'calc_cic_ratio_risky_contracts_to_granted_contracts',
       'calc_cic_Short_and_Term_Loans_granted_contracts_cnt_24M',
       'calc_cic_flg_zero_non_granted_ever',
       'calc_cic_Personal_Loans_granted_contracts_amt_24M',
       'calc_cic_CreditAvgCreditLimit', 'calc_cic_flg_zero_granted_ever'
]

# Define segment columns
segment_columns = ['new_loan_type', 'gender', 'osType', 'trenchCategory', 'product']
# Calculate month-on-month PSI
psi_results = calculate_month_on_month_psi(df, feature_list, segment_columns)
psi_results['modelDisplayName'] = df['modelDisplayName'].iloc[0]
psi_results['Model_Name'] = df['Model_Name'].iloc[0]
psi_results['modelVersionId'] = df['modelVersionId'].iloc[0]
psi_results['trenchCategory'] = df['trenchCategory'].iloc[0]
psi_results = psi_results[[ 'modelDisplayName', 'Model_Name',  'modelVersionId', 'trenchCategory','Feature', 'Feature_Type', 'Segment_Column', 'Segment_Value', 'Month',
       'Base_Month', 'Current_Month', 'PSI']].copy()

# Calculate bin-level PSI
bin_psi_results = calculate_bin_level_psi(df, feature_list, segment_columns)
bin_psi_results['modelDisplayName'] = df['modelDisplayName'].iloc[0]
bin_psi_results['Model_Name'] = df['Model_Name'].iloc[0]
bin_psi_results['modelVersionId'] = df['modelVersionId'].iloc[0]
bin_psi_results['trenchCategory'] = df['trenchCategory'].iloc[0]
bin_psi_results = bin_psi_results[[ 'modelDisplayName', 'Model_Name',  'modelVersionId', 'trenchCategory','Feature', 'Feature_Type', 'Segment_Column', 'Segment_Value', 'Month',
       'Base_Month', 'Current_Month', 'Bin', 'Bin_Range', 'Bin_Min', 'Bin_Max', 'Base_Percentage',
       'Actual_Percentage', 'Bin_PSI']].copy()

# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.temp_csi_new_monitoring_data"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(psi_results, table_id, job_config=job_config)
job.result()  # Wait for the job to complete


# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.temp_csi_new_monitoring_data_feature_bin_level"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(bin_psi_results, table_id, job_config=job_config)
job.result()  # Wait for the job to complete

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=5ccd885e-6821-4d55-9ffd-2f8b7d58ae1a>

In [25]:
psi_results

,modelDisplayName,Model_Name,modelVersionId,trenchCategory,Feature,Feature_Type,Segment_Column,Segment_Value,Month,Base_Month,Current_Month,PSI
0,Alpha-Cash-CIC-Model,Alpha-Cash-CIC-Model_All_Trench,v1,Trench 3,aCicScore,numerical,Overall,All,2025-09,2025-09,2025-09,0.000000
1,Alpha-Cash-CIC-Model,Alpha-Cash-CIC-Model_All_Trench,v1,Trench 3,aCicScore,numerical,Overall,All,2025-10,2025-09,2025-10,0.389456
2,Alpha-Cash-CIC-Model,Alpha-Cash-CIC-Model_All_Trench,v1,Trench 3,calc_cic_max_age_all_contracts_snapshot,numerical,Overall,All,2025-09,2025-09,2025-09,0.000000
3,Alpha-Cash-CIC-Model,Alpha-Cash-CIC-Model_All_Trench,v1,Trench 3,calc_cic_max_age_all_contracts_snapshot,numerical,Overall,All,2025-10,2025-09,2025-10,0.660361
4,Alpha-Cash-CIC-Model,Alpha-Cash-CIC-Model_All_Trench,v1,Trench 3,calc_cic_ratio_overdue_contracts_to_granted_co...,numerical,Overall,All,2025-09,2025-09,2025-09,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
219,Alpha-Cash-CIC-Model,Alpha-Cash-CIC-Model_All_Trench,v1,Trench 3,calc_cic_Personal_Loans_granted_contracts_amt_24M,numerical,product,Cash,2025-10,2025-09,2025-10,0.654428
220,Alpha-Cash-CIC-Model,Alpha-Cash-CIC-Model_All_Trench,v1,Trench 3,calc_cic_CreditAvgCreditLimit,numerical,product,Cash,2025-09,2025-09,2025-09,0.000000
221,Alpha-Cash-CIC-Model,Alpha-Cash-CIC-Model_All_Trench,v1,Trench 3,calc_cic_CreditAvgCreditLimit,numerical,product,Cash,2025-10,2025-09,2025-10,1.278355
222,Alpha-Cash-CIC-Model,Alpha-Cash-CIC-Model_All_Trench,v1,Trench 3,calc_cic_flg_zero_granted_ever,categorical,product,Cash,2025-09,2025-09,2025-09,0.000000


## Alpha-Cash-Stack-Model

In [26]:
sq = r""" 
WITH parsed as (
  select customerId, digitalLoanAccountId,modelDisplayName,modelVersionId,start_time,end_time,prediction,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
--REPLACE(REPLACE(prediction, "'", '"'), "None", "null") AS prediction_clean
FROM `prj-prod-dataplatform.audit_balance.ml_model_run_details`
where modelDisplayName = 'Alpha-Cash-Stack-Model'
),
latest_request as (
select * from parsed
QUALIFY ROW_NUMBER() OVER (PARTITION BY customerId, digitalLoanAccountId,modelDisplayName ORDER BY start_time DESC ) = 1),

model_run as (
select customerId,digitalLoanAccountId,modelName, publish_time,requestPayload as requestPayload_clean
--REPLACE(REPLACE(requestPayload, "'", '"'), "None", "null") AS requestPayload_clean
from `prj-prod-dataplatform.audit_balance.ml_request_details` 
WHERE modelName = 'Alpha-Cash-Model-response'
QUALIFY ROW_NUMBER() OVER (PARTITION BY customerId, digitalLoanAccountId,modelName ORDER BY publish_time DESC ) = 1),
base as (
select * from (
  select 
 r.customerId,
 r.digitalLoanAccountId,
 r.prediction ,
 r.start_time,
 r.end_time,
 r.modelDisplayName,
 r.modelVersionId,
 loanmaster.new_loan_type,
 loanmaster.gender,
 'Alpha-Cash-Stack-Model_Trench1' Model_Name,
 'Cash' as product,
  REGEXP_EXTRACT(m.requestPayload_clean, r"osType[:=]['\"]?([^'\"]+)['\"]?") AS osType,
  REGEXP_EXTRACT(m.requestPayload_clean, r"loanType[:=]['\"]?([^'\"]+)['\"]?") AS loanType,
  REGEXP_EXTRACT(m.requestPayload_clean, r"trenchCategory[:=]['\"]?([^'\"]+)['\"]?") AS trenchCategory,
   SAFE_CAST(REGEXP_EXTRACT(m.requestPayload_clean, r"aStackScore[:= ]([0-9\.]+)") AS FLOAT64) AS aStackScore,
  SAFE_CAST(REGEXP_EXTRACT(m.requestPayload_clean, r"aCicScore[:= ]([0-9\.]+)") AS FLOAT64) AS aCicScore,
  calcFeatures,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  r.start_time) AS appln_submit_datetime,
  loanmaster.disbursementDateTime,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  r.start_time)) as Application_month, 
FROM latest_request r
left join model_run m
on r.digitalLoanAccountId = m.digitalLoanAccountId 
left join risk_credit_mis.loan_master_table loanmaster 
  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
qualify row_number() over (partition by r.customerId,r.digitalLoanAccountId order by r.start_time desc) = 1
 )
)
select * from base where trenchCategory = 'Trench 1';
"""

dfd = client.query(sq).to_dataframe()
dfd.head()

,customerId,digitalLoanAccountId,prediction,start_time,end_time,modelDisplayName,modelVersionId,new_loan_type,gender,Model_Name,product,osType,loanType,trenchCategory,aStackScore,aCicScore,calcFeatures,appln_submit_datetime,disbursementDateTime,Application_month
0,3668281,5436bfa2-9a51-4e42-830c-d63655c60739,0.633173794938964,2025-09-26 12:24:35.244199,2025-09-26 12:24:35.328943,Alpha-Cash-Stack-Model,v1,Quick,M,Alpha-Cash-Stack-Model_Trench1,Cash,android,Quick,Trench 1,0.633174,0.574588,"{""apps_score"": 0.4928494678097085, ""c_demo_sco...",2025-09-26 20:24:25,NaT,2025-09
1,3729921,719ce69f-e5c1-40c4-9115-78a664c0750c,0.21797246719589317,2025-10-08 00:37:43.056962,2025-10-08 00:37:43.418498,Alpha-Cash-Stack-Model,v1,Quick,F,Alpha-Cash-Stack-Model_Trench1,Cash,ios,Quick,Trench 1,0.217972,0.291523,"{""apps_score"": null, ""c_demo_score"": 0.4350608...",2025-10-08 08:37:26,2025-10-09 05:53:44,2025-10
2,3732450,1525507a-82c9-4e34-9d0e-c7d3b65228d3,0.5760031035842773,2025-10-08 16:41:26.331082,2025-10-08 16:41:26.429988,Alpha-Cash-Stack-Model,v1,Quick,M,Alpha-Cash-Stack-Model_Trench1,Cash,android,Quick,Trench 1,0.576003,0.503540,"{""apps_score"": 0.5173846455480939, ""c_demo_sco...",2025-10-09 00:41:15,NaT,2025-10
3,3734182,b6c25885-8c97-4339-b602-4426b0595aad,0.6594798726339557,2025-10-12 08:39:20.835657,2025-10-12 08:39:21.024376,Alpha-Cash-Stack-Model,v1,Quick,F,Alpha-Cash-Stack-Model_Trench1,Cash,android,Quick,Trench 1,0.659480,0.658082,"{""apps_score"": 0.4819402966151577, ""c_demo_sco...",2025-10-12 16:39:10,NaT,2025-10
4,3734296,b91a7ba7-352a-4965-8527-87a21ecefcf7,0.6651406202517616,2025-10-09 15:29:42.308355,2025-10-09 15:29:42.434496,Alpha-Cash-Stack-Model,v1,Quick,F,Alpha-Cash-Stack-Model_Trench1,Cash,android,Quick,Trench 1,0.665141,0.750678,"{""apps_score"": 0.44678216066646576, ""c_demo_sc...",2025-10-09 23:29:32,NaT,2025-10


### Expand df

In [27]:
# Expand the calcFeatures column
expanded_df = expand_calc_features(dfd)

# Display the result
print(f"Original columns: {dfd.shape[1]}")
print(f"Expanded columns: {expanded_df.shape[1]}")
expanded_df.head()


Original columns: 20
Expanded columns: 25


,customerId,digitalLoanAccountId,prediction,start_time,end_time,modelDisplayName,modelVersionId,new_loan_type,gender,Model_Name,product,osType,loanType,trenchCategory,aStackScore,aCicScore,calcFeatures,appln_submit_datetime,disbursementDateTime,Application_month,calc_apps_score,calc_c_demo_score,calc_c_credo_score,calc_c_tx_score,calc_ca_cic_score
0,3668281,5436bfa2-9a51-4e42-830c-d63655c60739,0.633173794938964,2025-09-26 12:24:35.244199,2025-09-26 12:24:35.328943,Alpha-Cash-Stack-Model,v1,Quick,M,Alpha-Cash-Stack-Model_Trench1,Cash,android,Quick,Trench 1,0.633174,0.574588,"{""apps_score"": 0.4928494678097085, ""c_demo_sco...",2025-09-26 20:24:25,NaT,2025-09,0.492849,0.624497,0.3013,None,0.574588
1,3729921,719ce69f-e5c1-40c4-9115-78a664c0750c,0.21797246719589317,2025-10-08 00:37:43.056962,2025-10-08 00:37:43.418498,Alpha-Cash-Stack-Model,v1,Quick,F,Alpha-Cash-Stack-Model_Trench1,Cash,ios,Quick,Trench 1,0.217972,0.291523,"{""apps_score"": null, ""c_demo_score"": 0.4350608...",2025-10-08 08:37:26,2025-10-09 05:53:44,2025-10,NaN,0.435061,0.2068,None,0.291523
2,3732450,1525507a-82c9-4e34-9d0e-c7d3b65228d3,0.5760031035842773,2025-10-08 16:41:26.331082,2025-10-08 16:41:26.429988,Alpha-Cash-Stack-Model,v1,Quick,M,Alpha-Cash-Stack-Model_Trench1,Cash,android,Quick,Trench 1,0.576003,0.503540,"{""apps_score"": 0.5173846455480939, ""c_demo_sco...",2025-10-09 00:41:15,NaT,2025-10,0.517385,0.483696,0.0759,None,0.503540
3,3734182,b6c25885-8c97-4339-b602-4426b0595aad,0.6594798726339557,2025-10-12 08:39:20.835657,2025-10-12 08:39:21.024376,Alpha-Cash-Stack-Model,v1,Quick,F,Alpha-Cash-Stack-Model_Trench1,Cash,android,Quick,Trench 1,0.659480,0.658082,"{""apps_score"": 0.4819402966151577, ""c_demo_sco...",2025-10-12 16:39:10,NaT,2025-10,0.481940,0.574161,0.3284,None,0.658082
4,3734296,b91a7ba7-352a-4965-8527-87a21ecefcf7,0.6651406202517616,2025-10-09 15:29:42.308355,2025-10-09 15:29:42.434496,Alpha-Cash-Stack-Model,v1,Quick,F,Alpha-Cash-Stack-Model_Trench1,Cash,android,Quick,Trench 1,0.665141,0.750678,"{""apps_score"": 0.44678216066646576, ""c_demo_sc...",2025-10-09 23:29:32,NaT,2025-10,0.446782,0.607666,0.4595,None,0.750678


In [28]:
df = expanded_df.drop(columns=['calcFeatures']).copy()

In [29]:
# sq = r"""WITH parsed as (
#   select customerId, digitalLoanAccountId,modelDisplayName,modelVersionId,start_time,end_time,prediction,
# REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
# FROM `prj-prod-dataplatform.audit_balance.ml_model_run_details`
# where modelDisplayName = 'Alpha-Cash-Stack-Model'),
# latest_request as (
# select * from parsed
# QUALIFY ROW_NUMBER() OVER (PARTITION BY customerId, digitalLoanAccountId,modelDisplayName ORDER BY start_time DESC ) = 1),
# model_run as (
# select customerId,digitalLoanAccountId,modelName, publish_time,requestPayload as requestPayload_clean
# from `prj-prod-dataplatform.audit_balance.ml_request_details` 
# WHERE modelName = 'Alpha-Cash-Model-response'
# QUALIFY ROW_NUMBER() OVER (PARTITION BY customerId, digitalLoanAccountId,modelName ORDER BY publish_time DESC ) = 1)
# select * from (
#   select 
#  r.customerId,
#  r.digitalLoanAccountId,
#  r.prediction Alpha_Cash_Stack_Score,
#  r.start_time,
#  r.end_time,
#  r.modelDisplayName,
#  r.modelVersionId,
#   loanmaster.new_loan_type,
#  loanmaster.gender,

#   REGEXP_EXTRACT(m.requestPayload_clean, r"osType[:=]['\"]?([^'\"]+)['\"]?") AS osType,
#   REGEXP_EXTRACT(m.requestPayload_clean, r"loanType[:=]['\"]?([^'\"]+)['\"]?") AS loanType,
#   REGEXP_EXTRACT(m.requestPayload_clean, r"trenchCategory[:=]['\"]?([^'\"]+)['\"]?") AS trenchCategory,

#  SAFE_CAST(JSON_VALUE(r.calcFeatures, "$.apps_score") AS FLOAT64) AS  apps_score,
#  SAFE_CAST(JSON_VALUE(r.calcFeatures, "$.c_demo_score") AS FLOAT64) AS  c_demo_score,
#  SAFE_CAST(JSON_VALUE(r.calcFeatures, "$.c_credo_score") AS FLOAT64) AS  c_credo_score,
#  SAFE_CAST(JSON_VALUE(r.calcFeatures, "$.c_tx_score") AS FLOAT64) AS  c_tx_score,
#  SAFE_CAST(JSON_VALUE(r.calcFeatures, "$.ca_cic_score") AS FLOAT64) AS  ca_cic_score,
# coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  r.start_time) AS appln_submit_datetime,
#   loanmaster.disbursementDateTime,
#   format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  r.start_time)) as Application_month, 
# FROM latest_request r
# left join model_run m
# on r.digitalLoanAccountId = m.digitalLoanAccountId 
# left join risk_credit_mis.loan_master_table loanmaster 
#   ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
# ) where trenchCategory = 'Trench 1'
# ;
# """

# df = client.query(sq).to_dataframe()
# df.head()

### run the psi calculation

In [30]:
df = df.copy()

# Define feature list
feature_list = ['aStackScore',  
                    'calc_apps_score', 
                    'calc_c_demo_score', 
                    'calc_c_credo_score', 
                    'calc_c_tx_score', 
                    'calc_ca_cic_score'
]

# Define segment columns
segment_columns = ['new_loan_type', 'gender', 'osType', 'trenchCategory', 'product']
# Calculate month-on-month PSI
psi_results = calculate_month_on_month_psi(df, feature_list, segment_columns)
psi_results['modelDisplayName'] = df['modelDisplayName'].iloc[0]
psi_results['Model_Name'] = df['Model_Name'].iloc[0]
psi_results['modelVersionId'] = df['modelVersionId'].iloc[0]
psi_results['trenchCategory'] = df['trenchCategory'].iloc[0]
psi_results = psi_results[[ 'modelDisplayName', 'Model_Name',  'modelVersionId', 'trenchCategory','Feature', 'Feature_Type', 'Segment_Column', 'Segment_Value', 'Month',
       'Base_Month', 'Current_Month', 'PSI']].copy()

# Calculate bin-level PSI
bin_psi_results = calculate_bin_level_psi(df, feature_list, segment_columns)
bin_psi_results['modelDisplayName'] = df['modelDisplayName'].iloc[0]
bin_psi_results['Model_Name'] = df['Model_Name'].iloc[0]
bin_psi_results['modelVersionId'] = df['modelVersionId'].iloc[0]
bin_psi_results['trenchCategory'] = df['trenchCategory'].iloc[0]
bin_psi_results = bin_psi_results[[ 'modelDisplayName', 'Model_Name',  'modelVersionId', 'trenchCategory','Feature', 'Feature_Type', 'Segment_Column', 'Segment_Value', 'Month',
       'Base_Month', 'Current_Month', 'Bin', 'Bin_Range', 'Bin_Min', 'Bin_Max', 'Base_Percentage',
       'Actual_Percentage', 'Bin_PSI']].copy()

# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.temp_csi_new_monitoring_data"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(psi_results, table_id, job_config=job_config)
job.result()  # Wait for the job to complete


# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.temp_csi_new_monitoring_data_feature_bin_level"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(bin_psi_results, table_id, job_config=job_config)
job.result()  # Wait for the job to complete

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=d68987b4-b633-4a49-9e51-f3e543273979>

In [31]:
psi_results

,modelDisplayName,Model_Name,modelVersionId,trenchCategory,Feature,Feature_Type,Segment_Column,Segment_Value,Month,Base_Month,Current_Month,PSI
0,Alpha-Cash-Stack-Model,Alpha-Cash-Stack-Model_Trench1,v1,Trench 1,aStackScore,numerical,Overall,All,2025-09,2025-09,2025-09,0.000000
1,Alpha-Cash-Stack-Model,Alpha-Cash-Stack-Model_Trench1,v1,Trench 1,aStackScore,numerical,Overall,All,2025-10,2025-09,2025-10,0.242386
2,Alpha-Cash-Stack-Model,Alpha-Cash-Stack-Model_Trench1,v1,Trench 1,calc_apps_score,numerical,Overall,All,2025-09,2025-09,2025-09,0.000000
3,Alpha-Cash-Stack-Model,Alpha-Cash-Stack-Model_Trench1,v1,Trench 1,calc_apps_score,numerical,Overall,All,2025-10,2025-09,2025-10,0.331284
4,Alpha-Cash-Stack-Model,Alpha-Cash-Stack-Model_Trench1,v1,Trench 1,calc_c_demo_score,numerical,Overall,All,2025-09,2025-09,2025-09,0.000000
5,Alpha-Cash-Stack-Model,Alpha-Cash-Stack-Model_Trench1,v1,Trench 1,calc_c_demo_score,numerical,Overall,All,2025-10,2025-09,2025-10,0.257479
6,Alpha-Cash-Stack-Model,Alpha-Cash-Stack-Model_Trench1,v1,Trench 1,calc_c_credo_score,numerical,Overall,All,2025-09,2025-09,2025-09,0.000000
7,Alpha-Cash-Stack-Model,Alpha-Cash-Stack-Model_Trench1,v1,Trench 1,calc_c_credo_score,numerical,Overall,All,2025-10,2025-09,2025-10,0.301217
8,Alpha-Cash-Stack-Model,Alpha-Cash-Stack-Model_Trench1,v1,Trench 1,calc_c_tx_score,categorical,Overall,All,2025-09,2025-09,2025-09,0.000000
9,Alpha-Cash-Stack-Model,Alpha-Cash-Stack-Model_Trench1,v1,Trench 1,calc_c_tx_score,categorical,Overall,All,2025-10,2025-09,2025-10,0.000000


## Beta-Cash-Demo-Model (Trench1,2,3)

In [32]:
sq = r"""
WITH parsed as (
  select customerId, digitalLoanAccountId,modelDisplayName,modelVersionId,start_time,end_time,prediction,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures
FROM `prj-prod-dataplatform.audit_balance.ml_model_run_details`
where modelDisplayName = 'Beta-Cash-Demo-Model'),

latest_request as (
select * from parsed
QUALIFY ROW_NUMBER() OVER (PARTITION BY customerId, digitalLoanAccountId,modelDisplayName ORDER BY start_time DESC ) = 1),

model_run as (
select customerId,digitalLoanAccountId,modelName, publish_time,
REPLACE(REPLACE(requestPayload, "'", '"'), "None", "null") AS requestPayload_clean
from `prj-prod-dataplatform.audit_balance.ml_request_details` 
WHERE modelName = 'Beta-Cash-Model-response'
QUALIFY ROW_NUMBER() OVER (PARTITION BY customerId, digitalLoanAccountId,modelName ORDER BY publish_time DESC ) = 1)

select * from (select 
 r.customerId,
 r.digitalLoanAccountId,
 r.prediction Beta_Cash_Demo_Score,
 r.start_time,
 r.end_time,
 r.modelDisplayName,
 r.modelVersionId,
 loanmaster.new_loan_type,
 loanmaster.gender,
  'Beta-Cash-Demo-Model_All_Trench' Model_Name,
  'Cash' as product,
  JSON_VALUE(m.requestPayload_clean, "$.predictions.trenchCategory") AS trenchCategory,
  JSON_VALUE(m.requestPayload_clean, "$.predictions.loanType") AS loanType,
  --  Demo Score Model Features for all Trench 1,2,3
  calcFeatures,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  r.start_time) AS appln_submit_datetime,
  loanmaster.disbursementDateTime,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  r.start_time)) as Application_month, 
FROM latest_request r
left join model_run m
on r.digitalLoanAccountId = m.digitalLoanAccountId 
left join risk_credit_mis.loan_master_table loanmaster 
  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
qualify row_number() over (partition by r.customerId,r.digitalLoanAccountId order by r.start_time desc) = 1
)
-- where trenchCategory = 'Trench 1'
;
"""

dfd = client.query(sq).to_dataframe()
dfd.head()

,customerId,digitalLoanAccountId,Beta_Cash_Demo_Score,start_time,end_time,modelDisplayName,modelVersionId,new_loan_type,gender,Model_Name,product,trenchCategory,loanType,calcFeatures,appln_submit_datetime,disbursementDateTime,Application_month
0,1904854,a2ee582c-0ecd-4137-a0ec-4366f341c225,0.4878703331870483,2025-10-13 09:57:01.444621,2025-10-13 09:57:02.436508,Beta-Cash-Demo-Model,v1,Quick,F,Beta-Cash-Demo-Model_All_Trench,Cash,Trench 2,Quick,"{""ln_vas_opted_flag"": 1, ""ln_self_dec_income"":...",2025-10-13 17:56:47.000000,2025-10-13 18:39:52,2025-10
1,2256883,6bee918a-7f83-49ad-8144-d9192c070520,0.37814267688221087,2025-10-11 05:57:31.420195,2025-10-11 05:57:32.296993,Beta-Cash-Demo-Model,v1,Quick,M,Beta-Cash-Demo-Model_All_Trench,Cash,Trench 2,Quick,"{""ln_vas_opted_flag"": 1, ""ln_self_dec_income"":...",2025-10-11 13:57:28.000000,NaT,2025-10
2,2259034,d084cc6b-92f5-4eeb-a594-b52fb0cc7923,0.6942363571711032,2025-10-14 06:38:29.733713,2025-10-14 06:38:30.522382,Beta-Cash-Demo-Model,v1,Quick,F,Beta-Cash-Demo-Model_All_Trench,Cash,Trench 2,Quick,"{""ln_vas_opted_flag"": 1, ""ln_self_dec_income"":...",2025-10-14 14:38:19.000000,NaT,2025-10
3,2265407,04d428e1-bf86-4273-82d7-0e39110ea12d,0.4893855487313892,2025-10-15 01:49:51.683948,2025-10-15 01:49:52.528262,Beta-Cash-Demo-Model,v1,None,None,Beta-Cash-Demo-Model_All_Trench,Cash,Trench 3,Quick,"{""ln_vas_opted_flag"": 1, ""ln_self_dec_income"":...",2025-10-15 01:49:51.683948,NaT,2025-10
4,2523735,994ad6aa-a5de-44c6-bd76-a8b26fa94ed2,0.4548147556873481,2025-10-09 09:42:29.760492,2025-10-09 09:42:30.599343,Beta-Cash-Demo-Model,v1,Quick,F,Beta-Cash-Demo-Model_All_Trench,Cash,Trench 3,Quick,"{""ln_vas_opted_flag"": 0, ""ln_self_dec_income"":...",2025-10-09 17:42:26.000000,NaT,2025-10


### expand df

In [33]:
# Expand the calcFeatures column
expanded_df = expand_calc_features(dfd)

# Display the result
print(f"Original columns: {dfd.shape[1]}")
print(f"Expanded columns: {expanded_df.shape[1]}")
expanded_df.head()


Original columns: 17
Expanded columns: 33


,customerId,digitalLoanAccountId,Beta_Cash_Demo_Score,start_time,end_time,modelDisplayName,modelVersionId,new_loan_type,gender,Model_Name,product,trenchCategory,loanType,calcFeatures,appln_submit_datetime,disbursementDateTime,Application_month,calc_ln_vas_opted_flag,calc_ln_self_dec_income,calc_ln_age,calc_ln_source_funds_new_bin,calc_ln_loan_level_user_type,calc_ln_industry_new_cat_bin,calc_ln_marital_status,calc_ln_doc_type_rolled,calc_ln_education_level,calc_ln_ref2_type,calc_ln_email_primary_domain,calc_ln_os_type,calc_ln_mature_fspd30_flag,calc_ln_fspd30_flag,calc_ln_appln_submit_datetime,calc_ln_province_bin
0,1904854,a2ee582c-0ecd-4137-a0ec-4366f341c225,0.4878703331870483,2025-10-13 09:57:01.444621,2025-10-13 09:57:02.436508,Beta-Cash-Demo-Model,v1,Quick,F,Beta-Cash-Demo-Model_All_Trench,Cash,Trench 2,Quick,"{""ln_vas_opted_flag"": 1, ""ln_self_dec_income"":...",2025-10-13 17:56:47.000000,2025-10-13 18:39:52,2025-10,1,30000,42,salary,2_New Applicant,3.0,Married,umid,College Graduate,Spouse,gmail.com,0,None,None,2025-10-13T09:56:47+00:00,cavite
1,2256883,6bee918a-7f83-49ad-8144-d9192c070520,0.37814267688221087,2025-10-11 05:57:31.420195,2025-10-11 05:57:32.296993,Beta-Cash-Demo-Model,v1,Quick,M,Beta-Cash-Demo-Model_All_Trench,Cash,Trench 2,Quick,"{""ln_vas_opted_flag"": 1, ""ln_self_dec_income"":...",2025-10-11 13:57:28.000000,NaT,2025-10,1,135000,38,salary,2_New Applicant,2.0,Married,less_frequent_cat,College Graduate,Parent,gmail.com,0,None,None,2025-10-11T05:57:28+00:00,pampanga
2,2259034,d084cc6b-92f5-4eeb-a594-b52fb0cc7923,0.6942363571711032,2025-10-14 06:38:29.733713,2025-10-14 06:38:30.522382,Beta-Cash-Demo-Model,v1,Quick,F,Beta-Cash-Demo-Model_All_Trench,Cash,Trench 2,Quick,"{""ln_vas_opted_flag"": 1, ""ln_self_dec_income"":...",2025-10-14 14:38:19.000000,NaT,2025-10,1,7000,21,less_frequent_cat,2_New Applicant,missing,Single,driving license,College Undergraduate,Friend,gmail.com,1,None,None,2025-10-14T06:38:19+00:00,others
3,2265407,04d428e1-bf86-4273-82d7-0e39110ea12d,0.4893855487313892,2025-10-15 01:49:51.683948,2025-10-15 01:49:52.528262,Beta-Cash-Demo-Model,v1,None,None,Beta-Cash-Demo-Model_All_Trench,Cash,Trench 3,Quick,"{""ln_vas_opted_flag"": 1, ""ln_self_dec_income"":...",2025-10-15 01:49:51.683948,NaT,2025-10,1,80000,27,salary,1_Repeat Applicant,1.0,Single,driving license,College Graduate,Friend,gmail.com,0,None,None,2025-10-15T01:49:48+00:00,metro manila
4,2523735,994ad6aa-a5de-44c6-bd76-a8b26fa94ed2,0.4548147556873481,2025-10-09 09:42:29.760492,2025-10-09 09:42:30.599343,Beta-Cash-Demo-Model,v1,Quick,F,Beta-Cash-Demo-Model_All_Trench,Cash,Trench 3,Quick,"{""ln_vas_opted_flag"": 0, ""ln_self_dec_income"":...",2025-10-09 17:42:26.000000,NaT,2025-10,0,15000,31,remittance,1_Repeat Applicant,missing,Married,umid,College Graduate,Spouse,gmail.com,0,None,None,2025-10-09T09:42:26+00:00,metro manila


In [34]:
df = expanded_df.drop(columns=['calcFeatures']).copy()

In [35]:
bin_columns = [col for col in df.columns if 'bin' in col.lower()]
df[bin_columns] = df[bin_columns].astype('category')

### run the psi calculation

In [36]:
df = df.copy()

# Define feature list
feature_list = ['Beta_Cash_Demo_Score',  
                    'calc_ln_vas_opted_flag', 'calc_ln_self_dec_income', 'calc_ln_age',
       'calc_ln_source_funds_new_bin', 'calc_ln_loan_level_user_type',
       'calc_ln_industry_new_cat_bin', 'calc_ln_marital_status',
       'calc_ln_doc_type_rolled', 'calc_ln_education_level',
       'calc_ln_ref2_type', 'calc_ln_email_primary_domain', 'calc_ln_os_type', 'calc_ln_province_bin' 
]

# Define segment columns
segment_columns = ['new_loan_type', 'gender', 'osType', 'trenchCategory', 'product']
# Calculate month-on-month PSI
psi_results = calculate_month_on_month_psi(df, feature_list, segment_columns)
psi_results['modelDisplayName'] = df['modelDisplayName'].iloc[0]
psi_results['Model_Name'] = df['Model_Name'].iloc[0]
psi_results['modelVersionId'] = df['modelVersionId'].iloc[0]
psi_results['trenchCategory'] = df['trenchCategory'].iloc[0]
psi_results = psi_results[[ 'modelDisplayName', 'Model_Name',  'modelVersionId', 'trenchCategory','Feature', 'Feature_Type', 'Segment_Column', 'Segment_Value', 'Month',
       'Base_Month', 'Current_Month', 'PSI']].copy()

# Calculate bin-level PSI
bin_psi_results = calculate_bin_level_psi(df, feature_list, segment_columns)
bin_psi_results['modelDisplayName'] = df['modelDisplayName'].iloc[0]
bin_psi_results['Model_Name'] = df['Model_Name'].iloc[0]
bin_psi_results['modelVersionId'] = df['modelVersionId'].iloc[0]
bin_psi_results['trenchCategory'] = df['trenchCategory'].iloc[0]
bin_psi_results = bin_psi_results[[ 'modelDisplayName', 'Model_Name',  'modelVersionId', 'trenchCategory','Feature', 'Feature_Type', 'Segment_Column', 'Segment_Value', 'Month',
       'Base_Month', 'Current_Month', 'Bin', 'Bin_Range', 'Bin_Min', 'Bin_Max', 'Base_Percentage',
       'Actual_Percentage', 'Bin_PSI']].copy()

# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.temp_csi_new_monitoring_data"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(psi_results, table_id, job_config=job_config)
job.result()  # Wait for the job to complete


# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.temp_csi_new_monitoring_data_feature_bin_level"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(bin_psi_results, table_id, job_config=job_config)
job.result()  # Wait for the job to complete

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=a4ba8073-8e44-45d2-a784-73d40947bc2b>

In [37]:
psi_results

,modelDisplayName,Model_Name,modelVersionId,trenchCategory,Feature,Feature_Type,Segment_Column,Segment_Value,Month,Base_Month,Current_Month,PSI
0,Beta-Cash-Demo-Model,Beta-Cash-Demo-Model_All_Trench,v1,Trench 2,Beta_Cash_Demo_Score,categorical,Overall,All,2025-09,2025-09,2025-09,0.000000
1,Beta-Cash-Demo-Model,Beta-Cash-Demo-Model_All_Trench,v1,Trench 2,Beta_Cash_Demo_Score,categorical,Overall,All,2025-10,2025-09,2025-10,0.017460
2,Beta-Cash-Demo-Model,Beta-Cash-Demo-Model_All_Trench,v1,Trench 2,calc_ln_vas_opted_flag,categorical,Overall,All,2025-09,2025-09,2025-09,0.000000
3,Beta-Cash-Demo-Model,Beta-Cash-Demo-Model_All_Trench,v1,Trench 2,calc_ln_vas_opted_flag,categorical,Overall,All,2025-10,2025-09,2025-10,0.000000
4,Beta-Cash-Demo-Model,Beta-Cash-Demo-Model_All_Trench,v1,Trench 2,calc_ln_self_dec_income,numerical,Overall,All,2025-09,2025-09,2025-09,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
219,Beta-Cash-Demo-Model,Beta-Cash-Demo-Model_All_Trench,v1,Trench 2,calc_ln_email_primary_domain,categorical,product,Cash,2025-10,2025-09,2025-10,0.003260
220,Beta-Cash-Demo-Model,Beta-Cash-Demo-Model_All_Trench,v1,Trench 2,calc_ln_os_type,categorical,product,Cash,2025-09,2025-09,2025-09,0.000000
221,Beta-Cash-Demo-Model,Beta-Cash-Demo-Model_All_Trench,v1,Trench 2,calc_ln_os_type,categorical,product,Cash,2025-10,2025-09,2025-10,0.005873
222,Beta-Cash-Demo-Model,Beta-Cash-Demo-Model_All_Trench,v1,Trench 2,calc_ln_province_bin,categorical,product,Cash,2025-09,2025-09,2025-09,0.000000


## Beta-Cash-AppScore-Model

In [38]:
sq = r"""  
WITH parsed as (
  select customerId, digitalLoanAccountId,modelDisplayName,modelVersionId,start_time,end_time,prediction,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
REPLACE(REPLACE(prediction, "'", '"'), "None", "null") AS prediction_clean
FROM `prj-prod-dataplatform.audit_balance.ml_model_run_details`
where modelDisplayName = 'Beta-Cash-AppScore-Model'),

latest_request as (
select * from parsed
QUALIFY ROW_NUMBER() OVER (PARTITION BY customerId, digitalLoanAccountId,modelDisplayName ORDER BY start_time DESC ) = 1),

model_run as (
select customerId,digitalLoanAccountId,modelName, publish_time,
REPLACE(REPLACE(requestPayload, "'", '"'), "None", "null") AS requestPayload_clean
from `prj-prod-dataplatform.audit_balance.ml_request_details` 
WHERE modelName = 'Beta-Cash-Model-response'
QUALIFY ROW_NUMBER() OVER (PARTITION BY customerId, digitalLoanAccountId,modelName ORDER BY publish_time DESC ) = 1)

select * from (
  select 
 r.customerId,
 r.digitalLoanAccountId,
 --r.prediction,
 r.start_time,
 r.end_time,
 r.modelDisplayName,
 r.modelVersionId,
 loanmaster.new_loan_type,
 loanmaster.gender,
 'Beta-Cash-AppScore-Model_Trench1' Model_Name,
 'Cash' as product,
  JSON_VALUE(m.requestPayload_clean, "$.predictions.trenchCategory") AS trenchCategory,
  JSON_VALUE(m.requestPayload_clean, "$.predictions.loanType") AS loanType,
  --  Beta App Score Model Features for all Trench 1
  SAFE_CAST(JSON_VALUE(r.prediction_clean, "$.combined_score") AS Float64) AS beta_cash_app_score,
  calcFeatures,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  r.start_time) AS appln_submit_datetime,
  loanmaster.disbursementDateTime,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  r.start_time)) as Application_month,  
FROM latest_request r
left join model_run m
on r.digitalLoanAccountId = m.digitalLoanAccountId 
left join risk_credit_mis.loan_master_table loanmaster 
  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
qualify row_number() over (partition by r.customerId,r.digitalLoanAccountId order by r.start_time desc) = 1
)
where trenchCategory = 'Trench 1'
;
"""

dfd = client.query(sq).to_dataframe()
dfd.head()

,customerId,digitalLoanAccountId,start_time,end_time,modelDisplayName,modelVersionId,new_loan_type,gender,Model_Name,product,trenchCategory,loanType,beta_cash_app_score,calcFeatures,appln_submit_datetime,disbursementDateTime,Application_month
0,3674645,a7985b67-678e-4994-8806-77a3cece1029,2025-10-06 04:59:13.628196,2025-10-06 04:59:13.904200,Beta-Cash-AppScore-Model,v1,Quick,F,Beta-Cash-AppScore-Model_Trench1,Cash,Trench 1,Quick,0.461768,"{""app_cnt_health_and_fitness_ever"": 0, ""app_cn...",2025-10-06 12:59:11,NaT,2025-10
1,3703922,881e6885-d571-48e7-9858-82e5454287a8,2025-10-12 02:53:06.897551,2025-10-12 02:53:07.151137,Beta-Cash-AppScore-Model,v1,Quick,M,Beta-Cash-AppScore-Model_Trench1,Cash,Trench 1,Quick,0.571944,"{""app_cnt_health_and_fitness_ever"": 0, ""app_cn...",2025-10-12 10:53:04,NaT,2025-10
2,3704817,9eacece9-a6bc-4f24-aac1-3910fc173630,2025-09-25 10:06:23.950946,2025-09-25 10:06:24.215688,Beta-Cash-AppScore-Model,v1,Quick,M,Beta-Cash-AppScore-Model_Trench1,Cash,Trench 1,Quick,0.549034,"{""app_cnt_health_and_fitness_ever"": 0, ""app_cn...",2025-09-25 18:06:21,NaT,2025-09
3,3709387,4a170bbc-80db-498b-af0f-9e12b7d411a9,2025-09-27 13:53:04.969967,2025-09-27 13:53:05.217779,Beta-Cash-AppScore-Model,v1,Quick,M,Beta-Cash-AppScore-Model_Trench1,Cash,Trench 1,Quick,0.639864,"{""app_cnt_health_and_fitness_ever"": 0, ""app_cn...",2025-09-27 21:53:02,NaT,2025-09
4,3715030,b6907d15-7d6f-4486-b2f7-35064a094ab0,2025-09-30 13:42:26.651941,2025-09-30 13:42:26.894434,Beta-Cash-AppScore-Model,v1,Quick,F,Beta-Cash-AppScore-Model_Trench1,Cash,Trench 1,Quick,0.554022,"{""app_cnt_health_and_fitness_ever"": 0, ""app_cn...",2025-09-30 21:42:23,NaT,2025-09


### expand df

In [39]:
# Expand the calcFeatures column
expanded_df = expand_calc_features(dfd)

# Display the result
print(f"Original columns: {dfd.shape[1]}")
print(f"Expanded columns: {expanded_df.shape[1]}")
expanded_df.head()


Original columns: 17
Expanded columns: 27


,customerId,digitalLoanAccountId,start_time,end_time,modelDisplayName,modelVersionId,new_loan_type,gender,Model_Name,product,trenchCategory,loanType,beta_cash_app_score,calcFeatures,appln_submit_datetime,disbursementDateTime,Application_month,calc_app_cnt_health_and_fitness_ever,calc_app_cnt_shopping_ever,calc_app_cnt_crypto_ever,calc_app_cnt_driver_ever,calc_app_cnt_payday_180d,calc_app_cnt_gambling_180d,calc_app_avg_time_bw_installed_mins_3d,calc_app_median_time_bw_installed_mins_3d,calc_app_avg_time_bw_installed_mins_ever,calc_app_median_time_bw_installed_mins_ever
0,3674645,a7985b67-678e-4994-8806-77a3cece1029,2025-10-06 04:59:13.628196,2025-10-06 04:59:13.904200,Beta-Cash-AppScore-Model,v1,Quick,F,Beta-Cash-AppScore-Model_Trench1,Cash,Trench 1,Quick,0.461768,"{""app_cnt_health_and_fitness_ever"": 0, ""app_cn...",2025-10-06 12:59:11,NaT,2025-10,0,2,0,0,6,0,NaN,NaN,5.072974e+05,32.066667
1,3703922,881e6885-d571-48e7-9858-82e5454287a8,2025-10-12 02:53:06.897551,2025-10-12 02:53:07.151137,Beta-Cash-AppScore-Model,v1,Quick,M,Beta-Cash-AppScore-Model_Trench1,Cash,Trench 1,Quick,0.571944,"{""app_cnt_health_and_fitness_ever"": 0, ""app_cn...",2025-10-12 10:53:04,NaT,2025-10,0,1,0,0,1,0,1.103417e+03,1103.416667,2.384940e+05,8683.166667
2,3704817,9eacece9-a6bc-4f24-aac1-3910fc173630,2025-09-25 10:06:23.950946,2025-09-25 10:06:24.215688,Beta-Cash-AppScore-Model,v1,Quick,M,Beta-Cash-AppScore-Model_Trench1,Cash,Trench 1,Quick,0.549034,"{""app_cnt_health_and_fitness_ever"": 0, ""app_cn...",2025-09-25 18:06:21,NaT,2025-09,0,2,2,0,1,0,4.621683e+06,4.841667,2.017317e+06,13.116667
3,3709387,4a170bbc-80db-498b-af0f-9e12b7d411a9,2025-09-27 13:53:04.969967,2025-09-27 13:53:05.217779,Beta-Cash-AppScore-Model,v1,Quick,M,Beta-Cash-AppScore-Model_Trench1,Cash,Trench 1,Quick,0.639864,"{""app_cnt_health_and_fitness_ever"": 0, ""app_cn...",2025-09-27 21:53:02,NaT,2025-09,0,0,0,0,0,0,5.345533e+02,273.200000,7.535356e+02,128.066667
4,3715030,b6907d15-7d6f-4486-b2f7-35064a094ab0,2025-09-30 13:42:26.651941,2025-09-30 13:42:26.894434,Beta-Cash-AppScore-Model,v1,Quick,F,Beta-Cash-AppScore-Model_Trench1,Cash,Trench 1,Quick,0.554022,"{""app_cnt_health_and_fitness_ever"": 0, ""app_cn...",2025-09-30 21:42:23,NaT,2025-09,0,1,0,0,0,0,NaN,NaN,9.460105e+03,27.341667


In [40]:
df = expanded_df.drop(columns=['calcFeatures']).copy()

### run the psi calculation

In [41]:
df = df.copy()

# Define feature list
feature_list = ['beta_cash_app_score',  
                    'calc_app_cnt_health_and_fitness_ever', 'calc_app_cnt_shopping_ever',
       'calc_app_cnt_crypto_ever', 'calc_app_cnt_driver_ever',
       'calc_app_cnt_payday_180d', 'calc_app_cnt_gambling_180d',
       'calc_app_avg_time_bw_installed_mins_3d',
       'calc_app_median_time_bw_installed_mins_3d',
       'calc_app_avg_time_bw_installed_mins_ever',
       'calc_app_median_time_bw_installed_mins_ever'
]

# Define segment columns
segment_columns = ['new_loan_type', 'gender', 'osType', 'trenchCategory', 'product']
# Calculate month-on-month PSI
psi_results = calculate_month_on_month_psi(df, feature_list, segment_columns)
psi_results['modelDisplayName'] = df['modelDisplayName'].iloc[0]
psi_results['Model_Name'] = df['Model_Name'].iloc[0]
psi_results['modelVersionId'] = df['modelVersionId'].iloc[0]
psi_results['trenchCategory'] = df['trenchCategory'].iloc[0]
psi_results = psi_results[[ 'modelDisplayName', 'Model_Name',   'modelVersionId', 'trenchCategory','Feature', 'Feature_Type', 'Segment_Column', 'Segment_Value', 'Month',
       'Base_Month', 'Current_Month', 'PSI']].copy()

# Calculate bin-level PSI
bin_psi_results = calculate_bin_level_psi(df, feature_list, segment_columns)
bin_psi_results['modelDisplayName'] = df['modelDisplayName'].iloc[0]
bin_psi_results['Model_Name'] = df['Model_Name'].iloc[0]
bin_psi_results['modelVersionId'] = df['modelVersionId'].iloc[0]
bin_psi_results['trenchCategory'] = df['trenchCategory'].iloc[0]
bin_psi_results = bin_psi_results[[ 'modelDisplayName', 'Model_Name',  'modelVersionId', 'trenchCategory','Feature', 'Feature_Type', 'Segment_Column', 'Segment_Value', 'Month',
       'Base_Month', 'Current_Month', 'Bin', 'Bin_Range', 'Bin_Min', 'Bin_Max', 'Base_Percentage',
       'Actual_Percentage', 'Bin_PSI']].copy()

# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.temp_csi_new_monitoring_data"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(psi_results, table_id, job_config=job_config)
job.result()  # Wait for the job to complete


# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.temp_csi_new_monitoring_data_feature_bin_level"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(bin_psi_results, table_id, job_config=job_config)
job.result()  # Wait for the job to complete

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=9db3db46-2821-456b-9c0e-926483be7675>

In [42]:
psi_results

,modelDisplayName,Model_Name,modelVersionId,trenchCategory,Feature,Feature_Type,Segment_Column,Segment_Value,Month,Base_Month,Current_Month,PSI
0,Beta-Cash-AppScore-Model,Beta-Cash-AppScore-Model_Trench1,v1,Trench 1,beta_cash_app_score,numerical,Overall,All,2025-09,2025-09,2025-09,0.000000
1,Beta-Cash-AppScore-Model,Beta-Cash-AppScore-Model_Trench1,v1,Trench 1,beta_cash_app_score,numerical,Overall,All,2025-10,2025-09,2025-10,0.084782
2,Beta-Cash-AppScore-Model,Beta-Cash-AppScore-Model_Trench1,v1,Trench 1,calc_app_cnt_health_and_fitness_ever,categorical,Overall,All,2025-09,2025-09,2025-09,0.000000
3,Beta-Cash-AppScore-Model,Beta-Cash-AppScore-Model_Trench1,v1,Trench 1,calc_app_cnt_health_and_fitness_ever,categorical,Overall,All,2025-10,2025-09,2025-10,0.000000
4,Beta-Cash-AppScore-Model,Beta-Cash-AppScore-Model_Trench1,v1,Trench 1,calc_app_cnt_shopping_ever,numerical,Overall,All,2025-09,2025-09,2025-09,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
127,Beta-Cash-AppScore-Model,Beta-Cash-AppScore-Model_Trench1,v1,Trench 1,calc_app_median_time_bw_installed_mins_3d,numerical,product,Cash,2025-10,2025-09,2025-10,0.039222
128,Beta-Cash-AppScore-Model,Beta-Cash-AppScore-Model_Trench1,v1,Trench 1,calc_app_avg_time_bw_installed_mins_ever,numerical,product,Cash,2025-09,2025-09,2025-09,0.000000
129,Beta-Cash-AppScore-Model,Beta-Cash-AppScore-Model_Trench1,v1,Trench 1,calc_app_avg_time_bw_installed_mins_ever,numerical,product,Cash,2025-10,2025-09,2025-10,0.033667
130,Beta-Cash-AppScore-Model,Beta-Cash-AppScore-Model_Trench1,v1,Trench 1,calc_app_median_time_bw_installed_mins_ever,numerical,product,Cash,2025-09,2025-09,2025-09,0.000000


## Beta-Cash-Stack-Model Trench1

In [43]:
sq = r""" 
WITH parsed as (
  select customerId, digitalLoanAccountId,modelDisplayName,modelVersionId,start_time,end_time,prediction,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
--REPLACE(REPLACE(prediction, "'", '"'), "None", "null") AS prediction_clean
FROM `prj-prod-dataplatform.audit_balance.ml_model_run_details`
where modelDisplayName = 'Beta-Cash-Stack-Model'),

latest_request as (
select * from parsed
QUALIFY ROW_NUMBER() OVER (PARTITION BY customerId, digitalLoanAccountId,modelDisplayName ORDER BY start_time DESC ) = 1),

model_run as (
select customerId,digitalLoanAccountId,modelName, publish_time,
REPLACE(REPLACE(requestPayload, "'", '"'), "None", "null") AS requestPayload_clean
from `prj-prod-dataplatform.audit_balance.ml_request_details` 
WHERE modelName = 'Beta-Cash-Model-response'
QUALIFY ROW_NUMBER() OVER (PARTITION BY customerId, digitalLoanAccountId,modelName ORDER BY publish_time DESC ) = 1)

select * from (
  select 
 r.customerId,
 r.digitalLoanAccountId,
 r.prediction Beta_cash_stack_score,
 r.start_time,
 r.end_time,
 r.modelDisplayName,
  r.modelVersionId,
  loanmaster.new_loan_type,
 loanmaster.gender,
 'Beta-Cash-Stack-Model_Trench1' Model_Name,
 'Cash' as product,
 JSON_VALUE(m.requestPayload_clean, "$.predictions.trenchCategory") AS trenchCategory,
r.calcFeatures,
coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  r.start_time) AS appln_submit_datetime,
  loanmaster.disbursementDateTime,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  r.start_time)) as Application_month,  
FROM latest_request r
left join model_run m
on r.digitalLoanAccountId = m.digitalLoanAccountId 
left join risk_credit_mis.loan_master_table loanmaster 
  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
qualify row_number() over (partition by r.customerId,r.digitalLoanAccountId order by r.start_time desc) = 1
  )
 where trenchCategory = 'Trench 1'
;
"""

dfd = client.query(sq).to_dataframe()
dfd.head()

,customerId,digitalLoanAccountId,Beta_cash_stack_score,start_time,end_time,modelDisplayName,modelVersionId,new_loan_type,gender,Model_Name,product,trenchCategory,calcFeatures,appln_submit_datetime,disbursementDateTime,Application_month
0,3701246,9c698895-ba1e-4a18-a058-c4cae1311e82,0.0455749555692561,2025-10-03 12:32:42.844348,2025-10-03 12:32:43.096830,Beta-Cash-Stack-Model,v1,Quick,M,Beta-Cash-Stack-Model_Trench1,Cash,Trench 1,"{""demo_score"": ""0.48297651318600027"", ""credo_s...",2025-10-03 20:32:40,2025-10-03 20:54:23,2025-10
1,3701417,e746decb-fc45-46ba-801d-a58deaca08ab,0.4800245105185423,2025-09-26 11:55:51.729715,2025-09-26 11:55:51.742039,Beta-Cash-Stack-Model,v1,Quick,F,Beta-Cash-Stack-Model_Trench1,Cash,Trench 1,"{""demo_score"": ""0.5536176612461192"", ""credo_sc...",2025-09-26 19:55:49,NaT,2025-09
2,3704717,5220b9ca-f39c-4630-b6c9-45d47673a9aa,0.6841452993369396,2025-09-25 09:14:28.894876,2025-09-25 09:14:29.157369,Beta-Cash-Stack-Model,v1,Quick,F,Beta-Cash-Stack-Model_Trench1,Cash,Trench 1,"{""demo_score"": ""0.6352733030206352"", ""credo_sc...",2025-09-25 17:14:26,NaT,2025-09
3,3708627,7c468673-78d7-4584-8a3b-18ce26f4877a,0.5681545467502409,2025-09-27 08:16:05.207077,2025-09-27 08:16:05.470977,Beta-Cash-Stack-Model,v1,Quick,M,Beta-Cash-Stack-Model_Trench1,Cash,Trench 1,"{""demo_score"": ""0.5982026049071207"", ""credo_sc...",2025-09-27 16:16:02,NaT,2025-09
4,3708828,78617789-84f4-4385-b055-2439d506a1bd,0.7607497017690826,2025-10-11 15:59:38.953714,2025-10-11 15:59:38.965684,Beta-Cash-Stack-Model,v1,Quick,F,Beta-Cash-Stack-Model_Trench1,Cash,Trench 1,"{""demo_score"": ""0.677398920813756"", ""credo_sco...",2025-10-11 23:59:37,NaT,2025-10


### Expand_df

In [44]:
# Expand the calcFeatures column
expanded_df = expand_calc_features(dfd)

# Display the result
print(f"Original columns: {dfd.shape[1]}")
print(f"Expanded columns: {expanded_df.shape[1]}")
expanded_df.head()


Original columns: 16
Expanded columns: 20


,customerId,digitalLoanAccountId,Beta_cash_stack_score,start_time,end_time,modelDisplayName,modelVersionId,new_loan_type,gender,Model_Name,product,trenchCategory,calcFeatures,appln_submit_datetime,disbursementDateTime,Application_month,calc_demo_score,calc_credo_score,calc_trx_score,calc_app_score
0,3701246,9c698895-ba1e-4a18-a058-c4cae1311e82,0.0455749555692561,2025-10-03 12:32:42.844348,2025-10-03 12:32:43.096830,Beta-Cash-Stack-Model,v1,Quick,M,Beta-Cash-Stack-Model_Trench1,Cash,Trench 1,"{""demo_score"": ""0.48297651318600027"", ""credo_s...",2025-10-03 20:32:40,2025-10-03 20:54:23,2025-10,0.48297651318600027,0.1183,null,0.25866212168492725
1,3701417,e746decb-fc45-46ba-801d-a58deaca08ab,0.4800245105185423,2025-09-26 11:55:51.729715,2025-09-26 11:55:51.742039,Beta-Cash-Stack-Model,v1,Quick,F,Beta-Cash-Stack-Model_Trench1,Cash,Trench 1,"{""demo_score"": ""0.5536176612461192"", ""credo_sc...",2025-09-26 19:55:49,NaT,2025-09,0.5536176612461192,0.1307,null,NaN
2,3704717,5220b9ca-f39c-4630-b6c9-45d47673a9aa,0.6841452993369396,2025-09-25 09:14:28.894876,2025-09-25 09:14:29.157369,Beta-Cash-Stack-Model,v1,Quick,F,Beta-Cash-Stack-Model_Trench1,Cash,Trench 1,"{""demo_score"": ""0.6352733030206352"", ""credo_sc...",2025-09-25 17:14:26,NaT,2025-09,0.6352733030206352,0.2279,null,0.5339212522148606
3,3708627,7c468673-78d7-4584-8a3b-18ce26f4877a,0.5681545467502409,2025-09-27 08:16:05.207077,2025-09-27 08:16:05.470977,Beta-Cash-Stack-Model,v1,Quick,M,Beta-Cash-Stack-Model_Trench1,Cash,Trench 1,"{""demo_score"": ""0.5982026049071207"", ""credo_sc...",2025-09-27 16:16:02,NaT,2025-09,0.5982026049071207,0.3486,null,0.5004557587919182
4,3708828,78617789-84f4-4385-b055-2439d506a1bd,0.7607497017690826,2025-10-11 15:59:38.953714,2025-10-11 15:59:38.965684,Beta-Cash-Stack-Model,v1,Quick,F,Beta-Cash-Stack-Model_Trench1,Cash,Trench 1,"{""demo_score"": ""0.677398920813756"", ""credo_sco...",2025-10-11 23:59:37,NaT,2025-10,0.677398920813756,0.2933,null,NaN


In [45]:
df = expanded_df.drop(columns=['calcFeatures']).copy()

### run the psi calculation

In [46]:
df = df.copy()

# Define feature list
feature_list = ['Beta_cash_stack_score',  
                    'calc_demo_score', 'calc_credo_score',
       'calc_trx_score', 'calc_app_score'
]

# Define segment columns
segment_columns = ['new_loan_type', 'gender', 'osType', 'trenchCategory', 'product']
# Calculate month-on-month PSI
psi_results = calculate_month_on_month_psi(df, feature_list, segment_columns)
psi_results['modelDisplayName'] = df['modelDisplayName'].iloc[0]
psi_results['Model_Name'] = df['Model_Name'].iloc[0]
psi_results['modelVersionId'] = df['modelVersionId'].iloc[0]
psi_results['trenchCategory'] = df['trenchCategory'].iloc[0]
psi_results = psi_results[[ 'modelDisplayName', 'Model_Name',   'modelVersionId', 'trenchCategory','Feature', 'Feature_Type', 'Segment_Column', 'Segment_Value', 'Month',
       'Base_Month', 'Current_Month', 'PSI']].copy()

# Calculate bin-level PSI
bin_psi_results = calculate_bin_level_psi(df, feature_list, segment_columns)
bin_psi_results['modelDisplayName'] = df['modelDisplayName'].iloc[0]
bin_psi_results['Model_Name'] = df['Model_Name'].iloc[0]
bin_psi_results['modelVersionId'] = df['modelVersionId'].iloc[0]
bin_psi_results['trenchCategory'] = df['trenchCategory'].iloc[0]
bin_psi_results = bin_psi_results[[ 'modelDisplayName', 'Model_Name',  'modelVersionId', 'trenchCategory','Feature', 'Feature_Type', 'Segment_Column', 'Segment_Value', 'Month',
       'Base_Month', 'Current_Month', 'Bin', 'Bin_Range', 'Bin_Min', 'Bin_Max', 'Base_Percentage',
       'Actual_Percentage', 'Bin_PSI']].copy()
# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.temp_csi_new_monitoring_data"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(psi_results, table_id, job_config=job_config)
job.result()  # Wait for the job to complete


# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.temp_csi_new_monitoring_data_feature_bin_level"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(bin_psi_results, table_id, job_config=job_config)
job.result()  # Wait for the job to complete

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=95f7a83c-981c-4d24-a45a-bf9d0b170e51>

In [47]:
psi_results

,modelDisplayName,Model_Name,modelVersionId,trenchCategory,Feature,Feature_Type,Segment_Column,Segment_Value,Month,Base_Month,Current_Month,PSI
0,Beta-Cash-Stack-Model,Beta-Cash-Stack-Model_Trench1,v1,Trench 1,Beta_cash_stack_score,categorical,Overall,All,2025-09,2025-09,2025-09,0.000000
1,Beta-Cash-Stack-Model,Beta-Cash-Stack-Model_Trench1,v1,Trench 1,Beta_cash_stack_score,categorical,Overall,All,2025-10,2025-09,2025-10,0.030284
2,Beta-Cash-Stack-Model,Beta-Cash-Stack-Model_Trench1,v1,Trench 1,calc_demo_score,categorical,Overall,All,2025-09,2025-09,2025-09,0.000000
3,Beta-Cash-Stack-Model,Beta-Cash-Stack-Model_Trench1,v1,Trench 1,calc_demo_score,categorical,Overall,All,2025-10,2025-09,2025-10,0.021254
4,Beta-Cash-Stack-Model,Beta-Cash-Stack-Model_Trench1,v1,Trench 1,calc_credo_score,categorical,Overall,All,2025-09,2025-09,2025-09,0.000000
5,Beta-Cash-Stack-Model,Beta-Cash-Stack-Model_Trench1,v1,Trench 1,calc_credo_score,categorical,Overall,All,2025-10,2025-09,2025-10,0.030008
6,Beta-Cash-Stack-Model,Beta-Cash-Stack-Model_Trench1,v1,Trench 1,calc_trx_score,categorical,Overall,All,2025-09,2025-09,2025-09,0.000000
7,Beta-Cash-Stack-Model,Beta-Cash-Stack-Model_Trench1,v1,Trench 1,calc_trx_score,categorical,Overall,All,2025-10,2025-09,2025-10,0.000000
8,Beta-Cash-Stack-Model,Beta-Cash-Stack-Model_Trench1,v1,Trench 1,calc_app_score,categorical,Overall,All,2025-09,2025-09,2025-09,0.000000
9,Beta-Cash-Stack-Model,Beta-Cash-Stack-Model_Trench1,v1,Trench 1,calc_app_score,categorical,Overall,All,2025-10,2025-09,2025-10,0.038668


## Alpha-Cash-Stack-Model_Trench2

In [48]:
sq = r""" 
WITH parsed as (
  select customerId, digitalLoanAccountId,modelDisplayName,modelVersionId,start_time,end_time,prediction,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
FROM `prj-prod-dataplatform.audit_balance.ml_model_run_details`
where modelDisplayName = 'Alpha-Cash-Stack-Model'),

latest_request as (
select * from parsed
QUALIFY ROW_NUMBER() OVER (PARTITION BY customerId, digitalLoanAccountId,modelDisplayName ORDER BY start_time DESC ) = 1),

model_run as (
select customerId,digitalLoanAccountId,modelName, publish_time,requestPayload as requestPayload_clean
from `prj-prod-dataplatform.audit_balance.ml_request_details` 
WHERE modelName = 'Alpha-Cash-Model-response'
QUALIFY ROW_NUMBER() OVER (PARTITION BY customerId, digitalLoanAccountId,modelName ORDER BY publish_time DESC ) = 1)

select * from (
  select 
 r.customerId,
 r.digitalLoanAccountId,
 r.prediction aStackScore,
 r.start_time,
 r.end_time,
 r.modelDisplayName,
 r.modelVersionId,
 loanmaster.new_loan_type,
 loanmaster.gender,
'Alpha-Cash-Stack-Model_Trench2' Model_Name,
'Cash' as product,
  REGEXP_EXTRACT(m.requestPayload_clean, r"osType[:=]['\"]?([^'\"]+)['\"]?") AS osType,
  REGEXP_EXTRACT(m.requestPayload_clean, r"loanType[:=]['\"]?([^'\"]+)['\"]?") AS loanType,
  REGEXP_EXTRACT(m.requestPayload_clean, r"trenchCategory[:=]['\"]?([^'\"]+)['\"]?") AS trenchCategory,
r.calcFeatures,
 coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  r.start_time) AS appln_submit_datetime,
  loanmaster.disbursementDateTime,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  r.start_time)) as Application_month,  
FROM latest_request r
left join model_run m
on r.digitalLoanAccountId = m.digitalLoanAccountId 
left join risk_credit_mis.loan_master_table loanmaster 
  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
qualify row_number() over (partition by r.customerId,r.digitalLoanAccountId order by r.start_time desc) = 1
  
) where trenchCategory = 'Trench 2'
;
"""

dfd = client.query(sq).to_dataframe()
dfd.head()

,customerId,digitalLoanAccountId,aStackScore,start_time,end_time,modelDisplayName,modelVersionId,new_loan_type,gender,Model_Name,product,osType,loanType,trenchCategory,calcFeatures,appln_submit_datetime,disbursementDateTime,Application_month
0,2317696,6ec1545a-f258-44c4-8004-0e49a7b3b8be,0.2784156279383287,2025-10-04 09:09:49.054889,2025-10-04 09:09:49.269983,Alpha-Cash-Stack-Model,v1,Quick,M,Alpha-Cash-Stack-Model_Trench2,Cash,ios,Quick,Trench 2,"{""apps_score"": null, ""c_demo_score"": 0.5316426...",2025-10-04 17:09:25.000000,NaT,2025-10
1,2821279,a0ae09a1-7b1d-4906-9d7b-e300508b03c1,0.6962346545848281,2025-10-08 13:35:08.281290,2025-10-08 13:35:08.415377,Alpha-Cash-Stack-Model,v1,Quick,F,Alpha-Cash-Stack-Model_Trench2,Cash,ios,Quick,Trench 2,"{""apps_score"": null, ""c_demo_score"": 0.5798310...",2025-10-08 21:34:58.000000,NaT,2025-10
2,3233806,2963b2a9-4740-44d7-a8a4-a327dc578f3d,0.5806659184706724,2025-10-14 21:40:09.360359,2025-10-14 21:40:09.457266,Alpha-Cash-Stack-Model,v1,None,None,Alpha-Cash-Stack-Model_Trench2,Cash,ios,Quick,Trench 2,"{""apps_score"": null, ""c_demo_score"": 0.4841255...",2025-10-14 21:40:09.360359,NaT,2025-10
3,3273977,26092d61-4161-4060-9905-e6e29646e7d3,0.3326723808184682,2025-10-11 01:54:24.402546,2025-10-11 01:54:24.560715,Alpha-Cash-Stack-Model,v1,Quick,F,Alpha-Cash-Stack-Model_Trench2,Cash,android,Quick,Trench 2,"{""apps_score"": 0.4869644778993015, ""c_demo_sco...",2025-10-11 09:54:14.000000,NaT,2025-10
4,3342700,e48a781f-b577-4cda-99e8-66dc017de59e,0.29755415551182557,2025-10-14 08:31:07.856179,2025-10-14 08:31:08.030204,Alpha-Cash-Stack-Model,v1,Quick,M,Alpha-Cash-Stack-Model_Trench2,Cash,ios,Quick,Trench 2,"{""apps_score"": null, ""c_demo_score"": 0.4816481...",2025-10-14 16:30:54.000000,NaT,2025-10


### Expand_df

In [49]:
# Expand the calcFeatures column
expanded_df = expand_calc_features(dfd)

# Display the result
print(f"Original columns: {dfd.shape[1]}")
print(f"Expanded columns: {expanded_df.shape[1]}")
expanded_df.head()


Original columns: 18
Expanded columns: 23


,customerId,digitalLoanAccountId,aStackScore,start_time,end_time,modelDisplayName,modelVersionId,new_loan_type,gender,Model_Name,product,osType,loanType,trenchCategory,calcFeatures,appln_submit_datetime,disbursementDateTime,Application_month,calc_apps_score,calc_c_demo_score,calc_c_credo_score,calc_c_tx_score,calc_ca_cic_score
0,2317696,6ec1545a-f258-44c4-8004-0e49a7b3b8be,0.2784156279383287,2025-10-04 09:09:49.054889,2025-10-04 09:09:49.269983,Alpha-Cash-Stack-Model,v1,Quick,M,Alpha-Cash-Stack-Model_Trench2,Cash,ios,Quick,Trench 2,"{""apps_score"": null, ""c_demo_score"": 0.5316426...",2025-10-04 17:09:25.000000,NaT,2025-10,NaN,0.531643,0.2199,0.423681,0.351119
1,2821279,a0ae09a1-7b1d-4906-9d7b-e300508b03c1,0.6962346545848281,2025-10-08 13:35:08.281290,2025-10-08 13:35:08.415377,Alpha-Cash-Stack-Model,v1,Quick,F,Alpha-Cash-Stack-Model_Trench2,Cash,ios,Quick,Trench 2,"{""apps_score"": null, ""c_demo_score"": 0.5798310...",2025-10-08 21:34:58.000000,NaT,2025-10,NaN,0.579831,0.1958,0.312123,0.757514
2,3233806,2963b2a9-4740-44d7-a8a4-a327dc578f3d,0.5806659184706724,2025-10-14 21:40:09.360359,2025-10-14 21:40:09.457266,Alpha-Cash-Stack-Model,v1,None,None,Alpha-Cash-Stack-Model_Trench2,Cash,ios,Quick,Trench 2,"{""apps_score"": null, ""c_demo_score"": 0.4841255...",2025-10-14 21:40:09.360359,NaT,2025-10,NaN,0.484126,0.1958,0.517453,0.643696
3,3273977,26092d61-4161-4060-9905-e6e29646e7d3,0.3326723808184682,2025-10-11 01:54:24.402546,2025-10-11 01:54:24.560715,Alpha-Cash-Stack-Model,v1,Quick,F,Alpha-Cash-Stack-Model_Trench2,Cash,android,Quick,Trench 2,"{""apps_score"": 0.4869644778993015, ""c_demo_sco...",2025-10-11 09:54:14.000000,NaT,2025-10,0.486964,0.490319,0.4311,0.295836,0.457947
4,3342700,e48a781f-b577-4cda-99e8-66dc017de59e,0.29755415551182557,2025-10-14 08:31:07.856179,2025-10-14 08:31:08.030204,Alpha-Cash-Stack-Model,v1,Quick,M,Alpha-Cash-Stack-Model_Trench2,Cash,ios,Quick,Trench 2,"{""apps_score"": null, ""c_demo_score"": 0.4816481...",2025-10-14 16:30:54.000000,NaT,2025-10,NaN,0.481648,0.2050,0.520937,0.384391


In [50]:
df = expanded_df.drop(columns=['calcFeatures']).copy()

### run the psi calculation

In [51]:
df = df.copy()

# Define feature list
feature_list = ['aStackScore',  
                   'calc_apps_score', 'calc_c_demo_score',
       'calc_c_credo_score', 'calc_c_tx_score', 'calc_ca_cic_score'
]

# Define segment columns
segment_columns = ['new_loan_type', 'gender', 'osType', 'trenchCategory', 'product']
# Calculate month-on-month PSI
psi_results = calculate_month_on_month_psi(df, feature_list, segment_columns)
psi_results['modelDisplayName'] = df['modelDisplayName'].iloc[0]
psi_results['Model_Name'] = df['Model_Name'].iloc[0]
psi_results['modelVersionId'] = df['modelVersionId'].iloc[0]
psi_results['trenchCategory'] = df['trenchCategory'].iloc[0]
psi_results = psi_results[[ 'modelDisplayName', 'Model_Name',   'modelVersionId', 'trenchCategory','Feature', 'Feature_Type', 'Segment_Column', 'Segment_Value', 'Month',
       'Base_Month', 'Current_Month', 'PSI']].copy()

# Calculate bin-level PSI
bin_psi_results = calculate_bin_level_psi(df, feature_list, segment_columns)
bin_psi_results['modelDisplayName'] = df['modelDisplayName'].iloc[0]
bin_psi_results['Model_Name'] = df['Model_Name'].iloc[0]
bin_psi_results['modelVersionId'] = df['modelVersionId'].iloc[0]
bin_psi_results['trenchCategory'] = df['trenchCategory'].iloc[0]
bin_psi_results = bin_psi_results[[ 'modelDisplayName', 'Model_Name',  'modelVersionId', 'trenchCategory','Feature', 'Feature_Type', 'Segment_Column', 'Segment_Value', 'Month',
       'Base_Month', 'Current_Month', 'Bin', 'Bin_Range', 'Bin_Min', 'Bin_Max', 'Base_Percentage',
       'Actual_Percentage', 'Bin_PSI']].copy()

# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.temp_csi_new_monitoring_data"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(psi_results, table_id, job_config=job_config)
job.result()  # Wait for the job to complete


# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.temp_csi_new_monitoring_data_feature_bin_level"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(bin_psi_results, table_id, job_config=job_config)
job.result()  # Wait for the job to complete

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=c7c72250-4902-4195-ad54-ee11c70babdb>

In [52]:
psi_results

,modelDisplayName,Model_Name,modelVersionId,trenchCategory,Feature,Feature_Type,Segment_Column,Segment_Value,Month,Base_Month,Current_Month,PSI
0,Alpha-Cash-Stack-Model,Alpha-Cash-Stack-Model_Trench2,v1,Trench 2,aStackScore,categorical,Overall,All,2025-09,2025-09,2025-09,0.000000
1,Alpha-Cash-Stack-Model,Alpha-Cash-Stack-Model_Trench2,v1,Trench 2,aStackScore,categorical,Overall,All,2025-10,2025-09,2025-10,1.160233
2,Alpha-Cash-Stack-Model,Alpha-Cash-Stack-Model_Trench2,v1,Trench 2,calc_apps_score,numerical,Overall,All,2025-09,2025-09,2025-09,0.000000
3,Alpha-Cash-Stack-Model,Alpha-Cash-Stack-Model_Trench2,v1,Trench 2,calc_apps_score,numerical,Overall,All,2025-10,2025-09,2025-10,0.313180
4,Alpha-Cash-Stack-Model,Alpha-Cash-Stack-Model_Trench2,v1,Trench 2,calc_c_demo_score,numerical,Overall,All,2025-09,2025-09,2025-09,0.000000
5,Alpha-Cash-Stack-Model,Alpha-Cash-Stack-Model_Trench2,v1,Trench 2,calc_c_demo_score,numerical,Overall,All,2025-10,2025-09,2025-10,1.381989
6,Alpha-Cash-Stack-Model,Alpha-Cash-Stack-Model_Trench2,v1,Trench 2,calc_c_credo_score,numerical,Overall,All,2025-09,2025-09,2025-09,0.000000
7,Alpha-Cash-Stack-Model,Alpha-Cash-Stack-Model_Trench2,v1,Trench 2,calc_c_credo_score,numerical,Overall,All,2025-10,2025-09,2025-10,0.479137
8,Alpha-Cash-Stack-Model,Alpha-Cash-Stack-Model_Trench2,v1,Trench 2,calc_c_tx_score,numerical,Overall,All,2025-09,2025-09,2025-09,0.000000
9,Alpha-Cash-Stack-Model,Alpha-Cash-Stack-Model_Trench2,v1,Trench 2,calc_c_tx_score,numerical,Overall,All,2025-10,2025-09,2025-10,1.129771


## Beta-Cash-AppScore-Model_Trench2

In [53]:
sq = r""" 
WITH parsed as (
  select customerId, digitalLoanAccountId,modelDisplayName,modelVersionId,start_time,end_time,prediction,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
REPLACE(REPLACE(prediction, "'", '"'), "None", "null") AS prediction_clean
FROM `prj-prod-dataplatform.audit_balance.ml_model_run_details`
where modelDisplayName = 'Beta-Cash-AppScore-Model'),

latest_request as (
select * from parsed
QUALIFY ROW_NUMBER() OVER (PARTITION BY customerId, digitalLoanAccountId,modelDisplayName ORDER BY start_time DESC ) = 1),

model_run as (
select customerId,digitalLoanAccountId,modelName, publish_time,
REPLACE(REPLACE(requestPayload, "'", '"'), "None", "null") AS requestPayload_clean
from `prj-prod-dataplatform.audit_balance.ml_request_details` 
WHERE modelName = 'Beta-Cash-Model-response'
QUALIFY ROW_NUMBER() OVER (PARTITION BY customerId, digitalLoanAccountId,modelName ORDER BY publish_time DESC ) = 1)

select * from (
  select 
 r.customerId,
 r.digitalLoanAccountId,
 --r.prediction,
 r.start_time,
 r.end_time,
 r.modelDisplayName,
 r.modelVersionId,
 loanmaster.new_loan_type,
 loanmaster.gender,
 'Beta-Cash-AppScore-Model_Trench2' Model_Name,
 'Cash' as product,
  JSON_VALUE(m.requestPayload_clean, "$.predictions.trenchCategory") AS trenchCategory,
  JSON_VALUE(m.requestPayload_clean, "$.predictions.loanType") AS loanType,
  --  Beta App Score Model Features for all Trench 1
  SAFE_CAST(JSON_VALUE(r.prediction_clean, "$.combined_score") AS Float64) AS beta_cash_app_score,
  r.calcFeatures,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  r.start_time) AS appln_submit_datetime,
  loanmaster.disbursementDateTime,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  r.start_time)) as Application_month,  
FROM latest_request r
left join model_run m
on r.digitalLoanAccountId = m.digitalLoanAccountId 
left join risk_credit_mis.loan_master_table loanmaster 
  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
qualify row_number() over (partition by r.customerId,r.digitalLoanAccountId order by r.start_time desc) = 1
)
where trenchCategory = 'Trench 2'
;
"""

dfd = client.query(sq).to_dataframe()
dfd.head()

,customerId,digitalLoanAccountId,start_time,end_time,modelDisplayName,modelVersionId,new_loan_type,gender,Model_Name,product,trenchCategory,loanType,beta_cash_app_score,calcFeatures,appln_submit_datetime,disbursementDateTime,Application_month
0,3115833,7b788973-aae3-4a4d-8449-c5ed8c1fef76,2025-10-14 21:37:37.012560,2025-10-14 21:37:37.960088,Beta-Cash-AppScore-Model,v1,None,None,Beta-Cash-AppScore-Model_Trench2,Cash,Trench 2,Quick,0.506281,"{""app_cnt_health_and_fitness_ever"": 3, ""app_cn...",2025-10-14 21:37:37.012560,NaT,2025-10
1,3284593,f191ea13-2a2d-4599-b14d-cded7f874e3f,2025-10-15 01:35:10.847009,2025-10-15 01:35:11.686731,Beta-Cash-AppScore-Model,v1,None,None,Beta-Cash-AppScore-Model_Trench2,Cash,Trench 2,Quick,0.519411,"{""app_cnt_health_and_fitness_ever"": 0, ""app_cn...",2025-10-15 01:35:10.847009,NaT,2025-10
2,3557922,89cc6ed6-13d5-4f71-83ad-dcad99cb273d,2025-10-15 04:28:59.321252,2025-10-15 04:29:00.375682,Beta-Cash-AppScore-Model,v1,None,None,Beta-Cash-AppScore-Model_Trench2,Cash,Trench 2,Quick,0.529825,"{""app_cnt_health_and_fitness_ever"": 1, ""app_cn...",2025-10-15 04:28:59.321252,NaT,2025-10
3,3225619,848c8c30-7524-4f0a-a58b-701784756560,2025-10-15 05:19:11.208948,2025-10-15 05:19:12.421769,Beta-Cash-AppScore-Model,v1,None,None,Beta-Cash-AppScore-Model_Trench2,Cash,Trench 2,Quick,0.449731,"{""app_cnt_health_and_fitness_ever"": 1, ""app_cn...",2025-10-15 05:19:11.208948,NaT,2025-10
4,2315848,bc55b8f6-5395-40e0-b317-4114cba40602,2025-10-15 04:23:02.456585,2025-10-15 04:23:03.342167,Beta-Cash-AppScore-Model,v1,None,None,Beta-Cash-AppScore-Model_Trench2,Cash,Trench 2,Quick,0.545737,"{""app_cnt_health_and_fitness_ever"": 0, ""app_cn...",2025-10-15 04:23:02.456585,NaT,2025-10


### Expand_df

In [54]:
# Expand the calcFeatures column
expanded_df = expand_calc_features(dfd)

# Display the result
print(f"Original columns: {dfd.shape[1]}")
print(f"Expanded columns: {expanded_df.shape[1]}")
expanded_df.head()


Original columns: 17
Expanded columns: 27


,customerId,digitalLoanAccountId,start_time,end_time,modelDisplayName,modelVersionId,new_loan_type,gender,Model_Name,product,trenchCategory,loanType,beta_cash_app_score,calcFeatures,appln_submit_datetime,disbursementDateTime,Application_month,calc_app_cnt_health_and_fitness_ever,calc_app_cnt_shopping_ever,calc_app_cnt_crypto_ever,calc_app_cnt_driver_ever,calc_app_cnt_payday_180d,calc_app_cnt_gambling_180d,calc_app_avg_time_bw_installed_mins_3d,calc_app_median_time_bw_installed_mins_3d,calc_app_avg_time_bw_installed_mins_ever,calc_app_median_time_bw_installed_mins_ever
0,3115833,7b788973-aae3-4a4d-8449-c5ed8c1fef76,2025-10-14 21:37:37.012560,2025-10-14 21:37:37.960088,Beta-Cash-AppScore-Model,v1,None,None,Beta-Cash-AppScore-Model_Trench2,Cash,Trench 2,Quick,0.506281,"{""app_cnt_health_and_fitness_ever"": 3, ""app_cn...",2025-10-14 21:37:37.012560,NaT,2025-10,3,4,1,0,3,0,NaN,NaN,258604.223463,3128.966667
1,3284593,f191ea13-2a2d-4599-b14d-cded7f874e3f,2025-10-15 01:35:10.847009,2025-10-15 01:35:11.686731,Beta-Cash-AppScore-Model,v1,None,None,Beta-Cash-AppScore-Model_Trench2,Cash,Trench 2,Quick,0.519411,"{""app_cnt_health_and_fitness_ever"": 0, ""app_cn...",2025-10-15 01:35:10.847009,NaT,2025-10,0,2,1,0,13,0,NaN,NaN,98095.879074,2159.575000
2,3557922,89cc6ed6-13d5-4f71-83ad-dcad99cb273d,2025-10-15 04:28:59.321252,2025-10-15 04:29:00.375682,Beta-Cash-AppScore-Model,v1,None,None,Beta-Cash-AppScore-Model_Trench2,Cash,Trench 2,Quick,0.529825,"{""app_cnt_health_and_fitness_ever"": 1, ""app_cn...",2025-10-15 04:28:59.321252,NaT,2025-10,1,1,0,0,6,0,8.203937e+06,544.283333,665859.914151,777.866667
3,3225619,848c8c30-7524-4f0a-a58b-701784756560,2025-10-15 05:19:11.208948,2025-10-15 05:19:12.421769,Beta-Cash-AppScore-Model,v1,None,None,Beta-Cash-AppScore-Model_Trench2,Cash,Trench 2,Quick,0.449731,"{""app_cnt_health_and_fitness_ever"": 1, ""app_cn...",2025-10-15 05:19:11.208948,NaT,2025-10,1,3,0,0,5,0,1.337667e+02,133.766667,366822.782489,12.100000
4,2315848,bc55b8f6-5395-40e0-b317-4114cba40602,2025-10-15 04:23:02.456585,2025-10-15 04:23:03.342167,Beta-Cash-AppScore-Model,v1,None,None,Beta-Cash-AppScore-Model_Trench2,Cash,Trench 2,Quick,0.545737,"{""app_cnt_health_and_fitness_ever"": 0, ""app_cn...",2025-10-15 04:23:02.456585,NaT,2025-10,0,2,0,0,8,0,NaN,NaN,92956.212346,19144.916667


In [55]:
df = expanded_df.drop(columns=['calcFeatures']).copy()

In [56]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1727 entries, 0 to 1726
Data columns (total 26 columns):
 #   Column                                       Non-Null Count  Dtype         
---  ------                                       --------------  -----         
 0   customerId                                   1727 non-null   object        
 1   digitalLoanAccountId                         1727 non-null   object        
 2   start_time                                   1727 non-null   datetime64[us]
 3   end_time                                     1727 non-null   datetime64[us]
 4   modelDisplayName                             1727 non-null   object        
 5   modelVersionId                               1727 non-null   object        
 6   new_loan_type                                1642 non-null   object        
 7   gender                                       1642 non-null   object        
 8   Model_Name                                   1727 non-null   object        
 9

### run the psi calculation

In [57]:
df = df.copy()

# Define feature list
feature_list = ['beta_cash_app_score', 'calc_app_cnt_health_and_fitness_ever',  'calc_app_cnt_shopping_ever', 'calc_app_cnt_crypto_ever',
       'calc_app_cnt_driver_ever', 'calc_app_cnt_payday_180d',  'calc_app_cnt_gambling_180d', 'calc_app_avg_time_bw_installed_mins_3d',
       'calc_app_median_time_bw_installed_mins_3d', 'calc_app_avg_time_bw_installed_mins_ever', 'calc_app_median_time_bw_installed_mins_ever'
]

# Define segment columns
segment_columns = ['new_loan_type', 'gender', 'osType', 'trenchCategory', 'product']
# Calculate month-on-month PSI
psi_results = calculate_month_on_month_psi(df, feature_list, segment_columns)
psi_results['modelDisplayName'] = df['modelDisplayName'].iloc[0]
psi_results['Model_Name'] = df['Model_Name'].iloc[0]
psi_results['modelVersionId'] = df['modelVersionId'].iloc[0]
psi_results['trenchCategory'] = df['trenchCategory'].iloc[0]
psi_results = psi_results[[ 'modelDisplayName', 'Model_Name',   'modelVersionId', 'trenchCategory','Feature', 'Feature_Type', 'Segment_Column', 'Segment_Value', 'Month',
       'Base_Month', 'Current_Month', 'PSI']].copy()

# Calculate bin-level PSI
bin_psi_results = calculate_bin_level_psi(df, feature_list, segment_columns)
bin_psi_results['modelDisplayName'] = df['modelDisplayName'].iloc[0]
bin_psi_results['Model_Name'] = df['Model_Name'].iloc[0]
bin_psi_results['modelVersionId'] = df['modelVersionId'].iloc[0]
bin_psi_results['trenchCategory'] = df['trenchCategory'].iloc[0]
bin_psi_results = bin_psi_results[[ 'modelDisplayName', 'Model_Name',  'modelVersionId', 'trenchCategory','Feature', 'Feature_Type', 'Segment_Column', 'Segment_Value', 'Month',
       'Base_Month', 'Current_Month', 'Bin', 'Bin_Range', 'Bin_Min', 'Bin_Max', 'Base_Percentage',
       'Actual_Percentage', 'Bin_PSI']].copy()

# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.temp_csi_new_monitoring_data"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(psi_results, table_id, job_config=job_config)
job.result()  # Wait for the job to complete


# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.temp_csi_new_monitoring_data_feature_bin_level"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(bin_psi_results, table_id, job_config=job_config)
job.result()  # Wait for the job to complete

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=c52b5fcc-8c27-44d5-aebc-6176c5f3c78e>

In [58]:
psi_results

,modelDisplayName,Model_Name,modelVersionId,trenchCategory,Feature,Feature_Type,Segment_Column,Segment_Value,Month,Base_Month,Current_Month,PSI
0,Beta-Cash-AppScore-Model,Beta-Cash-AppScore-Model_Trench2,v1,Trench 2,beta_cash_app_score,numerical,Overall,All,2025-09,2025-09,2025-09,0.000000
1,Beta-Cash-AppScore-Model,Beta-Cash-AppScore-Model_Trench2,v1,Trench 2,beta_cash_app_score,numerical,Overall,All,2025-10,2025-09,2025-10,0.119756
2,Beta-Cash-AppScore-Model,Beta-Cash-AppScore-Model_Trench2,v1,Trench 2,calc_app_cnt_health_and_fitness_ever,categorical,Overall,All,2025-09,2025-09,2025-09,0.000000
3,Beta-Cash-AppScore-Model,Beta-Cash-AppScore-Model_Trench2,v1,Trench 2,calc_app_cnt_health_and_fitness_ever,categorical,Overall,All,2025-10,2025-09,2025-10,0.000000
4,Beta-Cash-AppScore-Model,Beta-Cash-AppScore-Model_Trench2,v1,Trench 2,calc_app_cnt_shopping_ever,numerical,Overall,All,2025-09,2025-09,2025-09,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
127,Beta-Cash-AppScore-Model,Beta-Cash-AppScore-Model_Trench2,v1,Trench 2,calc_app_median_time_bw_installed_mins_3d,numerical,product,Cash,2025-10,2025-09,2025-10,0.107423
128,Beta-Cash-AppScore-Model,Beta-Cash-AppScore-Model_Trench2,v1,Trench 2,calc_app_avg_time_bw_installed_mins_ever,numerical,product,Cash,2025-09,2025-09,2025-09,0.000000
129,Beta-Cash-AppScore-Model,Beta-Cash-AppScore-Model_Trench2,v1,Trench 2,calc_app_avg_time_bw_installed_mins_ever,numerical,product,Cash,2025-10,2025-09,2025-10,0.133050
130,Beta-Cash-AppScore-Model,Beta-Cash-AppScore-Model_Trench2,v1,Trench 2,calc_app_median_time_bw_installed_mins_ever,numerical,product,Cash,2025-09,2025-09,2025-09,0.000000


## Beta-Cash-Stack-Model_Trench2

In [59]:
sq = r""" 
WITH parsed as (
  select customerId, digitalLoanAccountId,modelDisplayName,modelVersionId,start_time,end_time,prediction,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
--REPLACE(REPLACE(prediction, "'", '"'), "None", "null") AS prediction_clean
FROM `prj-prod-dataplatform.audit_balance.ml_model_run_details`
where modelDisplayName = 'Beta-Cash-Stack-Model'),

latest_request as (
select * from parsed
QUALIFY ROW_NUMBER() OVER (PARTITION BY customerId, digitalLoanAccountId,modelDisplayName ORDER BY start_time DESC ) = 1),

model_run as (
select customerId,digitalLoanAccountId,modelName, publish_time,
REPLACE(REPLACE(requestPayload, "'", '"'), "None", "null") AS requestPayload_clean
from `prj-prod-dataplatform.audit_balance.ml_request_details` 
WHERE modelName = 'Beta-Cash-Model-response'
QUALIFY ROW_NUMBER() OVER (PARTITION BY customerId, digitalLoanAccountId,modelName ORDER BY publish_time DESC ) = 1)

select * from (
  select 
 r.customerId,
 r.digitalLoanAccountId,
 r.prediction Beta_cash_stack_score,
 r.start_time,
 r.end_time,
 r.modelDisplayName,
 r.modelVersionId,
 loanmaster.new_loan_type,
 loanmaster.gender,
 'Beta-Cash-Stack-Model_Trench2' Model_Name,
'Cash' as product,
 JSON_VALUE(m.requestPayload_clean, "$.predictions.trenchCategory") AS trenchCategory,
 r.calcFeatures,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  r.start_time) AS appln_submit_datetime,
  loanmaster.disbursementDateTime,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  r.start_time)) as Application_month, 
FROM latest_request r
left join model_run m
on r.digitalLoanAccountId = m.digitalLoanAccountId 
left join risk_credit_mis.loan_master_table loanmaster 
  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
qualify row_number() over (partition by r.customerId,r.digitalLoanAccountId order by r.start_time desc) = 1
  )
 where trenchCategory = 'Trench 2'
;
"""

dfd = client.query(sq).to_dataframe()
dfd.head()

,customerId,digitalLoanAccountId,Beta_cash_stack_score,start_time,end_time,modelDisplayName,modelVersionId,new_loan_type,gender,Model_Name,product,trenchCategory,calcFeatures,appln_submit_datetime,disbursementDateTime,Application_month
0,2681557,11b5c282-f98f-47d3-95e3-24efd38e0ca9,0.521825269991605,2025-10-15 07:41:58.324859,2025-10-15 07:41:59.073920,Beta-Cash-Stack-Model,v1,None,None,Beta-Cash-Stack-Model_Trench2,Cash,Trench 2,"{""demo_score"": ""0.47761517340640025"", ""credo_s...",2025-10-15 07:41:58.324859,NaT,2025-10
1,3119716,0fdc61ec-7779-41b6-8b8f-9373a67a10f9,0.5066940104900965,2025-10-15 06:58:51.476016,2025-10-15 06:58:52.470255,Beta-Cash-Stack-Model,v1,None,None,Beta-Cash-Stack-Model_Trench2,Cash,Trench 2,"{""demo_score"": ""0.5989816251749113"", ""credo_sc...",2025-10-15 06:58:51.476016,NaT,2025-10
2,1906556,b6df6f0d-9631-4b78-ba2a-606821ac7740,0.3721561919834904,2025-10-15 06:13:22.258707,2025-10-15 06:13:23.074737,Beta-Cash-Stack-Model,v1,None,None,Beta-Cash-Stack-Model_Trench2,Cash,Trench 2,"{""demo_score"": ""0.6544675991517022"", ""credo_sc...",2025-10-15 06:13:22.258707,NaT,2025-10
3,3494332,a4d903b6-aeb0-4d5a-8ad6-a1cd1090d8bc,0.646854997299086,2025-10-15 03:44:49.571344,2025-10-15 03:44:50.430603,Beta-Cash-Stack-Model,v1,None,None,Beta-Cash-Stack-Model_Trench2,Cash,Trench 2,"{""demo_score"": ""0.6069365390156738"", ""credo_sc...",2025-10-15 03:44:49.571344,NaT,2025-10
4,3535314,b870095e-31a2-4a62-9138-1093d1e210a0,0.6916462208917775,2025-10-15 01:14:53.364331,2025-10-15 01:14:53.989023,Beta-Cash-Stack-Model,v1,None,None,Beta-Cash-Stack-Model_Trench2,Cash,Trench 2,"{""demo_score"": ""0.5982026049071207"", ""credo_sc...",2025-10-15 01:14:53.364331,NaT,2025-10


### Expand_df

In [60]:
# Expand the calcFeatures column
expanded_df = expand_calc_features(dfd)

# Display the result
print(f"Original columns: {dfd.shape[1]}")
print(f"Expanded columns: {expanded_df.shape[1]}")
expanded_df.head()


Original columns: 16
Expanded columns: 20


,customerId,digitalLoanAccountId,Beta_cash_stack_score,start_time,end_time,modelDisplayName,modelVersionId,new_loan_type,gender,Model_Name,product,trenchCategory,calcFeatures,appln_submit_datetime,disbursementDateTime,Application_month,calc_demo_score,calc_credo_score,calc_trx_score,calc_app_score
0,2681557,11b5c282-f98f-47d3-95e3-24efd38e0ca9,0.521825269991605,2025-10-15 07:41:58.324859,2025-10-15 07:41:59.073920,Beta-Cash-Stack-Model,v1,None,None,Beta-Cash-Stack-Model_Trench2,Cash,Trench 2,"{""demo_score"": ""0.47761517340640025"", ""credo_s...",2025-10-15 07:41:58.324859,NaT,2025-10,0.47761517340640025,0.2287,0.5820553416768489,NaN
1,3119716,0fdc61ec-7779-41b6-8b8f-9373a67a10f9,0.5066940104900965,2025-10-15 06:58:51.476016,2025-10-15 06:58:52.470255,Beta-Cash-Stack-Model,v1,None,None,Beta-Cash-Stack-Model_Trench2,Cash,Trench 2,"{""demo_score"": ""0.5989816251749113"", ""credo_sc...",2025-10-15 06:58:51.476016,NaT,2025-10,0.5989816251749113,0.2532,0.6461741103051473,0.4662596069826172
2,1906556,b6df6f0d-9631-4b78-ba2a-606821ac7740,0.3721561919834904,2025-10-15 06:13:22.258707,2025-10-15 06:13:23.074737,Beta-Cash-Stack-Model,v1,None,None,Beta-Cash-Stack-Model_Trench2,Cash,Trench 2,"{""demo_score"": ""0.6544675991517022"", ""credo_sc...",2025-10-15 06:13:22.258707,NaT,2025-10,0.6544675991517022,0.2979,0.48289912393483947,0.4316899736284193
3,3494332,a4d903b6-aeb0-4d5a-8ad6-a1cd1090d8bc,0.646854997299086,2025-10-15 03:44:49.571344,2025-10-15 03:44:50.430603,Beta-Cash-Stack-Model,v1,None,None,Beta-Cash-Stack-Model_Trench2,Cash,Trench 2,"{""demo_score"": ""0.6069365390156738"", ""credo_sc...",2025-10-15 03:44:49.571344,NaT,2025-10,0.6069365390156738,0.1262,0.5484357707355284,0.5242719923112724
4,3535314,b870095e-31a2-4a62-9138-1093d1e210a0,0.6916462208917775,2025-10-15 01:14:53.364331,2025-10-15 01:14:53.989023,Beta-Cash-Stack-Model,v1,None,None,Beta-Cash-Stack-Model_Trench2,Cash,Trench 2,"{""demo_score"": ""0.5982026049071207"", ""credo_sc...",2025-10-15 01:14:53.364331,NaT,2025-10,0.5982026049071207,0.2163,0.6300027313606614,NaN


In [61]:
df = expanded_df.drop(columns=['calcFeatures']).copy()

### run the psi calculation

In [62]:
df = df.copy()

# Define feature list
feature_list = ['Beta_cash_stack_score',  'calc_demo_score', 'calc_credo_score',     'calc_trx_score', 'calc_app_score']

# Define segment columns
segment_columns = ['new_loan_type', 'gender', 'osType', 'trenchCategory', 'product']
# Calculate month-on-month PSI
psi_results = calculate_month_on_month_psi(df, feature_list, segment_columns)
psi_results['modelDisplayName'] = df['modelDisplayName'].iloc[0]
psi_results['Model_Name'] = df['Model_Name'].iloc[0]
psi_results['modelVersionId'] = df['modelVersionId'].iloc[0]
psi_results['trenchCategory'] = df['trenchCategory'].iloc[0]
psi_results = psi_results[[ 'modelDisplayName', 'Model_Name',   'modelVersionId', 'trenchCategory','Feature', 'Feature_Type', 'Segment_Column', 'Segment_Value', 'Month',
       'Base_Month', 'Current_Month', 'PSI']].copy()

# Calculate bin-level PSI
bin_psi_results = calculate_bin_level_psi(df, feature_list, segment_columns)
bin_psi_results['modelDisplayName'] = df['modelDisplayName'].iloc[0]
bin_psi_results['Model_Name'] = df['Model_Name'].iloc[0]
bin_psi_results['modelVersionId'] = df['modelVersionId'].iloc[0]
bin_psi_results['trenchCategory'] = df['trenchCategory'].iloc[0]
bin_psi_results = bin_psi_results[[ 'modelDisplayName', 'Model_Name',  'modelVersionId', 'trenchCategory','Feature', 'Feature_Type', 'Segment_Column', 'Segment_Value', 'Month',
       'Base_Month', 'Current_Month', 'Bin', 'Bin_Range', 'Bin_Min', 'Bin_Max', 'Base_Percentage',
       'Actual_Percentage', 'Bin_PSI']].copy()

# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.temp_csi_new_monitoring_data"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(psi_results, table_id, job_config=job_config)
job.result()  # Wait for the job to complete


# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.temp_csi_new_monitoring_data_feature_bin_level"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(bin_psi_results, table_id, job_config=job_config)
job.result()  # Wait for the job to complete

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=e00a9f23-0c2e-46dc-9fdc-d297c81366f5>

In [63]:
psi_results

,modelDisplayName,Model_Name,modelVersionId,trenchCategory,Feature,Feature_Type,Segment_Column,Segment_Value,Month,Base_Month,Current_Month,PSI
0,Beta-Cash-Stack-Model,Beta-Cash-Stack-Model_Trench2,v1,Trench 2,Beta_cash_stack_score,categorical,Overall,All,2025-09,2025-09,2025-09,0.000000
1,Beta-Cash-Stack-Model,Beta-Cash-Stack-Model_Trench2,v1,Trench 2,Beta_cash_stack_score,categorical,Overall,All,2025-10,2025-09,2025-10,0.153506
2,Beta-Cash-Stack-Model,Beta-Cash-Stack-Model_Trench2,v1,Trench 2,calc_demo_score,categorical,Overall,All,2025-09,2025-09,2025-09,0.000000
3,Beta-Cash-Stack-Model,Beta-Cash-Stack-Model_Trench2,v1,Trench 2,calc_demo_score,categorical,Overall,All,2025-10,2025-09,2025-10,0.154839
4,Beta-Cash-Stack-Model,Beta-Cash-Stack-Model_Trench2,v1,Trench 2,calc_credo_score,categorical,Overall,All,2025-09,2025-09,2025-09,0.000000
5,Beta-Cash-Stack-Model,Beta-Cash-Stack-Model_Trench2,v1,Trench 2,calc_credo_score,categorical,Overall,All,2025-10,2025-09,2025-10,0.153437
6,Beta-Cash-Stack-Model,Beta-Cash-Stack-Model_Trench2,v1,Trench 2,calc_trx_score,categorical,Overall,All,2025-09,2025-09,2025-09,0.000000
7,Beta-Cash-Stack-Model,Beta-Cash-Stack-Model_Trench2,v1,Trench 2,calc_trx_score,categorical,Overall,All,2025-10,2025-09,2025-10,0.141878
8,Beta-Cash-Stack-Model,Beta-Cash-Stack-Model_Trench2,v1,Trench 2,calc_app_score,categorical,Overall,All,2025-09,2025-09,2025-09,0.000000
9,Beta-Cash-Stack-Model,Beta-Cash-Stack-Model_Trench2,v1,Trench 2,calc_app_score,categorical,Overall,All,2025-10,2025-09,2025-10,0.165780


## Alpha-Cash-Stack-Model_Trench3

In [64]:
sq = r""" 
WITH parsed as (
  select customerId, digitalLoanAccountId,modelDisplayName,modelVersionId,start_time,end_time,prediction,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
FROM `prj-prod-dataplatform.audit_balance.ml_model_run_details`
where modelDisplayName = 'Alpha-Cash-Stack-Model'),
latest_request as (
select * from parsed
QUALIFY ROW_NUMBER() OVER (PARTITION BY customerId, digitalLoanAccountId,modelDisplayName ORDER BY start_time DESC ) = 1),
model_run as (
select customerId,digitalLoanAccountId,modelName, publish_time,requestPayload as requestPayload_clean
from `prj-prod-dataplatform.audit_balance.ml_request_details` 
WHERE modelName = 'Alpha-Cash-Model-response'
QUALIFY ROW_NUMBER() OVER (PARTITION BY customerId, digitalLoanAccountId,modelName ORDER BY publish_time DESC ) = 1)
select * from (
  select 
 r.customerId,
 r.digitalLoanAccountId,
 r.prediction aStackScore,
 r.start_time,
 r.end_time,
 r.modelDisplayName,
 r.modelVersionId,
 loanmaster.new_loan_type,
 loanmaster.gender,
 'Alpha-Cash-Stack-Model_Trench3' Model_Name,
 'Cash' as product,
  REGEXP_EXTRACT(m.requestPayload_clean, r"osType[:=]['\"]?([^'\"]+)['\"]?") AS osType,
  REGEXP_EXTRACT(m.requestPayload_clean, r"loanType[:=]['\"]?([^'\"]+)['\"]?") AS loanType,
  REGEXP_EXTRACT(m.requestPayload_clean, r"trenchCategory[:=]['\"]?([^'\"]+)['\"]?") AS trenchCategory,
 r.calcFeatures,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  r.start_time) AS appln_submit_datetime,
  loanmaster.disbursementDateTime,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  r.start_time)) as Application_month,  
FROM latest_request r
left join model_run m
on r.digitalLoanAccountId = m.digitalLoanAccountId 
left join risk_credit_mis.loan_master_table loanmaster 
  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
qualify row_number() over (partition by r.customerId,r.digitalLoanAccountId order by r.start_time desc) = 1
) where trenchCategory = 'Trench 3'
;
"""

dfd = client.query(sq).to_dataframe()
dfd.head()

,customerId,digitalLoanAccountId,aStackScore,start_time,end_time,modelDisplayName,modelVersionId,new_loan_type,gender,Model_Name,product,osType,loanType,trenchCategory,calcFeatures,appln_submit_datetime,disbursementDateTime,Application_month
0,2826643,3f344a2a-fa6f-4916-8d53-b8aa6789925e,0.38197871063905053,2025-10-14 03:43:50.812263,2025-10-14 03:43:51.009542,Alpha-Cash-Stack-Model,v1,Quick,F,Alpha-Cash-Stack-Model_Trench3,Cash,android,Quick,Trench 3,"{""apps_score"": 0.4815879815157128, ""c_demo_sco...",2025-10-14 11:43:37.000000,2025-10-14 14:06:46,2025-10
1,1421326,0deeb01c-ae13-48e2-a3c3-33a55052f6da,0.2268292078067165,2025-10-07 17:25:09.055211,2025-10-07 17:25:09.240317,Alpha-Cash-Stack-Model,v1,Quick,M,Alpha-Cash-Stack-Model_Trench3,Cash,android,Quick,Trench 3,"{""apps_score"": 0.46268666206563563, ""c_demo_sc...",2025-10-08 01:24:56.000000,2025-10-09 10:42:15,2025-10
2,2334704,884b5088-4f01-4d7a-aabb-0faebebea74a,0.36468145988184336,2025-10-10 12:06:50.548385,2025-10-10 12:06:50.718909,Alpha-Cash-Stack-Model,v1,Quick,M,Alpha-Cash-Stack-Model_Trench3,Cash,android,Quick,Trench 3,"{""apps_score"": 0.423288962911544, ""c_demo_scor...",2025-10-10 20:06:33.000000,2025-10-11 14:45:54,2025-10
3,2509061,736fe58b-fa9b-4d87-b021-5812a0ae082a,0.41691997345062115,2025-10-14 23:55:48.016636,2025-10-14 23:55:48.185067,Alpha-Cash-Stack-Model,v1,None,None,Alpha-Cash-Stack-Model_Trench3,Cash,android,Quick,Trench 3,"{""apps_score"": 0.4838149268232431, ""c_demo_sco...",2025-10-14 23:55:48.016636,NaT,2025-10
4,2999824,b7d6f903-240a-4f26-aa1d-54c14811022a,0.3551485542446573,2025-10-07 22:18:03.879036,2025-10-07 22:18:04.027246,Alpha-Cash-Stack-Model,v1,Quick,M,Alpha-Cash-Stack-Model_Trench3,Cash,ios,Quick,Trench 3,"{""apps_score"": null, ""c_demo_score"": 0.3349292...",2025-10-08 06:17:43.000000,2025-10-08 07:42:44,2025-10


### Expand_df

In [65]:
# Expand the calcFeatures column
expanded_df = expand_calc_features(dfd)

# Display the result
print(f"Original columns: {dfd.shape[1]}")
print(f"Expanded columns: {expanded_df.shape[1]}")
expanded_df.head()


Original columns: 18
Expanded columns: 23


,customerId,digitalLoanAccountId,aStackScore,start_time,end_time,modelDisplayName,modelVersionId,new_loan_type,gender,Model_Name,product,osType,loanType,trenchCategory,calcFeatures,appln_submit_datetime,disbursementDateTime,Application_month,calc_apps_score,calc_c_demo_score,calc_c_credo_score,calc_c_tx_score,calc_ca_cic_score
0,2826643,3f344a2a-fa6f-4916-8d53-b8aa6789925e,0.38197871063905053,2025-10-14 03:43:50.812263,2025-10-14 03:43:51.009542,Alpha-Cash-Stack-Model,v1,Quick,F,Alpha-Cash-Stack-Model_Trench3,Cash,android,Quick,Trench 3,"{""apps_score"": 0.4815879815157128, ""c_demo_sco...",2025-10-14 11:43:37.000000,2025-10-14 14:06:46,2025-10,0.481588,0.428332,0.1163,0.442847,0.360488
1,1421326,0deeb01c-ae13-48e2-a3c3-33a55052f6da,0.2268292078067165,2025-10-07 17:25:09.055211,2025-10-07 17:25:09.240317,Alpha-Cash-Stack-Model,v1,Quick,M,Alpha-Cash-Stack-Model_Trench3,Cash,android,Quick,Trench 3,"{""apps_score"": 0.46268666206563563, ""c_demo_sc...",2025-10-08 01:24:56.000000,2025-10-09 10:42:15,2025-10,0.462687,0.343088,0.1146,0.333870,0.387196
2,2334704,884b5088-4f01-4d7a-aabb-0faebebea74a,0.36468145988184336,2025-10-10 12:06:50.548385,2025-10-10 12:06:50.718909,Alpha-Cash-Stack-Model,v1,Quick,M,Alpha-Cash-Stack-Model_Trench3,Cash,android,Quick,Trench 3,"{""apps_score"": 0.423288962911544, ""c_demo_scor...",2025-10-10 20:06:33.000000,2025-10-11 14:45:54,2025-10,0.423289,0.448658,0.1132,0.567654,0.404803
3,2509061,736fe58b-fa9b-4d87-b021-5812a0ae082a,0.41691997345062115,2025-10-14 23:55:48.016636,2025-10-14 23:55:48.185067,Alpha-Cash-Stack-Model,v1,None,None,Alpha-Cash-Stack-Model_Trench3,Cash,android,Quick,Trench 3,"{""apps_score"": 0.4838149268232431, ""c_demo_sco...",2025-10-14 23:55:48.016636,NaT,2025-10,0.483815,0.449953,0.1196,0.404093,0.468901
4,2999824,b7d6f903-240a-4f26-aa1d-54c14811022a,0.3551485542446573,2025-10-07 22:18:03.879036,2025-10-07 22:18:04.027246,Alpha-Cash-Stack-Model,v1,Quick,M,Alpha-Cash-Stack-Model_Trench3,Cash,ios,Quick,Trench 3,"{""apps_score"": null, ""c_demo_score"": 0.3349292...",2025-10-08 06:17:43.000000,2025-10-08 07:42:44,2025-10,NaN,0.334929,0.1030,0.500253,0.384770


In [66]:
df = expanded_df.drop(columns=['calcFeatures']).copy()

### run the psi calculation

In [67]:
df = df.copy()

# Define feature list
feature_list = ['aStackScore', 'calc_apps_score', 'calc_c_demo_score', 'calc_c_credo_score', 'calc_c_tx_score', 'calc_ca_cic_score']

# Define segment columns
segment_columns = ['new_loan_type', 'gender', 'osType', 'trenchCategory', 'product']
# Calculate month-on-month PSI
psi_results = calculate_month_on_month_psi(df, feature_list, segment_columns)
psi_results['modelDisplayName'] = df['modelDisplayName'].iloc[0]
psi_results['Model_Name'] = df['Model_Name'].iloc[0]
psi_results['modelVersionId'] = df['modelVersionId'].iloc[0]
psi_results['trenchCategory'] = df['trenchCategory'].iloc[0]
psi_results = psi_results[[ 'modelDisplayName', 'Model_Name',   'modelVersionId', 'trenchCategory','Feature', 'Feature_Type', 'Segment_Column', 'Segment_Value', 'Month',
       'Base_Month', 'Current_Month', 'PSI']].copy()

# Calculate bin-level PSI
bin_psi_results = calculate_bin_level_psi(df, feature_list, segment_columns)
bin_psi_results['modelDisplayName'] = df['modelDisplayName'].iloc[0]
bin_psi_results['Model_Name'] = df['Model_Name'].iloc[0]
bin_psi_results['modelVersionId'] = df['modelVersionId'].iloc[0]
bin_psi_results['trenchCategory'] = df['trenchCategory'].iloc[0]
bin_psi_results = bin_psi_results[[ 'modelDisplayName', 'Model_Name',  'modelVersionId', 'trenchCategory','Feature', 'Feature_Type', 'Segment_Column', 'Segment_Value', 'Month',
       'Base_Month', 'Current_Month', 'Bin', 'Bin_Range', 'Bin_Min', 'Bin_Max', 'Base_Percentage',
       'Actual_Percentage', 'Bin_PSI']].copy()

# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.temp_csi_new_monitoring_data"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(psi_results, table_id, job_config=job_config)
job.result()  # Wait for the job to complete


# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.temp_csi_new_monitoring_data_feature_bin_level"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(bin_psi_results, table_id, job_config=job_config)
job.result()  # Wait for the job to complete

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=b82111e7-703a-4e00-9589-db6152029526>

In [68]:
psi_results

,modelDisplayName,Model_Name,modelVersionId,trenchCategory,Feature,Feature_Type,Segment_Column,Segment_Value,Month,Base_Month,Current_Month,PSI
0,Alpha-Cash-Stack-Model,Alpha-Cash-Stack-Model_Trench3,v1,Trench 3,aStackScore,categorical,Overall,All,2025-09,2025-09,2025-09,0.000000
1,Alpha-Cash-Stack-Model,Alpha-Cash-Stack-Model_Trench3,v1,Trench 3,aStackScore,categorical,Overall,All,2025-10,2025-09,2025-10,2.404973
2,Alpha-Cash-Stack-Model,Alpha-Cash-Stack-Model_Trench3,v1,Trench 3,calc_apps_score,numerical,Overall,All,2025-09,2025-09,2025-09,0.000000
3,Alpha-Cash-Stack-Model,Alpha-Cash-Stack-Model_Trench3,v1,Trench 3,calc_apps_score,numerical,Overall,All,2025-10,2025-09,2025-10,2.480244
4,Alpha-Cash-Stack-Model,Alpha-Cash-Stack-Model_Trench3,v1,Trench 3,calc_c_demo_score,numerical,Overall,All,2025-09,2025-09,2025-09,0.000000
5,Alpha-Cash-Stack-Model,Alpha-Cash-Stack-Model_Trench3,v1,Trench 3,calc_c_demo_score,numerical,Overall,All,2025-10,2025-09,2025-10,1.135442
6,Alpha-Cash-Stack-Model,Alpha-Cash-Stack-Model_Trench3,v1,Trench 3,calc_c_credo_score,numerical,Overall,All,2025-09,2025-09,2025-09,0.000000
7,Alpha-Cash-Stack-Model,Alpha-Cash-Stack-Model_Trench3,v1,Trench 3,calc_c_credo_score,numerical,Overall,All,2025-10,2025-09,2025-10,0.679496
8,Alpha-Cash-Stack-Model,Alpha-Cash-Stack-Model_Trench3,v1,Trench 3,calc_c_tx_score,numerical,Overall,All,2025-09,2025-09,2025-09,0.000000
9,Alpha-Cash-Stack-Model,Alpha-Cash-Stack-Model_Trench3,v1,Trench 3,calc_c_tx_score,numerical,Overall,All,2025-10,2025-09,2025-10,1.727761


## Beta-Cash-AppScore-Model_Trench3

In [69]:
sq = r""" WITH parsed as (
  select customerId, digitalLoanAccountId,modelDisplayName,modelVersionId,start_time,end_time,prediction,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
REPLACE(REPLACE(prediction, "'", '"'), "None", "null") AS prediction_clean
FROM `prj-prod-dataplatform.audit_balance.ml_model_run_details`
where modelDisplayName = 'Beta-Cash-AppScore-Model'),

latest_request as (
select * from parsed
QUALIFY ROW_NUMBER() OVER (PARTITION BY customerId, digitalLoanAccountId,modelDisplayName ORDER BY start_time DESC ) = 1),

model_run as (
select customerId,digitalLoanAccountId,modelName, publish_time,
REPLACE(REPLACE(requestPayload, "'", '"'), "None", "null") AS requestPayload_clean
from `prj-prod-dataplatform.audit_balance.ml_request_details` 
WHERE modelName = 'Beta-Cash-Model-response'
QUALIFY ROW_NUMBER() OVER (PARTITION BY customerId, digitalLoanAccountId,modelName ORDER BY publish_time DESC ) = 1)

select * from (
  select 
 r.customerId,
 r.digitalLoanAccountId,
 --r.prediction,
 r.start_time,
 r.end_time,
 r.modelDisplayName,
 r.modelVersionId,
 loanmaster.new_loan_type,
 loanmaster.gender,
 'Beta-Cash-AppScore-Model_Trench3' Model_Name,
  JSON_VALUE(m.requestPayload_clean, "$.predictions.trenchCategory") AS trenchCategory,
  JSON_VALUE(m.requestPayload_clean, "$.predictions.loanType") AS loanType,
  --  Beta App Score Model Features for Trench 3
  SAFE_CAST(JSON_VALUE(r.prediction_clean, "$.combined_score") AS Float64) AS beta_cash_app_score,
 r.calcFeatures,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  r.start_time) AS appln_submit_datetime,
  loanmaster.disbursementDateTime,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  r.start_time)) as Application_month,  
FROM latest_request r
left join model_run m
on r.digitalLoanAccountId = m.digitalLoanAccountId 
left join risk_credit_mis.loan_master_table loanmaster 
  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
qualify row_number() over (partition by r.customerId,r.digitalLoanAccountId order by r.start_time desc) = 1
)
where trenchCategory = 'Trench 3'

;"""
dfd = client.query(sq).to_dataframe()
dfd.head()

,customerId,digitalLoanAccountId,start_time,end_time,modelDisplayName,modelVersionId,new_loan_type,gender,Model_Name,trenchCategory,loanType,beta_cash_app_score,calcFeatures,appln_submit_datetime,disbursementDateTime,Application_month
0,2056320,4fcab9a9-5e07-4712-90fa-c98f6031fd1b,2025-10-11 14:21:00.150816,2025-10-11 14:21:01.177843,Beta-Cash-AppScore-Model,v1,Quick,F,Beta-Cash-AppScore-Model_Trench3,Trench 3,Quick,0.539246,"{""app_cnt_health_and_fitness_ever"": 0, ""app_cn...",2025-10-11 22:20:56,NaT,2025-10
1,2885269,e81938c2-4b17-4ef2-b420-b7b3fbc880a1,2025-10-03 08:58:28.140611,2025-10-03 08:58:29.412995,Beta-Cash-AppScore-Model,v1,Quick,F,Beta-Cash-AppScore-Model_Trench3,Trench 3,Quick,0.533268,"{""app_cnt_health_and_fitness_ever"": 1, ""app_cn...",2025-10-03 16:58:25,NaT,2025-10
2,3066495,75da5a84-b7b6-4c21-9775-4d01a16a903c,2025-09-30 10:59:04.338303,2025-09-30 10:59:05.218193,Beta-Cash-AppScore-Model,v1,Quick,F,Beta-Cash-AppScore-Model_Trench3,Trench 3,Quick,0.535543,"{""app_cnt_health_and_fitness_ever"": 0, ""app_cn...",2025-09-30 18:59:01,2025-10-01 07:20:25,2025-09
3,2063303,5bcd9dc8-ba35-4cbe-9c9f-7f3c91c2e117,2025-10-12 05:25:31.791766,2025-10-12 05:25:32.767139,Beta-Cash-AppScore-Model,v1,Quick,M,Beta-Cash-AppScore-Model_Trench3,Trench 3,Quick,0.542907,"{""app_cnt_health_and_fitness_ever"": 0, ""app_cn...",2025-10-12 13:25:29,NaT,2025-10
4,2702527,c9b20d91-31d2-48fb-885e-a3e8c89a251a,2025-10-12 11:36:38.754524,2025-10-12 11:36:39.590301,Beta-Cash-AppScore-Model,v1,Quick,F,Beta-Cash-AppScore-Model_Trench3,Trench 3,Quick,0.470726,"{""app_cnt_health_and_fitness_ever"": 0, ""app_cn...",2025-10-12 19:36:36,NaT,2025-10


### Expand_df

In [70]:
# Expand the calcFeatures column
expanded_df = expand_calc_features(dfd)

# Display the result
print(f"Original columns: {dfd.shape[1]}")
print(f"Expanded columns: {expanded_df.shape[1]}")
expanded_df.head()


Original columns: 16
Expanded columns: 31


,customerId,digitalLoanAccountId,start_time,end_time,modelDisplayName,modelVersionId,new_loan_type,gender,Model_Name,trenchCategory,loanType,beta_cash_app_score,calcFeatures,appln_submit_datetime,disbursementDateTime,Application_month,calc_app_cnt_health_and_fitness_ever,calc_app_cnt_productivity_ever,calc_app_cnt_rated_for_18plus_ever,calc_app_cnt_books_and_reference_ever,calc_app_cnt_gaming_180d,calc_app_cnt_absence_tag_365d,calc_app_last_payday_install_to_apply_days,calc_app_cnt_absence_tag_365d_binned,calc_app_cnt_gaming_180d_binned,calc_app_cnt_productivity_ever_binned,calc_app_cnt_rated_for_18plus_ever_binned,calc_app_cnt_health_and_fitness_ever_binned,calc_app_cnt_books_and_reference_ever_binned,calc_app_last_payday_install_to_apply_days_binned,calc_ln_user_type
0,2056320,4fcab9a9-5e07-4712-90fa-c98f6031fd1b,2025-10-11 14:21:00.150816,2025-10-11 14:21:01.177843,Beta-Cash-AppScore-Model,v1,Quick,F,Beta-Cash-AppScore-Model_Trench3,Trench 3,Quick,0.539246,"{""app_cnt_health_and_fitness_ever"": 0, ""app_cn...",2025-10-11 22:20:56,NaT,2025-10,0,2,0,1,1,1,NaN,<= 4.182,<= 5.717,<= 2.333,<= 1.010,<= 2.121,<= 1.273,nan,1_Repeat Applicant
1,2885269,e81938c2-4b17-4ef2-b420-b7b3fbc880a1,2025-10-03 08:58:28.140611,2025-10-03 08:58:29.412995,Beta-Cash-AppScore-Model,v1,Quick,F,Beta-Cash-AppScore-Model_Trench3,Trench 3,Quick,0.533268,"{""app_cnt_health_and_fitness_ever"": 1, ""app_cn...",2025-10-03 16:58:25,NaT,2025-10,1,13,2,1,0,19,11.9,> 6.970,<= 5.717,> 2.333,> 1.010,<= 2.121,<= 1.273,<= 18.344,1_Repeat Applicant
2,3066495,75da5a84-b7b6-4c21-9775-4d01a16a903c,2025-09-30 10:59:04.338303,2025-09-30 10:59:05.218193,Beta-Cash-AppScore-Model,v1,Quick,F,Beta-Cash-AppScore-Model_Trench3,Trench 3,Quick,0.535543,"{""app_cnt_health_and_fitness_ever"": 0, ""app_cn...",2025-09-30 18:59:01,2025-10-01 07:20:25,2025-09,0,2,0,3,3,3,24.6,<= 4.182,<= 5.717,<= 2.333,<= 1.010,<= 2.121,> 1.273,> 18.344,1_Repeat Applicant
3,2063303,5bcd9dc8-ba35-4cbe-9c9f-7f3c91c2e117,2025-10-12 05:25:31.791766,2025-10-12 05:25:32.767139,Beta-Cash-AppScore-Model,v1,Quick,M,Beta-Cash-AppScore-Model_Trench3,Trench 3,Quick,0.542907,"{""app_cnt_health_and_fitness_ever"": 0, ""app_cn...",2025-10-12 13:25:29,NaT,2025-10,0,3,1,0,0,1,NaN,<= 4.182,<= 5.717,> 2.333,<= 1.010,<= 2.121,<= 1.273,nan,1_Repeat Applicant
4,2702527,c9b20d91-31d2-48fb-885e-a3e8c89a251a,2025-10-12 11:36:38.754524,2025-10-12 11:36:39.590301,Beta-Cash-AppScore-Model,v1,Quick,F,Beta-Cash-AppScore-Model_Trench3,Trench 3,Quick,0.470726,"{""app_cnt_health_and_fitness_ever"": 0, ""app_cn...",2025-10-12 19:36:36,NaT,2025-10,0,4,0,0,0,5,1.7,4.182 – 6.970,<= 5.717,> 2.333,<= 1.010,<= 2.121,<= 1.273,<= 18.344,1_Repeat Applicant


In [71]:
df = expanded_df.drop(columns=['calcFeatures']).copy()

### run the psi calculation

In [72]:
df = df.copy()

# Define feature list
feature_list = ['beta_cash_app_score', 'calc_app_cnt_health_and_fitness_ever',
       'calc_app_cnt_productivity_ever', 'calc_app_cnt_rated_for_18plus_ever',
       'calc_app_cnt_books_and_reference_ever', 'calc_app_cnt_gaming_180d',
       'calc_app_cnt_absence_tag_365d',
       'calc_app_last_payday_install_to_apply_days',
       'calc_app_cnt_absence_tag_365d_binned',
       'calc_app_cnt_gaming_180d_binned',
       'calc_app_cnt_productivity_ever_binned',
       'calc_app_cnt_rated_for_18plus_ever_binned',
       'calc_app_cnt_health_and_fitness_ever_binned',
       'calc_app_cnt_books_and_reference_ever_binned',
       'calc_app_last_payday_install_to_apply_days_binned',
       'calc_ln_user_type']

# Define segment columns
segment_columns = ['new_loan_type', 'gender', 'osType', 'trenchCategory']
# Calculate month-on-month PSI
psi_results = calculate_month_on_month_psi(df, feature_list, segment_columns)
psi_results['modelDisplayName'] = df['modelDisplayName'].iloc[0]
psi_results['Model_Name'] = df['Model_Name'].iloc[0]
psi_results['modelVersionId'] = df['modelVersionId'].iloc[0]
psi_results['trenchCategory'] = df['trenchCategory'].iloc[0]
psi_results = psi_results[[ 'modelDisplayName', 'Model_Name',   'modelVersionId', 'trenchCategory','Feature', 'Feature_Type', 'Segment_Column', 'Segment_Value', 'Month',
       'Base_Month', 'Current_Month', 'PSI']].copy()

# Calculate bin-level PSI
bin_psi_results = calculate_bin_level_psi(df, feature_list, segment_columns)
bin_psi_results['modelDisplayName'] = df['modelDisplayName'].iloc[0]
bin_psi_results['Model_Name'] = df['Model_Name'].iloc[0]
bin_psi_results['modelVersionId'] = df['modelVersionId'].iloc[0]
bin_psi_results['trenchCategory'] = df['trenchCategory'].iloc[0]
bin_psi_results = bin_psi_results[[ 'modelDisplayName', 'Model_Name',  'modelVersionId', 'trenchCategory','Feature', 'Feature_Type', 'Segment_Column', 'Segment_Value', 'Month',
       'Base_Month', 'Current_Month', 'Bin', 'Bin_Range', 'Bin_Min', 'Bin_Max', 'Base_Percentage',
       'Actual_Percentage', 'Bin_PSI']].copy()

# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.temp_csi_new_monitoring_data"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(psi_results, table_id, job_config=job_config)
job.result()  # Wait for the job to complete


# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.temp_csi_new_monitoring_data_feature_bin_level"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(bin_psi_results, table_id, job_config=job_config)
job.result()  # Wait for the job to complete

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=dee08906-4bf2-4207-9cc2-b5868bbdc8a5>

In [73]:
psi_results

,modelDisplayName,Model_Name,modelVersionId,trenchCategory,Feature,Feature_Type,Segment_Column,Segment_Value,Month,Base_Month,Current_Month,PSI
0,Beta-Cash-AppScore-Model,Beta-Cash-AppScore-Model_Trench3,v1,Trench 3,beta_cash_app_score,numerical,Overall,All,2025-09,2025-09,2025-09,0.000000
1,Beta-Cash-AppScore-Model,Beta-Cash-AppScore-Model_Trench3,v1,Trench 3,beta_cash_app_score,numerical,Overall,All,2025-10,2025-09,2025-10,0.674717
2,Beta-Cash-AppScore-Model,Beta-Cash-AppScore-Model_Trench3,v1,Trench 3,calc_app_cnt_health_and_fitness_ever,categorical,Overall,All,2025-09,2025-09,2025-09,0.000000
3,Beta-Cash-AppScore-Model,Beta-Cash-AppScore-Model_Trench3,v1,Trench 3,calc_app_cnt_health_and_fitness_ever,categorical,Overall,All,2025-10,2025-09,2025-10,0.000000
4,Beta-Cash-AppScore-Model,Beta-Cash-AppScore-Model_Trench3,v1,Trench 3,calc_app_cnt_productivity_ever,numerical,Overall,All,2025-09,2025-09,2025-09,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
155,Beta-Cash-AppScore-Model,Beta-Cash-AppScore-Model_Trench3,v1,Trench 3,calc_app_cnt_books_and_reference_ever_binned,categorical,trenchCategory,Trench 3,2025-10,2025-09,2025-10,0.000000
156,Beta-Cash-AppScore-Model,Beta-Cash-AppScore-Model_Trench3,v1,Trench 3,calc_app_last_payday_install_to_apply_days_binned,categorical,trenchCategory,Trench 3,2025-09,2025-09,2025-09,0.000000
157,Beta-Cash-AppScore-Model,Beta-Cash-AppScore-Model_Trench3,v1,Trench 3,calc_app_last_payday_install_to_apply_days_binned,categorical,trenchCategory,Trench 3,2025-10,2025-09,2025-10,0.197571
158,Beta-Cash-AppScore-Model,Beta-Cash-AppScore-Model_Trench3,v1,Trench 3,calc_ln_user_type,categorical,trenchCategory,Trench 3,2025-09,2025-09,2025-09,0.000000


## Beta-Cash-Stack-Model_Trench3

In [74]:
sq = r"""WITH parsed as (
  select customerId, digitalLoanAccountId,modelDisplayName,modelVersionId,start_time,end_time,prediction,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
--REPLACE(REPLACE(prediction, "'", '"'), "None", "null") AS prediction_clean
FROM `prj-prod-dataplatform.audit_balance.ml_model_run_details`
where modelDisplayName = 'Beta-Cash-Stack-Model'),

latest_request as (
select * from parsed
QUALIFY ROW_NUMBER() OVER (PARTITION BY customerId, digitalLoanAccountId,modelDisplayName ORDER BY start_time DESC ) = 1),

model_run as (
select customerId,digitalLoanAccountId,modelName, publish_time,
REPLACE(REPLACE(requestPayload, "'", '"'), "None", "null") AS requestPayload_clean
from `prj-prod-dataplatform.audit_balance.ml_request_details` 
WHERE modelName = 'Beta-Cash-Model-response'
QUALIFY ROW_NUMBER() OVER (PARTITION BY customerId, digitalLoanAccountId,modelName ORDER BY publish_time DESC ) = 1)

select * from (
  select 
 r.customerId,
 r.digitalLoanAccountId,
 r.prediction Beta_cash_stack_score,
 r.start_time,
 r.end_time,
 r.modelDisplayName,
 r.modelVersionId,
 loanmaster.new_loan_type,
 loanmaster.gender,
 'Beta-Cash-Stack-Model_Trench3' Model_Name,
 'Cash' as product,
 JSON_VALUE(m.requestPayload_clean, "$.predictions.trenchCategory") AS trenchCategory,
  r.calcFeatures,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  r.start_time) AS appln_submit_datetime,
  loanmaster.disbursementDateTime,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  r.start_time)) as Application_month,  
FROM latest_request r
left join model_run m
on r.digitalLoanAccountId = m.digitalLoanAccountId 
left join risk_credit_mis.loan_master_table loanmaster 
  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
qualify row_number() over (partition by r.customerId,r.digitalLoanAccountId order by r.start_time desc) = 1
  )
 where trenchCategory = 'Trench 3'

;"""
dfd = client.query(sq).to_dataframe()
dfd.head()

,customerId,digitalLoanAccountId,Beta_cash_stack_score,start_time,end_time,modelDisplayName,modelVersionId,new_loan_type,gender,Model_Name,product,trenchCategory,calcFeatures,appln_submit_datetime,disbursementDateTime,Application_month
0,1386299,b0caf33d-83f3-4427-a8d8-e5eca82f6355,0.2886257354628351,2025-10-09 15:05:03.215975,2025-10-09 15:05:04.142213,Beta-Cash-Stack-Model,v1,Quick,F,Beta-Cash-Stack-Model_Trench3,Cash,Trench 3,"{""demo_score"": ""0.40485963581973106"", ""credo_s...",2025-10-09 23:04:59,2025-10-10 07:12:46,2025-10
1,1977941,6e2fac9e-9875-4c25-9198-ce6f53f7b849,0.3215324381303579,2025-10-08 13:11:16.971346,2025-10-08 13:11:17.637938,Beta-Cash-Stack-Model,v1,Quick,F,Beta-Cash-Stack-Model_Trench3,Cash,Trench 3,"{""demo_score"": ""0.2825585784624324"", ""credo_sc...",2025-10-08 21:11:12,2025-10-08 21:14:40,2025-10
2,2271543,eb7cd334-ee02-4f08-8855-e7e1defb9c55,0.421839817195601,2025-10-13 18:58:22.787044,2025-10-13 18:58:23.567458,Beta-Cash-Stack-Model,v1,Quick,M,Beta-Cash-Stack-Model_Trench3,Cash,Trench 3,"{""demo_score"": ""0.29217097978349144"", ""credo_s...",2025-10-14 02:58:19,2025-10-14 08:34:26,2025-10
3,2496079,c1bbd16f-65f3-4a12-b3c6-8e10882cc4c1,0.5558672596173421,2025-10-11 10:14:21.240806,2025-10-11 10:14:22.042895,Beta-Cash-Stack-Model,v1,Quick,F,Beta-Cash-Stack-Model_Trench3,Cash,Trench 3,"{""demo_score"": ""0.5493286638433467"", ""credo_sc...",2025-10-11 18:14:17,NaT,2025-10
4,2960429,cc550106-2cd3-4840-8db1-58b0bbd1b710,0.7054030215936711,2025-10-12 12:40:01.375319,2025-10-12 12:40:02.354585,Beta-Cash-Stack-Model,v1,Quick,M,Beta-Cash-Stack-Model_Trench3,Cash,Trench 3,"{""demo_score"": ""0.39234714413138033"", ""credo_s...",2025-10-12 20:39:58,NaT,2025-10


### Expand_df

In [75]:
# Expand the calcFeatures column
expanded_df = expand_calc_features(dfd)

# Display the result
print(f"Original columns: {dfd.shape[1]}")
print(f"Expanded columns: {expanded_df.shape[1]}")
expanded_df.head()


Original columns: 16
Expanded columns: 20


,customerId,digitalLoanAccountId,Beta_cash_stack_score,start_time,end_time,modelDisplayName,modelVersionId,new_loan_type,gender,Model_Name,product,trenchCategory,calcFeatures,appln_submit_datetime,disbursementDateTime,Application_month,calc_demo_score,calc_credo_score,calc_trx_score,calc_app_score
0,1386299,b0caf33d-83f3-4427-a8d8-e5eca82f6355,0.2886257354628351,2025-10-09 15:05:03.215975,2025-10-09 15:05:04.142213,Beta-Cash-Stack-Model,v1,Quick,F,Beta-Cash-Stack-Model_Trench3,Cash,Trench 3,"{""demo_score"": ""0.40485963581973106"", ""credo_s...",2025-10-09 23:04:59,2025-10-10 07:12:46,2025-10,0.40485963581973106,0.0698,0.4023404325122784,0.467487047687154
1,1977941,6e2fac9e-9875-4c25-9198-ce6f53f7b849,0.3215324381303579,2025-10-08 13:11:16.971346,2025-10-08 13:11:17.637938,Beta-Cash-Stack-Model,v1,Quick,F,Beta-Cash-Stack-Model_Trench3,Cash,Trench 3,"{""demo_score"": ""0.2825585784624324"", ""credo_sc...",2025-10-08 21:11:12,2025-10-08 21:14:40,2025-10,0.2825585784624324,0.1353,0.612092355732312,NaN
2,2271543,eb7cd334-ee02-4f08-8855-e7e1defb9c55,0.421839817195601,2025-10-13 18:58:22.787044,2025-10-13 18:58:23.567458,Beta-Cash-Stack-Model,v1,Quick,M,Beta-Cash-Stack-Model_Trench3,Cash,Trench 3,"{""demo_score"": ""0.29217097978349144"", ""credo_s...",2025-10-14 02:58:19,2025-10-14 08:34:26,2025-10,0.29217097978349144,0.0943,0.6753408266940086,NaN
3,2496079,c1bbd16f-65f3-4a12-b3c6-8e10882cc4c1,0.5558672596173421,2025-10-11 10:14:21.240806,2025-10-11 10:14:22.042895,Beta-Cash-Stack-Model,v1,Quick,F,Beta-Cash-Stack-Model_Trench3,Cash,Trench 3,"{""demo_score"": ""0.5493286638433467"", ""credo_sc...",2025-10-11 18:14:17,NaT,2025-10,0.5493286638433467,0.1073,0.5765407696232918,NaN
4,2960429,cc550106-2cd3-4840-8db1-58b0bbd1b710,0.7054030215936711,2025-10-12 12:40:01.375319,2025-10-12 12:40:02.354585,Beta-Cash-Stack-Model,v1,Quick,M,Beta-Cash-Stack-Model_Trench3,Cash,Trench 3,"{""demo_score"": ""0.39234714413138033"", ""credo_s...",2025-10-12 20:39:58,NaT,2025-10,0.39234714413138033,0.0766,0.6627409147072032,0.5638987929552683


In [76]:
df = expanded_df.drop(columns=['calcFeatures']).copy()

### run the psi calculation

In [77]:
df = df.copy()

# Define feature list
feature_list = ['Beta_cash_stack_score', 'calc_demo_score', 'calc_credo_score',  'calc_trx_score', 'calc_app_score']

# Define segment columns
segment_columns = ['new_loan_type', 'gender', 'osType', 'trenchCategory', 'product']
# Calculate month-on-month PSI
psi_results = calculate_month_on_month_psi(df, feature_list, segment_columns)
psi_results['modelDisplayName'] = df['modelDisplayName'].iloc[0]
psi_results['Model_Name'] = df['Model_Name'].iloc[0]
psi_results['modelVersionId'] = df['modelVersionId'].iloc[0]
psi_results['trenchCategory'] = df['trenchCategory'].iloc[0]
psi_results = psi_results[[ 'modelDisplayName', 'Model_Name',   'modelVersionId', 'trenchCategory','Feature', 'Feature_Type', 'Segment_Column', 'Segment_Value', 'Month',
       'Base_Month', 'Current_Month', 'PSI']].copy()

# Calculate bin-level PSI
bin_psi_results = calculate_bin_level_psi(df, feature_list, segment_columns)
bin_psi_results['modelDisplayName'] = df['modelDisplayName'].iloc[0]
bin_psi_results['Model_Name'] = df['Model_Name'].iloc[0]
bin_psi_results['modelVersionId'] = df['modelVersionId'].iloc[0]
bin_psi_results['trenchCategory'] = df['trenchCategory'].iloc[0]
bin_psi_results = bin_psi_results[[ 'modelDisplayName', 'Model_Name',  'modelVersionId', 'trenchCategory','Feature', 'Feature_Type', 'Segment_Column', 'Segment_Value', 'Month',
       'Base_Month', 'Current_Month', 'Bin', 'Bin_Range', 'Bin_Min', 'Bin_Max', 'Base_Percentage',
       'Actual_Percentage', 'Bin_PSI']].copy()

# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.temp_csi_new_monitoring_data"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(psi_results, table_id, job_config=job_config)
job.result()  # Wait for the job to complete


# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.temp_csi_new_monitoring_data_feature_bin_level"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(bin_psi_results, table_id, job_config=job_config)
job.result()  # Wait for the job to complete

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=67179046-10ac-47ee-9ef3-5592e18ddff5>

In [78]:
psi_results

,modelDisplayName,Model_Name,modelVersionId,trenchCategory,Feature,Feature_Type,Segment_Column,Segment_Value,Month,Base_Month,Current_Month,PSI
0,Beta-Cash-Stack-Model,Beta-Cash-Stack-Model_Trench3,v1,Trench 3,Beta_cash_stack_score,categorical,Overall,All,2025-09,2025-09,2025-09,0.000000
1,Beta-Cash-Stack-Model,Beta-Cash-Stack-Model_Trench3,v1,Trench 3,Beta_cash_stack_score,categorical,Overall,All,2025-10,2025-09,2025-10,0.512368
2,Beta-Cash-Stack-Model,Beta-Cash-Stack-Model_Trench3,v1,Trench 3,calc_demo_score,categorical,Overall,All,2025-09,2025-09,2025-09,0.000000
3,Beta-Cash-Stack-Model,Beta-Cash-Stack-Model_Trench3,v1,Trench 3,calc_demo_score,categorical,Overall,All,2025-10,2025-09,2025-10,0.512368
4,Beta-Cash-Stack-Model,Beta-Cash-Stack-Model_Trench3,v1,Trench 3,calc_credo_score,categorical,Overall,All,2025-09,2025-09,2025-09,0.000000
5,Beta-Cash-Stack-Model,Beta-Cash-Stack-Model_Trench3,v1,Trench 3,calc_credo_score,categorical,Overall,All,2025-10,2025-09,2025-10,0.743335
6,Beta-Cash-Stack-Model,Beta-Cash-Stack-Model_Trench3,v1,Trench 3,calc_trx_score,categorical,Overall,All,2025-09,2025-09,2025-09,0.000000
7,Beta-Cash-Stack-Model,Beta-Cash-Stack-Model_Trench3,v1,Trench 3,calc_trx_score,categorical,Overall,All,2025-10,2025-09,2025-10,0.512368
8,Beta-Cash-Stack-Model,Beta-Cash-Stack-Model_Trench3,v1,Trench 3,calc_app_score,categorical,Overall,All,2025-09,2025-09,2025-09,0.000000
9,Beta-Cash-Stack-Model,Beta-Cash-Stack-Model_Trench3,v1,Trench 3,calc_app_score,categorical,Overall,All,2025-10,2025-09,2025-10,0.518309


# SIL

## Alpha - CIC-SIL-Model

In [79]:
sq = """ 
WITH cleaned AS (
  SELECT
    customerId,digitalLoanAccountId,prediction,start_time,end_time,modelDisplayName,modelVersionId,

    REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeature
  FROM `prj-prod-dataplatform.audit_balance.ml_model_run_details`
  WHERE modelDisplayName = 'Alpha - CIC-SIL-Model')
SELECT
  r.customerId,r.digitalLoanAccountId,prediction Alpha_cic_sil_score
    ,start_time,end_time,modelDisplayName,modelVersionId,
   loanmaster.new_loan_type,
 loanmaster.gender,
 'Alpha - CIC-SIL-Model' Model_Name,
 'SIL' as product,
 'NA' trenchCategory,
  r.calcFeature calcFeatures,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  r.start_time) AS appln_submit_datetime,
  loanmaster.disbursementDateTime,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  r.start_time)) as Application_month,  
FROM cleaned r 
left join risk_credit_mis.loan_master_table loanmaster 
  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
qualify row_number() over (partition by r.customerId,r.digitalLoanAccountId order by r.start_time desc) = 1
;
"""

dfd = client.query(sq).to_dataframe()
dfd.head()

,customerId,digitalLoanAccountId,Alpha_cic_sil_score,start_time,end_time,modelDisplayName,modelVersionId,new_loan_type,gender,Model_Name,product,trenchCategory,calcFeatures,appln_submit_datetime,disbursementDateTime,Application_month
0,3745439,dd2af354-db94-4ee8-a1f6-adeb9bba07f0,0.12556721423919442,2025-10-15 02:43:22.054181,2025-10-15 02:43:22.061638,Alpha - CIC-SIL-Model,v1,None,None,Alpha - CIC-SIL-Model,SIL,NA,"{""run_date"":1760486400000,""cic_Personal_Loans_...",2025-10-15 02:43:22.054181,NaT,2025-10
1,3745468,0b782713-0715-4932-afb4-0e91a9dcdfb1,0.07737738063850823,2025-10-15 02:55:12.663737,2025-10-15 02:55:12.669885,Alpha - CIC-SIL-Model,v1,None,None,Alpha - CIC-SIL-Model,SIL,NA,"{""run_date"":1760486400000,""cic_Personal_Loans_...",2025-10-15 02:55:12.663737,NaT,2025-10
2,3745780,68e43a22-9ab1-4653-90cc-300b865713a0,0.07600679533434303,2025-10-15 05:26:12.383751,2025-10-15 05:26:12.390027,Alpha - CIC-SIL-Model,v1,None,None,Alpha - CIC-SIL-Model,SIL,NA,"{""run_date"":1760486400000,""cic_Personal_Loans_...",2025-10-15 05:26:12.383751,NaT,2025-10
3,3745922,ce203a37-1f8f-49b1-880c-0ca8821e9851,0.10154681125757155,2025-10-15 06:11:51.749322,2025-10-15 06:11:51.756022,Alpha - CIC-SIL-Model,v1,None,None,Alpha - CIC-SIL-Model,SIL,NA,"{""run_date"":1760486400000,""cic_Personal_Loans_...",2025-10-15 06:11:51.749322,NaT,2025-10
4,3745969,27482adb-06ee-4047-a7a9-f40c941982ee,0.22872105434128923,2025-10-15 06:26:43.943733,2025-10-15 06:26:43.950163,Alpha - CIC-SIL-Model,v1,None,None,Alpha - CIC-SIL-Model,SIL,NA,"{""run_date"":1760486400000,""cic_Personal_Loans_...",2025-10-15 06:26:43.943733,NaT,2025-10


### Expand df

In [80]:
# Expand the calcFeatures column
expanded_df = expand_calc_features(dfd)

# Display the result
print(f"Original columns: {dfd.shape[1]}")
print(f"Expanded columns: {expanded_df.shape[1]}")
expanded_df.head()


Original columns: 16
Expanded columns: 26


,customerId,digitalLoanAccountId,Alpha_cic_sil_score,start_time,end_time,modelDisplayName,modelVersionId,new_loan_type,gender,Model_Name,product,trenchCategory,calcFeatures,appln_submit_datetime,disbursementDateTime,Application_month,calc_run_date,calc_cic_Personal_Loans_granted_contracts_amt_24M,calc_cic_days_since_last_inquiry,calc_cic_cnt_active_contracts,calc_cic_vel_contract_nongranted_cnt_12on24,calc_cic_max_amt_granted_24M,calc_cic_zero_non_granted_ever_flag,calc_cic_tot_active_contracts_util,calc_cic_vel_contract_granted_amt_12on24,calc_cic_zero_granted_ever_flag
0,3745439,dd2af354-db94-4ee8-a1f6-adeb9bba07f0,0.12556721423919442,2025-10-15 02:43:22.054181,2025-10-15 02:43:22.061638,Alpha - CIC-SIL-Model,v1,None,None,Alpha - CIC-SIL-Model,SIL,NA,"{""run_date"":1760486400000,""cic_Personal_Loans_...",2025-10-15 02:43:22.054181,NaT,2025-10,1.760486e+12,NaN,1251.0,NaN,NaN,NaN,1,NaN,NaN,0
1,3745468,0b782713-0715-4932-afb4-0e91a9dcdfb1,0.07737738063850823,2025-10-15 02:55:12.663737,2025-10-15 02:55:12.669885,Alpha - CIC-SIL-Model,v1,None,None,Alpha - CIC-SIL-Model,SIL,NA,"{""run_date"":1760486400000,""cic_Personal_Loans_...",2025-10-15 02:55:12.663737,NaT,2025-10,1.760486e+12,186331.0,53.0,29.0,0.664000,50000.0,0,0.633606,0.695159,0
2,3745780,68e43a22-9ab1-4653-90cc-300b865713a0,0.07600679533434303,2025-10-15 05:26:12.383751,2025-10-15 05:26:12.390027,Alpha - CIC-SIL-Model,v1,None,None,Alpha - CIC-SIL-Model,SIL,NA,"{""run_date"":1760486400000,""cic_Personal_Loans_...",2025-10-15 05:26:12.383751,NaT,2025-10,1.760486e+12,6565.0,202.0,1.0,0.664000,4000.0,0,0.000000,1.297017,0
3,3745922,ce203a37-1f8f-49b1-880c-0ca8821e9851,0.10154681125757155,2025-10-15 06:11:51.749322,2025-10-15 06:11:51.756022,Alpha - CIC-SIL-Model,v1,None,None,Alpha - CIC-SIL-Model,SIL,NA,"{""run_date"":1760486400000,""cic_Personal_Loans_...",2025-10-15 06:11:51.749322,NaT,2025-10,1.760486e+12,1372.0,46.0,NaN,0.664000,5000.0,0,NaN,1.492351,0
4,3745969,27482adb-06ee-4047-a7a9-f40c941982ee,0.22872105434128923,2025-10-15 06:26:43.943733,2025-10-15 06:26:43.950163,Alpha - CIC-SIL-Model,v1,None,None,Alpha - CIC-SIL-Model,SIL,NA,"{""run_date"":1760486400000,""cic_Personal_Loans_...",2025-10-15 06:26:43.943733,NaT,2025-10,1.760486e+12,NaN,0.0,NaN,2.012048,NaN,0,NaN,NaN,0


In [81]:
df = expanded_df.drop(columns=['calcFeatures']).copy()

In [82]:
df['Alpha_cic_sil_score'] = pd.to_numeric(df['Alpha_cic_sil_score'], errors='coerce')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62056 entries, 0 to 62055
Data columns (total 25 columns):
 #   Column                                             Non-Null Count  Dtype         
---  ------                                             --------------  -----         
 0   customerId                                         62056 non-null  object        
 1   digitalLoanAccountId                               62056 non-null  object        
 2   Alpha_cic_sil_score                                62056 non-null  float64       
 3   start_time                                         62056 non-null  datetime64[us]
 4   end_time                                           62056 non-null  datetime64[us]
 5   modelDisplayName                                   62056 non-null  object        
 6   modelVersionId                                     62056 non-null  object        
 7   new_loan_type                                      61812 non-null  object        
 8   gender          

### run the psi calculation

In [83]:
df = df.copy()

# Define feature list
feature_list = ['Alpha_cic_sil_score',  'calc_cic_Personal_Loans_granted_contracts_amt_24M',
       'calc_cic_days_since_last_inquiry', 'calc_cic_cnt_active_contracts',
       'calc_cic_vel_contract_nongranted_cnt_12on24',
       'calc_cic_max_amt_granted_24M', 'calc_cic_zero_non_granted_ever_flag',
       'calc_cic_tot_active_contracts_util',
       'calc_cic_vel_contract_granted_amt_12on24',
       'calc_cic_zero_granted_ever_flag']

# Define segment columns
segment_columns = ['new_loan_type', 'gender', 'osType', 'trenchCategory', 'product']
# Calculate month-on-month PSI
psi_results = calculate_month_on_month_psi(df, feature_list, segment_columns)
psi_results['modelDisplayName'] = df['modelDisplayName'].iloc[0]
psi_results['Model_Name'] = df['Model_Name'].iloc[0]
psi_results['modelVersionId'] = df['modelVersionId'].iloc[0]
psi_results['trenchCategory'] = df['trenchCategory'].iloc[0]
psi_results = psi_results[[ 'modelDisplayName', 'Model_Name',   'modelVersionId', 'trenchCategory','Feature', 'Feature_Type', 'Segment_Column', 'Segment_Value', 'Month',
       'Base_Month', 'Current_Month', 'PSI']].copy()

# Calculate bin-level PSI
bin_psi_results = calculate_bin_level_psi(df, feature_list, segment_columns)
bin_psi_results['modelDisplayName'] = df['modelDisplayName'].iloc[0]
bin_psi_results['Model_Name'] = df['Model_Name'].iloc[0]
bin_psi_results['modelVersionId'] = df['modelVersionId'].iloc[0]
bin_psi_results['trenchCategory'] = df['trenchCategory'].iloc[0]
bin_psi_results = bin_psi_results[[ 'modelDisplayName', 'Model_Name',  'modelVersionId', 'trenchCategory','Feature', 'Feature_Type', 'Segment_Column', 'Segment_Value', 'Month',
       'Base_Month', 'Current_Month', 'Bin', 'Bin_Range', 'Bin_Min', 'Bin_Max', 'Base_Percentage',
       'Actual_Percentage', 'Bin_PSI']].copy()

# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.temp_csi_new_monitoring_data"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(psi_results, table_id, job_config=job_config)
job.result()  # Wait for the job to complete


# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.temp_csi_new_monitoring_data_feature_bin_level"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(bin_psi_results, table_id, job_config=job_config)
job.result()  # Wait for the job to complete

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=79c1a6ca-d32f-4105-96ce-54bb19f94b6f>

In [84]:
psi_results

,modelDisplayName,Model_Name,modelVersionId,trenchCategory,Feature,Feature_Type,Segment_Column,Segment_Value,Month,Base_Month,Current_Month,PSI
0,Alpha - CIC-SIL-Model,Alpha - CIC-SIL-Model,v1,NA,Alpha_cic_sil_score,numerical,Overall,All,2025-03,2025-03,2025-03,0.000000
1,Alpha - CIC-SIL-Model,Alpha - CIC-SIL-Model,v1,NA,Alpha_cic_sil_score,numerical,Overall,All,2025-04,2025-03,2025-04,0.202530
2,Alpha - CIC-SIL-Model,Alpha - CIC-SIL-Model,v1,NA,Alpha_cic_sil_score,numerical,Overall,All,2025-05,2025-03,2025-05,0.120327
3,Alpha - CIC-SIL-Model,Alpha - CIC-SIL-Model,v1,NA,Alpha_cic_sil_score,numerical,Overall,All,2025-06,2025-03,2025-06,0.264234
4,Alpha - CIC-SIL-Model,Alpha - CIC-SIL-Model,v1,NA,Alpha_cic_sil_score,numerical,Overall,All,2025-07,2025-03,2025-07,0.268351
...,...,...,...,...,...,...,...,...,...,...,...,...
555,Alpha - CIC-SIL-Model,Alpha - CIC-SIL-Model,v1,NA,calc_cic_zero_granted_ever_flag,categorical,product,SIL,2025-06,2025-03,2025-06,0.000000
556,Alpha - CIC-SIL-Model,Alpha - CIC-SIL-Model,v1,NA,calc_cic_zero_granted_ever_flag,categorical,product,SIL,2025-07,2025-03,2025-07,0.000000
557,Alpha - CIC-SIL-Model,Alpha - CIC-SIL-Model,v1,NA,calc_cic_zero_granted_ever_flag,categorical,product,SIL,2025-08,2025-03,2025-08,0.000000
558,Alpha - CIC-SIL-Model,Alpha - CIC-SIL-Model,v1,NA,calc_cic_zero_granted_ever_flag,categorical,product,SIL,2025-09,2025-03,2025-09,0.000000


## Alpha - IncomeEstimationModel

In [85]:
sq = """ 
WITH cleaned AS (
  SELECT
    customerId,digitalLoanAccountId,prediction,start_time,end_time,modelDisplayName,modelVersionId,
    REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeature
  FROM `prj-prod-dataplatform.audit_balance.ml_model_run_details`
  WHERE modelDisplayName = 'Alpha  - IncomeEstimationModel')
SELECT
  r.customerId,r.digitalLoanAccountId,prediction Alpha_Income_Estimated_score,start_time,end_time,modelDisplayName,modelVersionId,
  loanmaster.new_loan_type,
 loanmaster.gender,
  'Alpha  - IncomeEstimationModel' Model_Name,
  'SIL' as product,
  'NA' trenchCategory,
  r.calcFeature calcFeatures,
  JSON_VALUE(calcFeature, "$.inc_alpha_cic_credit_avg_credit_limit") AS inc_alpha_cic_credit_avg_credit_limit,
  JSON_VALUE(calcFeature, "$.inc_alpha_cic_max_active_contracts_amt") AS inc_alpha_cic_max_active_contracts_amt,
  JSON_VALUE(calcFeature, "$.inc_alpha_ln_age") AS inc_alpha_ln_age,
  JSON_VALUE(calcFeature, "$.inc_alpha_doc_type_rolled") AS inc_alpha_doc_type_rolled,
  JSON_VALUE(calcFeature, "$.inc_alpha_ln_brand") AS inc_alpha_ln_brand,
  JSON_VALUE(calcFeature, "$.inc_alpha_ln_city") AS inc_alpha_ln_city,
  JSON_VALUE(calcFeature, "$.inc_alpha_ln_cnt_dependents") AS inc_alpha_ln_cnt_dependents,
  JSON_VALUE(calcFeature, "$.inc_alpha_ln_education_level") AS inc_alpha_ln_education_level,
  JSON_VALUE(calcFeature, "$.inc_alpha_ln_employment_type_new") AS inc_alpha_ln_employment_type_new,
  JSON_VALUE(calcFeature, "$.inc_alpha_ln_gender") AS inc_alpha_ln_gender,
  JSON_VALUE(calcFeature, "$.inc_alpha_ln_industry_new") AS inc_alpha_ln_industry_new,
  JSON_VALUE(calcFeature, "$.inc_alpha_ln_loan_prod_type") AS inc_alpha_ln_loan_prod_type,
  JSON_VALUE(calcFeature, "$.inc_alpha_ln_marital_status_new") AS inc_alpha_ln_marital_status_new,
  JSON_VALUE(calcFeature, "$.inc_alpha_ln_nature_of_work_new") AS inc_alpha_ln_nature_of_work_new,
  JSON_VALUE(calcFeature, "$.inc_alpha_ln_osversion_bin") AS inc_alpha_ln_osversion_bin,
  JSON_VALUE(calcFeature, "$.inc_alpha_ln_purpose") AS inc_alpha_ln_purpose,
  JSON_VALUE(calcFeature, "$.inc_alpha_ln_source_of_funds_new") AS inc_alpha_ln_source_of_funds_new,
  JSON_VALUE(calcFeature, "$.inc_alpha_encoded_company_name_grouped") AS inc_alpha_encoded_company_name_grouped,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  r.start_time) AS appln_submit_datetime,
  loanmaster.disbursementDateTime,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  r.start_time)) as Application_month,
FROM cleaned r
left join risk_credit_mis.loan_master_table loanmaster 
  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
qualify row_number() over (partition by r.customerId,r.digitalLoanAccountId order by r.start_time desc) = 1
  ;
"""
dfd = client.query(sq).to_dataframe()
dfd.head()

,customerId,digitalLoanAccountId,Alpha_Income_Estimated_score,start_time,end_time,modelDisplayName,modelVersionId,new_loan_type,gender,Model_Name,product,trenchCategory,calcFeatures,inc_alpha_cic_credit_avg_credit_limit,inc_alpha_cic_max_active_contracts_amt,inc_alpha_ln_age,inc_alpha_doc_type_rolled,inc_alpha_ln_brand,inc_alpha_ln_city,inc_alpha_ln_cnt_dependents,inc_alpha_ln_education_level,inc_alpha_ln_employment_type_new,inc_alpha_ln_gender,inc_alpha_ln_industry_new,inc_alpha_ln_loan_prod_type,inc_alpha_ln_marital_status_new,inc_alpha_ln_nature_of_work_new,inc_alpha_ln_osversion_bin,inc_alpha_ln_purpose,inc_alpha_ln_source_of_funds_new,inc_alpha_encoded_company_name_grouped,appln_submit_datetime,disbursementDateTime,Application_month
0,2977464,ca4670a7-d0f7-482c-99cd-f67d37eec32a,9404.504511199038,2025-08-24 09:17:37.660941,2025-08-24 09:17:37.753093,Alpha - IncomeEstimationModel,v1,SIL-Instore,M,Alpha - IncomeEstimationModel,SIL,NA,"{""inc_alpha_cic_credit_avg_credit_limit"":0,""in...",0,50000.0,30,ID Card,Apple,CF0190,1,Technical/Vocational Graduate,Employed - Govt. Employee,M,Govt. Institution,BNPL,Single,Government - Elected Official,5-Pioneer,"Sala set, Couch, Sofa set",Salary,0.0,2025-08-24 17:17:29,NaT,2025-08
1,3346341,f9855652-a682-44d7-a7ac-168f8809629e,28641.592199975992,2025-03-29 01:50:59.112011,2025-03-29 01:50:59.189990,Alpha - IncomeEstimationModel,v1,SIL-Instore,F,Alpha - IncomeEstimationModel,SIL,NA,"{""inc_alpha_cic_credit_avg_credit_limit"":0,""in...",0,0.0,45,Other,Infinix,CF0314,2,College Graduate,Self-Employed/Private Practice,F,Online Seller,BNPL,Married,Online Seller,4-Innovator,"Washing machine, dryer or both",2,30892.5062448281,2025-03-29 09:50:48,2025-03-29 09:53:29,2025-03
2,3347664,4e59ea47-8c7d-4c80-ae9f-084d918b8210,20695.003653980988,2025-03-29 10:34:28.903647,2025-03-29 10:34:28.977758,Alpha - IncomeEstimationModel,v1,SIL-Instore,F,Alpha - IncomeEstimationModel,SIL,NA,"{""inc_alpha_cic_credit_avg_credit_limit"":20000...",20000,20000.0,30,Other,Apple,CF1738,0,Post-Graduate/Doctorate Degree Holder,Employed,F,Other Professional services,BNPL,Married,Other Professional services,3-ComfortSeeker,"Stereo set, Soundbar, Audio equipment",1,30274.2246045263,2025-03-29 18:34:19,NaT,2025-03
3,3383159,80859ffe-4a2b-43cb-b886-32bd1236b05c,29075.44096502884,2025-04-16 06:27:43.168129,2025-04-16 06:27:43.242437,Alpha - IncomeEstimationModel,v1,SIL ZERO,M,Alpha - IncomeEstimationModel,SIL,NA,"{""inc_alpha_cic_credit_avg_credit_limit"":0,""in...",0,19992,48,Other,HONOR,CF0366,2,Technical/Vocational Graduate,Business Owner,M,Business Owner (MSME),BNPL0.1,Married,Business Owner (MSME),4-Innovator,Air conditioners,3,30892.5062448281,2025-04-16 14:27:34,2025-04-16 14:31:00,2025-04
4,3383630,7d7e7b71-1a09-412f-a652-5d377a1f9035,8169.341321672695,2025-04-16 09:04:06.627319,2025-04-16 09:04:06.702753,Alpha - IncomeEstimationModel,v1,SIL-Instore,M,Alpha - IncomeEstimationModel,SIL,NA,"{""inc_alpha_cic_credit_avg_credit_limit"":40000...",40000,40000.0,43,Other,vivo,CF1085,1,High School Undergraduate,Employed,M,Engineer,BNPL,Married,Engineer,4-Innovator,Air conditioners,1,0.0,2025-04-16 17:03:59,2025-04-16 17:07:32,2025-04


### Expand df


In [86]:
# Expand the calcFeatures column
expanded_df = dfd.copy()

# Display the result
print(f"Original columns: {dfd.shape[1]}")
print(f"Expanded columns: {expanded_df.shape[1]}")
expanded_df.head()


Original columns: 34
Expanded columns: 34


,customerId,digitalLoanAccountId,Alpha_Income_Estimated_score,start_time,end_time,modelDisplayName,modelVersionId,new_loan_type,gender,Model_Name,product,trenchCategory,calcFeatures,inc_alpha_cic_credit_avg_credit_limit,inc_alpha_cic_max_active_contracts_amt,inc_alpha_ln_age,inc_alpha_doc_type_rolled,inc_alpha_ln_brand,inc_alpha_ln_city,inc_alpha_ln_cnt_dependents,inc_alpha_ln_education_level,inc_alpha_ln_employment_type_new,inc_alpha_ln_gender,inc_alpha_ln_industry_new,inc_alpha_ln_loan_prod_type,inc_alpha_ln_marital_status_new,inc_alpha_ln_nature_of_work_new,inc_alpha_ln_osversion_bin,inc_alpha_ln_purpose,inc_alpha_ln_source_of_funds_new,inc_alpha_encoded_company_name_grouped,appln_submit_datetime,disbursementDateTime,Application_month
0,2977464,ca4670a7-d0f7-482c-99cd-f67d37eec32a,9404.504511199038,2025-08-24 09:17:37.660941,2025-08-24 09:17:37.753093,Alpha - IncomeEstimationModel,v1,SIL-Instore,M,Alpha - IncomeEstimationModel,SIL,NA,"{""inc_alpha_cic_credit_avg_credit_limit"":0,""in...",0,50000.0,30,ID Card,Apple,CF0190,1,Technical/Vocational Graduate,Employed - Govt. Employee,M,Govt. Institution,BNPL,Single,Government - Elected Official,5-Pioneer,"Sala set, Couch, Sofa set",Salary,0.0,2025-08-24 17:17:29,NaT,2025-08
1,3346341,f9855652-a682-44d7-a7ac-168f8809629e,28641.592199975992,2025-03-29 01:50:59.112011,2025-03-29 01:50:59.189990,Alpha - IncomeEstimationModel,v1,SIL-Instore,F,Alpha - IncomeEstimationModel,SIL,NA,"{""inc_alpha_cic_credit_avg_credit_limit"":0,""in...",0,0.0,45,Other,Infinix,CF0314,2,College Graduate,Self-Employed/Private Practice,F,Online Seller,BNPL,Married,Online Seller,4-Innovator,"Washing machine, dryer or both",2,30892.5062448281,2025-03-29 09:50:48,2025-03-29 09:53:29,2025-03
2,3347664,4e59ea47-8c7d-4c80-ae9f-084d918b8210,20695.003653980988,2025-03-29 10:34:28.903647,2025-03-29 10:34:28.977758,Alpha - IncomeEstimationModel,v1,SIL-Instore,F,Alpha - IncomeEstimationModel,SIL,NA,"{""inc_alpha_cic_credit_avg_credit_limit"":20000...",20000,20000.0,30,Other,Apple,CF1738,0,Post-Graduate/Doctorate Degree Holder,Employed,F,Other Professional services,BNPL,Married,Other Professional services,3-ComfortSeeker,"Stereo set, Soundbar, Audio equipment",1,30274.2246045263,2025-03-29 18:34:19,NaT,2025-03
3,3383159,80859ffe-4a2b-43cb-b886-32bd1236b05c,29075.44096502884,2025-04-16 06:27:43.168129,2025-04-16 06:27:43.242437,Alpha - IncomeEstimationModel,v1,SIL ZERO,M,Alpha - IncomeEstimationModel,SIL,NA,"{""inc_alpha_cic_credit_avg_credit_limit"":0,""in...",0,19992,48,Other,HONOR,CF0366,2,Technical/Vocational Graduate,Business Owner,M,Business Owner (MSME),BNPL0.1,Married,Business Owner (MSME),4-Innovator,Air conditioners,3,30892.5062448281,2025-04-16 14:27:34,2025-04-16 14:31:00,2025-04
4,3383630,7d7e7b71-1a09-412f-a652-5d377a1f9035,8169.341321672695,2025-04-16 09:04:06.627319,2025-04-16 09:04:06.702753,Alpha - IncomeEstimationModel,v1,SIL-Instore,M,Alpha - IncomeEstimationModel,SIL,NA,"{""inc_alpha_cic_credit_avg_credit_limit"":40000...",40000,40000.0,43,Other,vivo,CF1085,1,High School Undergraduate,Employed,M,Engineer,BNPL,Married,Engineer,4-Innovator,Air conditioners,1,0.0,2025-04-16 17:03:59,2025-04-16 17:07:32,2025-04


In [87]:
df = expanded_df.drop(columns=['calcFeatures']).copy()

In [88]:
df['Alpha_Income_Estimated_score'] = pd.to_numeric(df['Alpha_Income_Estimated_score'], errors='coerce')
df.columns

Index(['customerId', 'digitalLoanAccountId', 'Alpha_Income_Estimated_score',
       'start_time', 'end_time', 'modelDisplayName', 'modelVersionId',
       'new_loan_type', 'gender', 'Model_Name', 'product', 'trenchCategory',
       'inc_alpha_cic_credit_avg_credit_limit',
       'inc_alpha_cic_max_active_contracts_amt', 'inc_alpha_ln_age',
       'inc_alpha_doc_type_rolled', 'inc_alpha_ln_brand', 'inc_alpha_ln_city',
       'inc_alpha_ln_cnt_dependents', 'inc_alpha_ln_education_level',
       'inc_alpha_ln_employment_type_new', 'inc_alpha_ln_gender',
       'inc_alpha_ln_industry_new', 'inc_alpha_ln_loan_prod_type',
       'inc_alpha_ln_marital_status_new', 'inc_alpha_ln_nature_of_work_new',
       'inc_alpha_ln_osversion_bin', 'inc_alpha_ln_purpose',
       'inc_alpha_ln_source_of_funds_new',
       'inc_alpha_encoded_company_name_grouped', 'appln_submit_datetime',
       'disbursementDateTime', 'Application_month'],
      dtype='object')

### run psi calculation

In [89]:
df = df.copy()

# Define feature list
feature_list = ['Alpha_Income_Estimated_score',   'inc_alpha_cic_credit_avg_credit_limit',
       'inc_alpha_cic_max_active_contracts_amt', 'inc_alpha_ln_age',
       'inc_alpha_doc_type_rolled', 'inc_alpha_ln_brand', 'inc_alpha_ln_city',
       'inc_alpha_ln_cnt_dependents', 'inc_alpha_ln_education_level',
       'inc_alpha_ln_employment_type_new', 'inc_alpha_ln_gender',
       'inc_alpha_ln_industry_new', 'inc_alpha_ln_loan_prod_type',
       'inc_alpha_ln_marital_status_new', 'inc_alpha_ln_nature_of_work_new',
       'inc_alpha_ln_osversion_bin', 'inc_alpha_ln_purpose',
       'inc_alpha_ln_source_of_funds_new',
       'inc_alpha_encoded_company_name_grouped',]

# Define segment columns
segment_columns = ['new_loan_type', 'gender', 'osType', 'trenchCategory', 'product']
# Calculate month-on-month PSI
psi_results = calculate_month_on_month_psi(df, feature_list, segment_columns)
psi_results['modelDisplayName'] = df['modelDisplayName'].iloc[0]
psi_results['Model_Name'] = df['Model_Name'].iloc[0]
psi_results['modelVersionId'] = df['modelVersionId'].iloc[0]
psi_results['trenchCategory'] = df['trenchCategory'].iloc[0]
psi_results = psi_results[[ 'modelDisplayName', 'Model_Name',   'modelVersionId', 'trenchCategory','Feature', 'Feature_Type', 'Segment_Column', 'Segment_Value', 'Month',
       'Base_Month', 'Current_Month', 'PSI']].copy()

# Calculate bin-level PSI
bin_psi_results = calculate_bin_level_psi(df, feature_list, segment_columns)
bin_psi_results['modelDisplayName'] = df['modelDisplayName'].iloc[0]
bin_psi_results['Model_Name'] = df['Model_Name'].iloc[0]
bin_psi_results['modelVersionId'] = df['modelVersionId'].iloc[0]
bin_psi_results['trenchCategory'] = df['trenchCategory'].iloc[0]
bin_psi_results = bin_psi_results[[ 'modelDisplayName', 'Model_Name',  'modelVersionId', 'trenchCategory','Feature', 'Feature_Type', 'Segment_Column', 'Segment_Value', 'Month',
       'Base_Month', 'Current_Month', 'Bin', 'Bin_Range', 'Bin_Min', 'Bin_Max', 'Base_Percentage',
       'Actual_Percentage', 'Bin_PSI']].copy()

# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.temp_csi_new_monitoring_data"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(psi_results, table_id, job_config=job_config)
job.result()  # Wait for the job to complete


# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.temp_csi_new_monitoring_data_feature_bin_level"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(bin_psi_results, table_id, job_config=job_config)
job.result()  # Wait for the job to complete

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=d0b508d2-6027-4950-b161-d245d9a25bc7>

In [90]:
psi_results

,modelDisplayName,Model_Name,modelVersionId,trenchCategory,Feature,Feature_Type,Segment_Column,Segment_Value,Month,Base_Month,Current_Month,PSI
0,Alpha - IncomeEstimationModel,Alpha - IncomeEstimationModel,v1,NA,Alpha_Income_Estimated_score,numerical,Overall,All,2025-03,2025-03,2025-03,0.000000
1,Alpha - IncomeEstimationModel,Alpha - IncomeEstimationModel,v1,NA,Alpha_Income_Estimated_score,numerical,Overall,All,2025-04,2025-03,2025-04,0.170057
2,Alpha - IncomeEstimationModel,Alpha - IncomeEstimationModel,v1,NA,Alpha_Income_Estimated_score,numerical,Overall,All,2025-05,2025-03,2025-05,0.171290
3,Alpha - IncomeEstimationModel,Alpha - IncomeEstimationModel,v1,NA,Alpha_Income_Estimated_score,numerical,Overall,All,2025-06,2025-03,2025-06,0.412254
4,Alpha - IncomeEstimationModel,Alpha - IncomeEstimationModel,v1,NA,Alpha_Income_Estimated_score,numerical,Overall,All,2025-07,2025-03,2025-07,0.380292
...,...,...,...,...,...,...,...,...,...,...,...,...
1059,Alpha - IncomeEstimationModel,Alpha - IncomeEstimationModel,v1,NA,inc_alpha_encoded_company_name_grouped,categorical,product,SIL,2025-06,2025-03,2025-06,0.203156
1060,Alpha - IncomeEstimationModel,Alpha - IncomeEstimationModel,v1,NA,inc_alpha_encoded_company_name_grouped,categorical,product,SIL,2025-07,2025-03,2025-07,0.175992
1061,Alpha - IncomeEstimationModel,Alpha - IncomeEstimationModel,v1,NA,inc_alpha_encoded_company_name_grouped,categorical,product,SIL,2025-08,2025-03,2025-08,0.215967
1062,Alpha - IncomeEstimationModel,Alpha - IncomeEstimationModel,v1,NA,inc_alpha_encoded_company_name_grouped,categorical,product,SIL,2025-09,2025-03,2025-09,0.199190


## SIL Alpha - StackingModel

In [91]:
sq = """ WITH cleaned AS (
  SELECT
    customerId,digitalLoanAccountId,prediction,start_time,end_time,modelDisplayName,modelVersionId,
    REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeature
  FROM `prj-prod-dataplatform.audit_balance.ml_model_run_details`
  WHERE modelDisplayName = 'Alpha - StackingModel')
SELECT distinct
 r.customerId,r.digitalLoanAccountId,prediction Sil_Alpha_Stack_score,start_time,end_time,modelDisplayName,modelVersionId,
 loanmaster.new_loan_type,
 loanmaster.gender,
 'SIL Alpha - StackingModel' Model_Name,
 'SIL' as product,
 'NA' trenchCategory,
  r.calcFeature calcFeatures,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  r.start_time) AS appln_submit_datetime,
  loanmaster.disbursementDateTime,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  r.start_time)) as Application_month,  
FROM cleaned r 
left join risk_credit_mis.loan_master_table loanmaster 
  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
qualify row_number() over(partition by r.digitalLoanAccountId order by r.start_time desc)=1
  ;"""
dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
dfd.head()

,customerId,digitalLoanAccountId,Sil_Alpha_Stack_score,start_time,end_time,modelDisplayName,modelVersionId,new_loan_type,gender,Model_Name,product,trenchCategory,calcFeatures,appln_submit_datetime,disbursementDateTime,Application_month
0,3342206,8370425d-a926-4dd0-91e6-923c5b635975,0.010184323374035558,2025-03-26 11:05:11.353452,2025-03-26 11:05:11.356909,Alpha - StackingModel,v1,SIL-Instore,F,SIL Alpha - StackingModel,SIL,NA,"{""sb_demo_score"": null, ""s_cic_score"": 0.12352...",2025-03-26 19:05:02,2025-03-26 19:12:06,2025-03
1,3341143,d2e1e455-b74a-43fd-8583-3fb0a92e4c38,0.009456102482162755,2025-03-26 02:39:31.814207,2025-03-26 02:39:31.817840,Alpha - StackingModel,v1,SIL ZERO,F,SIL Alpha - StackingModel,SIL,NA,"{""sb_demo_score"": null, ""s_cic_score"": 0.10247...",2025-03-26 10:39:23,2025-03-26 10:42:33,2025-03
2,3348528,999b153a-a0f9-4a5c-80ae-d07d946f051c,0.010246952080656055,2025-03-30 03:08:06.093068,2025-03-30 03:08:06.096661,Alpha - StackingModel,v1,SIL-Instore,M,SIL Alpha - StackingModel,SIL,NA,"{""sb_demo_score"": null, ""s_cic_score"": 0.12666...",2025-03-30 11:07:59,2025-03-30 11:13:29,2025-03
3,3341472,27ef1d15-0b45-4d23-9231-22e9adb9a0b4,0.009838923270438915,2025-03-26 05:36:53.345841,2025-03-26 05:36:53.349260,Alpha - StackingModel,v1,SIL-Instore,F,SIL Alpha - StackingModel,SIL,NA,"{""sb_demo_score"": null, ""s_cic_score"": 0.10794...",2025-03-26 13:36:45,2025-03-26 13:47:50,2025-03
4,3346587,f9563b52-d370-4902-a4b3-5ce9fd99af08,0.009131927802806888,2025-03-29 03:49:21.858720,2025-03-29 03:49:21.862367,Alpha - StackingModel,v1,SIL-Instore,F,SIL Alpha - StackingModel,SIL,NA,"{""sb_demo_score"": null, ""s_cic_score"": 0.08191...",2025-03-29 11:49:12,2025-03-29 11:55:45,2025-03


### Expand df

In [92]:
# Expand the calcFeatures column
expanded_df = expand_calc_features(dfd)

# Display the result
print(f"Original columns: {dfd.shape[1]}")
print(f"Expanded columns: {expanded_df.shape[1]}")
expanded_df.head()


Original columns: 16
Expanded columns: 20


,customerId,digitalLoanAccountId,Sil_Alpha_Stack_score,start_time,end_time,modelDisplayName,modelVersionId,new_loan_type,gender,Model_Name,product,trenchCategory,calcFeatures,appln_submit_datetime,disbursementDateTime,Application_month,calc_sb_demo_score,calc_s_cic_score,calc_s_credo_score,calc_s_apps_score
0,3342206,8370425d-a926-4dd0-91e6-923c5b635975,0.010184323374035558,2025-03-26 11:05:11.353452,2025-03-26 11:05:11.356909,Alpha - StackingModel,v1,SIL-Instore,F,SIL Alpha - StackingModel,SIL,NA,"{""sb_demo_score"": null, ""s_cic_score"": 0.12352...",2025-03-26 19:05:02,2025-03-26 19:12:06,2025-03,NaN,0.123520,NaN,NaN
1,3341143,d2e1e455-b74a-43fd-8583-3fb0a92e4c38,0.009456102482162755,2025-03-26 02:39:31.814207,2025-03-26 02:39:31.817840,Alpha - StackingModel,v1,SIL ZERO,F,SIL Alpha - StackingModel,SIL,NA,"{""sb_demo_score"": null, ""s_cic_score"": 0.10247...",2025-03-26 10:39:23,2025-03-26 10:42:33,2025-03,NaN,0.102474,NaN,NaN
2,3348528,999b153a-a0f9-4a5c-80ae-d07d946f051c,0.010246952080656055,2025-03-30 03:08:06.093068,2025-03-30 03:08:06.096661,Alpha - StackingModel,v1,SIL-Instore,M,SIL Alpha - StackingModel,SIL,NA,"{""sb_demo_score"": null, ""s_cic_score"": 0.12666...",2025-03-30 11:07:59,2025-03-30 11:13:29,2025-03,NaN,0.126668,NaN,NaN
3,3341472,27ef1d15-0b45-4d23-9231-22e9adb9a0b4,0.009838923270438915,2025-03-26 05:36:53.345841,2025-03-26 05:36:53.349260,Alpha - StackingModel,v1,SIL-Instore,F,SIL Alpha - StackingModel,SIL,NA,"{""sb_demo_score"": null, ""s_cic_score"": 0.10794...",2025-03-26 13:36:45,2025-03-26 13:47:50,2025-03,NaN,0.107947,NaN,NaN
4,3346587,f9563b52-d370-4902-a4b3-5ce9fd99af08,0.009131927802806888,2025-03-29 03:49:21.858720,2025-03-29 03:49:21.862367,Alpha - StackingModel,v1,SIL-Instore,F,SIL Alpha - StackingModel,SIL,NA,"{""sb_demo_score"": null, ""s_cic_score"": 0.08191...",2025-03-29 11:49:12,2025-03-29 11:55:45,2025-03,NaN,0.081917,NaN,NaN


In [93]:
df = expanded_df.drop(columns=['calcFeatures']).copy()
df = df.drop_duplicates(keep='first')

In [94]:
df['Sil_Alpha_Stack_score'] = pd.to_numeric(df['Sil_Alpha_Stack_score'], errors='coerce')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62058 entries, 0 to 62057
Data columns (total 19 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   customerId             62058 non-null  object        
 1   digitalLoanAccountId   62058 non-null  object        
 2   Sil_Alpha_Stack_score  62058 non-null  float64       
 3   start_time             62058 non-null  datetime64[us]
 4   end_time               62058 non-null  datetime64[us]
 5   modelDisplayName       62058 non-null  object        
 6   modelVersionId         62058 non-null  object        
 7   new_loan_type          61812 non-null  object        
 8   gender                 61812 non-null  object        
 9   Model_Name             62058 non-null  object        
 10  product                62058 non-null  object        
 11  trenchCategory         62058 non-null  object        
 12  appln_submit_datetime  62058 non-null  datetime64[us]
 13  d

### run psi calculation

In [95]:
df = df.copy()

# Define feature list
feature_list = ['Sil_Alpha_Stack_score',   'calc_sb_demo_score',
       'calc_s_cic_score', 'calc_s_credo_score', 'calc_s_apps_score']

# Define segment columns
segment_columns = ['new_loan_type', 'gender', 'osType', 'trenchCategory', 'product']
# Calculate month-on-month PSI
psi_results = calculate_month_on_month_psi(df, feature_list, segment_columns)
psi_results['modelDisplayName'] = df['modelDisplayName'].iloc[0]
psi_results['Model_Name'] = df['Model_Name'].iloc[0]
psi_results['modelVersionId'] = df['modelVersionId'].iloc[0]
psi_results['trenchCategory'] = df['trenchCategory'].iloc[0]
psi_results = psi_results[[ 'modelDisplayName', 'Model_Name',   'modelVersionId', 'trenchCategory','Feature', 'Feature_Type', 'Segment_Column', 'Segment_Value', 'Month',
       'Base_Month', 'Current_Month', 'PSI']].copy()

# Calculate bin-level PSI
bin_psi_results = calculate_bin_level_psi(df, feature_list, segment_columns)
bin_psi_results['modelDisplayName'] = df['modelDisplayName'].iloc[0]
bin_psi_results['Model_Name'] = df['Model_Name'].iloc[0]
bin_psi_results['modelVersionId'] = df['modelVersionId'].iloc[0]
bin_psi_results['trenchCategory'] = df['trenchCategory'].iloc[0]
bin_psi_results = bin_psi_results[[ 'modelDisplayName', 'Model_Name',  'modelVersionId', 'trenchCategory','Feature', 'Feature_Type', 'Segment_Column', 'Segment_Value', 'Month',
       'Base_Month', 'Current_Month', 'Bin', 'Bin_Range', 'Bin_Min', 'Bin_Max', 'Base_Percentage',
       'Actual_Percentage', 'Bin_PSI']].copy()

# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.temp_csi_new_monitoring_data"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(psi_results, table_id, job_config=job_config)
job.result()  # Wait for the job to complete


# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.temp_csi_new_monitoring_data_feature_bin_level"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(bin_psi_results, table_id, job_config=job_config)
job.result()  # Wait for the job to complete

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=2158cdaa-d808-4181-a9ac-9489e086a37a>

In [96]:
psi_results

,modelDisplayName,Model_Name,modelVersionId,trenchCategory,Feature,Feature_Type,Segment_Column,Segment_Value,Month,Base_Month,Current_Month,PSI
0,Alpha - StackingModel,SIL Alpha - StackingModel,v1,NA,Sil_Alpha_Stack_score,numerical,Overall,All,2025-03,2025-03,2025-03,0.000000
1,Alpha - StackingModel,SIL Alpha - StackingModel,v1,NA,Sil_Alpha_Stack_score,numerical,Overall,All,2025-04,2025-03,2025-04,3.601985
2,Alpha - StackingModel,SIL Alpha - StackingModel,v1,NA,Sil_Alpha_Stack_score,numerical,Overall,All,2025-05,2025-03,2025-05,7.812161
3,Alpha - StackingModel,SIL Alpha - StackingModel,v1,NA,Sil_Alpha_Stack_score,numerical,Overall,All,2025-06,2025-03,2025-06,6.279809
4,Alpha - StackingModel,SIL Alpha - StackingModel,v1,NA,Sil_Alpha_Stack_score,numerical,Overall,All,2025-07,2025-03,2025-07,6.263542
...,...,...,...,...,...,...,...,...,...,...,...,...
275,Alpha - StackingModel,SIL Alpha - StackingModel,v1,NA,calc_s_apps_score,numerical,product,SIL,2025-06,2025-03,2025-06,0.000000
276,Alpha - StackingModel,SIL Alpha - StackingModel,v1,NA,calc_s_apps_score,numerical,product,SIL,2025-07,2025-03,2025-07,0.000000
277,Alpha - StackingModel,SIL Alpha - StackingModel,v1,NA,calc_s_apps_score,numerical,product,SIL,2025-08,2025-03,2025-08,0.000000
278,Alpha - StackingModel,SIL Alpha - StackingModel,v1,NA,calc_s_apps_score,numerical,product,SIL,2025-09,2025-03,2025-09,0.000000


## SIL Beta - AppsScoreModel

In [97]:
sq = """ 
WITH cleaned AS (
  SELECT
    customerId,digitalLoanAccountId,prediction,start_time,end_time,modelDisplayName,modelVersionId,

    REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeature,
    REPLACE(REPLACE(prediction, "'", '"'), "None", "null") AS prediction_clean
  FROM `prj-prod-dataplatform.audit_balance.ml_model_run_details`
  WHERE modelDisplayName = 'Beta - AppsScoreModel')
SELECT
  r.customerId,r.digitalLoanAccountId,prediction,start_time,end_time,
  modelDisplayName,modelVersionId,
     loanmaster.new_loan_type,
 loanmaster.gender,
 'SIL Beta - AppsScoreModel' Model_Name,
 'SIL' as product,
  'NA' trenchCategory,
  safe_cast(JSON_VALUE(prediction_clean, "$.combined_score") AS float64) as sil_beta_app_score,
 calcFeature calcFeatures, 
    coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  r.start_time) AS appln_submit_datetime,
    loanmaster.disbursementDateTime,
    format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  r.start_time)) as Application_month,
 FROM cleaned r
left join risk_credit_mis.loan_master_table loanmaster 
  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  qualify row_number() over(partition by r.customerId, r.digitalLoanAccountid order by start_time desc) = 1
;
"""
dfd = client.query(sq).to_dataframe()
dfd.head()

,customerId,digitalLoanAccountId,prediction,start_time,end_time,modelDisplayName,modelVersionId,new_loan_type,gender,Model_Name,product,trenchCategory,sil_beta_app_score,calcFeatures,appln_submit_datetime,disbursementDateTime,Application_month
0,3745339,24a1c3c8-966d-4652-881e-f4ef0dea8714,"{'cb_score': 0.46689811413887217, 'dl_score': ...",2025-10-15 01:46:51.291327,2025-10-15 01:46:51.781644,Beta - AppsScoreModel,v1,None,None,SIL Beta - AppsScoreModel,SIL,NA,0.561947,"{""app_cnt_rated_for_3plus_ever"":15,""app_cnt_ed...",2025-10-15 01:46:51.291327,NaT,2025-10
1,3745360,39eb2195-8c50-4ba8-b094-a228e81f9886,"{'cb_score': 0.3835346334949697, 'dl_score': 0...",2025-10-15 02:00:25.144805,2025-10-15 02:00:25.640042,Beta - AppsScoreModel,v1,None,None,SIL Beta - AppsScoreModel,SIL,NA,0.514026,"{""app_cnt_rated_for_3plus_ever"":25,""app_cnt_ed...",2025-10-15 02:00:25.144805,NaT,2025-10
2,3745463,3451d3b2-5b57-418a-a7b5-b2a53b7ff4b0,"{'cb_score': 0.4706818163364773, 'dl_score': 0...",2025-10-15 03:01:56.855069,2025-10-15 03:01:57.377352,Beta - AppsScoreModel,v1,None,None,SIL Beta - AppsScoreModel,SIL,NA,0.516166,"{""app_cnt_rated_for_3plus_ever"":20,""app_cnt_ed...",2025-10-15 03:01:56.855069,NaT,2025-10
3,3745828,605c44bd-c549-4e83-ad67-b85faafd0e0a,"{'cb_score': 0.6333466445151616, 'dl_score': 0...",2025-10-15 05:35:00.247710,2025-10-15 05:35:00.779392,Beta - AppsScoreModel,v1,None,None,SIL Beta - AppsScoreModel,SIL,NA,0.546444,"{""app_cnt_rated_for_3plus_ever"":13,""app_cnt_ed...",2025-10-15 05:35:00.247710,NaT,2025-10
4,3746069,fc8b550b-902e-4177-8853-3b549807a66b,"{'cb_score': 0.4625681859149773, 'dl_score': 0...",2025-10-15 07:10:53.359463,2025-10-15 07:10:53.857891,Beta - AppsScoreModel,v1,None,None,SIL Beta - AppsScoreModel,SIL,NA,0.511496,"{""app_cnt_rated_for_3plus_ever"":47,""app_cnt_ed...",2025-10-15 07:10:53.359463,NaT,2025-10


### Expand df

In [98]:
# Expand the calcFeatures column
expanded_df = expand_calc_features(dfd)

# Display the result
print(f"Original columns: {dfd.shape[1]}")
print(f"Expanded columns: {expanded_df.shape[1]}")
expanded_df.head()


Original columns: 17
Expanded columns: 35


,customerId,digitalLoanAccountId,prediction,start_time,end_time,modelDisplayName,modelVersionId,new_loan_type,gender,Model_Name,product,trenchCategory,sil_beta_app_score,calcFeatures,appln_submit_datetime,disbursementDateTime,Application_month,calc_app_cnt_rated_for_3plus_ever,calc_app_cnt_education_ever,calc_app_cnt_business_ever,calc_app_cnt_music_and_audio_ever,calc_app_cnt_travel_and_local_ever,calc_app_cnt_finance_7d,calc_app_cnt_finance_30d,calc_app_cnt_competitors_30d,calc_app_cnt_absence_tag_30d,calc_app_cnt_absence_tag_90d,calc_app_cnt_finance_90d,calc_app_cnt_competitors_90d,calc_app_cnt_payday_90d,calc_app_avg_time_bw_installed_mins_30d,calc_app_median_time_bw_installed_mins_30d,calc_app_first_competitors_install_to_apply_days,calc_app_first_payday_install_to_apply_days,calc_app_vel_finance_30_over_365
0,3745339,24a1c3c8-966d-4652-881e-f4ef0dea8714,"{'cb_score': 0.46689811413887217, 'dl_score': ...",2025-10-15 01:46:51.291327,2025-10-15 01:46:51.781644,Beta - AppsScoreModel,v1,None,None,SIL Beta - AppsScoreModel,SIL,NA,0.561947,"{""app_cnt_rated_for_3plus_ever"":15,""app_cnt_ed...",2025-10-15 01:46:51.291327,NaT,2025-10,15,0,0,1,0,0,0,0,1,2,0,1,0,7462.016667,6077.283333,346.5,346.5,0.0
1,3745360,39eb2195-8c50-4ba8-b094-a228e81f9886,"{'cb_score': 0.3835346334949697, 'dl_score': 0...",2025-10-15 02:00:25.144805,2025-10-15 02:00:25.640042,Beta - AppsScoreModel,v1,None,None,SIL Beta - AppsScoreModel,SIL,NA,0.514026,"{""app_cnt_rated_for_3plus_ever"":25,""app_cnt_ed...",2025-10-15 02:00:25.144805,NaT,2025-10,25,0,1,0,1,0,0,0,0,0,0,0,0,7951.180000,8317.750000,459.3,459.3,0.0
2,3745463,3451d3b2-5b57-418a-a7b5-b2a53b7ff4b0,"{'cb_score': 0.4706818163364773, 'dl_score': 0...",2025-10-15 03:01:56.855069,2025-10-15 03:01:57.377352,Beta - AppsScoreModel,v1,None,None,SIL Beta - AppsScoreModel,SIL,NA,0.516166,"{""app_cnt_rated_for_3plus_ever"":20,""app_cnt_ed...",2025-10-15 03:01:56.855069,NaT,2025-10,20,0,0,0,0,0,0,0,8,9,0,0,0,1557.974000,390.433333,707.6,707.6,0.0
3,3745828,605c44bd-c549-4e83-ad67-b85faafd0e0a,"{'cb_score': 0.6333466445151616, 'dl_score': 0...",2025-10-15 05:35:00.247710,2025-10-15 05:35:00.779392,Beta - AppsScoreModel,v1,None,None,SIL Beta - AppsScoreModel,SIL,NA,0.546444,"{""app_cnt_rated_for_3plus_ever"":13,""app_cnt_ed...",2025-10-15 05:35:00.247710,NaT,2025-10,13,0,1,1,2,0,1,1,3,3,1,1,1,3732.759524,130.150000,16.6,16.6,1.0
4,3746069,fc8b550b-902e-4177-8853-3b549807a66b,"{'cb_score': 0.4625681859149773, 'dl_score': 0...",2025-10-15 07:10:53.359463,2025-10-15 07:10:53.857891,Beta - AppsScoreModel,v1,None,None,SIL Beta - AppsScoreModel,SIL,NA,0.511496,"{""app_cnt_rated_for_3plus_ever"":47,""app_cnt_ed...",2025-10-15 07:10:53.359463,NaT,2025-10,47,1,4,0,1,0,0,0,3,5,0,0,0,3390.503030,1346.366667,434.7,434.7,0.0


In [99]:
df = expanded_df.drop(columns=['calcFeatures']).copy()
df = df.drop_duplicates(keep='first')

In [100]:
df['sil_beta_app_score'] = pd.to_numeric(df['sil_beta_app_score'], errors='coerce')


### run psi calculation

In [101]:
df = df.copy()

# Define feature list
feature_list = ['sil_beta_app_score',
       'calc_app_cnt_rated_for_3plus_ever', 'calc_app_cnt_education_ever',
       'calc_app_cnt_business_ever', 'calc_app_cnt_music_and_audio_ever',
       'calc_app_cnt_travel_and_local_ever', 'calc_app_cnt_finance_7d',
       'calc_app_cnt_absence_tag_30d', 'calc_app_cnt_competitors_30d',
       'calc_app_cnt_finance_30d', 'calc_app_cnt_absence_tag_90d',
       'calc_app_cnt_finance_90d', 'calc_app_cnt_competitors_90d',
       'calc_app_cnt_payday_90d', 'calc_app_avg_time_bw_installed_mins_30d',
       'calc_app_median_time_bw_installed_mins_30d',
       'calc_app_first_competitors_install_to_apply_days',
       'calc_app_first_payday_install_to_apply_days',
       'calc_app_vel_finance_30_over_365']

# Define segment columns
segment_columns = ['new_loan_type', 'gender', 'osType', 'trenchCategory', 'product']
# Calculate month-on-month PSI
psi_results = calculate_month_on_month_psi(df, feature_list, segment_columns)
psi_results['modelDisplayName'] = df['modelDisplayName'].iloc[0]
psi_results['Model_Name'] = df['Model_Name'].iloc[0]
psi_results['modelVersionId'] = df['modelVersionId'].iloc[0]
psi_results['trenchCategory'] = df['trenchCategory'].iloc[0]
psi_results = psi_results[[ 'modelDisplayName', 'Model_Name',   'modelVersionId', 'trenchCategory','Feature', 'Feature_Type', 'Segment_Column', 'Segment_Value', 'Month',
       'Base_Month', 'Current_Month', 'PSI']].copy()

# Calculate bin-level PSI
bin_psi_results = calculate_bin_level_psi(df, feature_list, segment_columns)
bin_psi_results['modelDisplayName'] = df['modelDisplayName'].iloc[0]
bin_psi_results['Model_Name'] = df['Model_Name'].iloc[0]
bin_psi_results['modelVersionId'] = df['modelVersionId'].iloc[0]
bin_psi_results['trenchCategory'] = df['trenchCategory'].iloc[0]
bin_psi_results = bin_psi_results[[ 'modelDisplayName', 'Model_Name',  'modelVersionId', 'trenchCategory','Feature', 'Feature_Type', 'Segment_Column', 'Segment_Value', 'Month',
       'Base_Month', 'Current_Month', 'Bin', 'Bin_Range', 'Bin_Min', 'Bin_Max', 'Base_Percentage',
       'Actual_Percentage', 'Bin_PSI']].copy()

# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.temp_csi_new_monitoring_data"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(psi_results, table_id, job_config=job_config)
job.result()  # Wait for the job to complete


# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.temp_csi_new_monitoring_data_feature_bin_level"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(bin_psi_results, table_id, job_config=job_config)
job.result()  # Wait for the job to complete

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=8914146d-ce3e-494a-96da-fbad6594b508>

## SIL Beta - DemoScoreModel

In [102]:
sq = """ WITH cleaned AS (
  SELECT
  customerId,digitalLoanAccountId,prediction,start_time,end_time,modelDisplayName,modelVersionId,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeature_cleaned
  FROM `prj-prod-dataplatform.audit_balance.ml_model_run_details`
  WHERE modelDisplayName = 'Beta - DemoScoreModel')
SELECT
  r.customerId,  r.digitalLoanAccountId,start_time, prediction sil_beta_demo_score, modelDisplayName,modelVersionId,
      loanmaster.new_loan_type,
 loanmaster.gender,
 'SIL Beta - DemoScoreModel' Model_Name,
 'SIL' as product,
 'NA' trenchCategory,
  r.calcFeature_cleaned calcFeatures,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  r.start_time) AS appln_submit_datetime,
  loanmaster.disbursementDateTime,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  r.start_time)) as Application_month,
FROM cleaned r
left join risk_credit_mis.loan_master_table loanmaster 
  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
qualify row_number() over(partition by r.customerId, r.digitalLoanAccountId order by start_time desc) = 1;"""

dfd = client.query(sq).to_dataframe()
dfd.head()

,customerId,digitalLoanAccountId,start_time,sil_beta_demo_score,modelDisplayName,modelVersionId,new_loan_type,gender,Model_Name,product,trenchCategory,calcFeatures,appln_submit_datetime,disbursementDateTime,Application_month
0,3745651,69f7d195-762c-4bf4-bf1b-50a58c308326,2025-10-15 04:22:24.716771,0.055928969,Beta - DemoScoreModel,v1,None,None,SIL Beta - DemoScoreModel,SIL,NA,"{""beta_de_ln_vas_opted_flag"": ""0"", ""beta_de_ln...",2025-10-15 04:22:24.716771,NaT,2025-10
1,3745956,f3d6da3a-3f20-44a7-b3e3-72c26be5d5c0,2025-10-15 06:22:28.782144,0.0463957383,Beta - DemoScoreModel,v1,None,None,SIL Beta - DemoScoreModel,SIL,NA,"{""beta_de_ln_vas_opted_flag"": ""0"", ""beta_de_ln...",2025-10-15 06:22:28.782144,NaT,2025-10
2,3745475,32fd7e70-f7ae-45a3-ad94-ededea5b9586,2025-10-15 02:58:07.065415,0.11300935,Beta - DemoScoreModel,v1,None,None,SIL Beta - DemoScoreModel,SIL,NA,"{""beta_de_ln_vas_opted_flag"": ""0"", ""beta_de_ln...",2025-10-15 02:58:07.065415,NaT,2025-10
3,3745751,f448c91e-667e-43b8-9cdd-8e628253dcbe,2025-10-15 05:19:44.459218,0.0469868285,Beta - DemoScoreModel,v1,None,None,SIL Beta - DemoScoreModel,SIL,NA,"{""beta_de_ln_vas_opted_flag"": ""0"", ""beta_de_ln...",2025-10-15 05:19:44.459218,NaT,2025-10
4,3745978,a0df9aa2-3a8a-4df0-9b8f-cf0581146990,2025-10-15 06:33:22.408776,0.0793052211,Beta - DemoScoreModel,v1,None,None,SIL Beta - DemoScoreModel,SIL,NA,"{""beta_de_ln_vas_opted_flag"": ""1"", ""beta_de_ln...",2025-10-15 06:33:22.408776,NaT,2025-10


### Expand df

In [103]:
# Expand the calcFeatures column
expanded_df = expand_calc_features(dfd)

# Display the result
print(f"Original columns: {dfd.shape[1]}")
print(f"Expanded columns: {expanded_df.shape[1]}")
expanded_df.head()


Original columns: 15
Expanded columns: 32


,customerId,digitalLoanAccountId,start_time,sil_beta_demo_score,modelDisplayName,modelVersionId,new_loan_type,gender,Model_Name,product,trenchCategory,calcFeatures,appln_submit_datetime,disbursementDateTime,Application_month,calc_beta_de_ln_vas_opted_flag,calc_beta_de_ln_doc_type_rolled,calc_beta_de_ln_marital_status,calc_beta_de_ln_age_bin,calc_beta_de_ln_province_bin,calc_beta_de_ln_ref2_type,calc_beta_de_ln_education_level,calc_beta_de_ln_ref1_type,calc_beta_de_ln_industry_new_bin,calc_beta_de_ln_appln_day_of_week,calc_beta_de_onb_name_email_match_score,calc_beta_de_ln_employment_type_new_bin,calc_beta_de_ln_telconame,calc_beta_de_time_bw_onb_loan_appln_mins,calc_beta_de_ln_source_of_funds_new_bin,calc_beta_de_ln_brand_bin,calc_beta_de_ln_email_primary_domain
0,3745651,69f7d195-762c-4bf4-bf1b-50a58c308326,2025-10-15 04:22:24.716771,0.055928969,Beta - DemoScoreModel,v1,None,None,SIL Beta - DemoScoreModel,SIL,NA,"{""beta_de_ln_vas_opted_flag"": ""0"", ""beta_de_ln...",2025-10-15 04:22:24.716771,NaT,2025-10,0,National ID,Married,"(32.0, 38.0]",rizal,Parent,College Undergraduate,Spouse,2,Wednesday,100,Employed - Private Employee,DITO,16.0,Salary,apple,yahoo.com
1,3745956,f3d6da3a-3f20-44a7-b3e3-72c26be5d5c0,2025-10-15 06:22:28.782144,0.0463957383,Beta - DemoScoreModel,v1,None,None,SIL Beta - DemoScoreModel,SIL,NA,"{""beta_de_ln_vas_opted_flag"": ""0"", ""beta_de_ln...",2025-10-15 06:22:28.782144,NaT,2025-10,0,Driving License,Married,"(38.0, 46.0]",metro manila,Child,High School Graduate,Friend,3,Wednesday,33,Self-Employed/Private Practice,Smart,11.0,Income from Business,realme,gmail.com
2,3745475,32fd7e70-f7ae-45a3-ad94-ededea5b9586,2025-10-15 02:58:07.065415,0.11300935,Beta - DemoScoreModel,v1,None,None,SIL Beta - DemoScoreModel,SIL,NA,"{""beta_de_ln_vas_opted_flag"": ""0"", ""beta_de_ln...",2025-10-15 02:58:07.065415,NaT,2025-10,0,UMID Card,Single,"(32.0, 38.0]",metro manila,Parent,High School Graduate,Friend,nan,Wednesday,86,Remittance Beneficiary,Globe,7.0,Remittance,oppo,gmail.com
3,3745751,f448c91e-667e-43b8-9cdd-8e628253dcbe,2025-10-15 05:19:44.459218,0.0469868285,Beta - DemoScoreModel,v1,None,None,SIL Beta - DemoScoreModel,SIL,NA,"{""beta_de_ln_vas_opted_flag"": ""0"", ""beta_de_ln...",2025-10-15 05:19:44.459218,NaT,2025-10,0,National ID,Married,"(27.0, 32.0]",others,Sibling,College Graduate,Sibling,3,Wednesday,100,Overseas Filipino Worker,DITO,32.0,Salary,samsung,gmail.com
4,3745978,a0df9aa2-3a8a-4df0-9b8f-cf0581146990,2025-10-15 06:33:22.408776,0.0793052211,Beta - DemoScoreModel,v1,None,None,SIL Beta - DemoScoreModel,SIL,NA,"{""beta_de_ln_vas_opted_flag"": ""1"", ""beta_de_ln...",2025-10-15 06:33:22.408776,NaT,2025-10,1,National ID,Single,"(27.0, 32.0]",laguna,Parent,High School Graduate,Spouse,2,Wednesday,100,New,Smart,8.0,Income from Business,redmi,gmail.com


In [104]:
df = expanded_df.drop(columns=['calcFeatures']).copy()
df = df.drop_duplicates(keep='first')

In [105]:
df['sil_beta_demo_score'] = pd.to_numeric(df['sil_beta_demo_score'], errors='coerce')


### run psi calculation

In [106]:
df = df.copy()

# Define feature list
feature_list = ['sil_beta_demo_score',
       'calc_beta_de_ln_vas_opted_flag',
       'calc_beta_de_ln_doc_type_rolled', 'calc_beta_de_ln_marital_status',
       'calc_beta_de_ln_age_bin', 'calc_beta_de_ln_province_bin',
       'calc_beta_de_ln_ref2_type', 'calc_beta_de_ln_education_level',
       'calc_beta_de_ln_ref1_type', 'calc_beta_de_ln_industry_new_bin',
       'calc_beta_de_ln_appln_day_of_week',
       'calc_beta_de_onb_name_email_match_score',
       'calc_beta_de_ln_employment_type_new_bin', 'calc_beta_de_ln_telconame',
       'calc_beta_de_time_bw_onb_loan_appln_mins',
       'calc_beta_de_ln_source_of_funds_new_bin', 'calc_beta_de_ln_brand_bin',
       'calc_beta_de_ln_email_primary_domain']

# Define segment columns
segment_columns = ['new_loan_type', 'gender', 'osType', 'trenchCategory', 'product']
# Calculate month-on-month PSI
psi_results = calculate_month_on_month_psi(df, feature_list, segment_columns)
psi_results['modelDisplayName'] = df['modelDisplayName'].iloc[0]
psi_results['Model_Name'] = df['Model_Name'].iloc[0]
psi_results['modelVersionId'] = df['modelVersionId'].iloc[0]
psi_results['trenchCategory'] = df['trenchCategory'].iloc[0]
psi_results = psi_results[[ 'modelDisplayName', 'Model_Name',   'modelVersionId', 'trenchCategory','Feature', 'Feature_Type', 'Segment_Column', 'Segment_Value', 'Month',
       'Base_Month', 'Current_Month', 'PSI']].copy()

# Calculate bin-level PSI
bin_psi_results = calculate_bin_level_psi(df, feature_list, segment_columns)
bin_psi_results['modelDisplayName'] = df['modelDisplayName'].iloc[0]
bin_psi_results['Model_Name'] = df['Model_Name'].iloc[0]
bin_psi_results['modelVersionId'] = df['modelVersionId'].iloc[0]
bin_psi_results['trenchCategory'] = df['trenchCategory'].iloc[0]
bin_psi_results = bin_psi_results[[ 'modelDisplayName', 'Model_Name',  'modelVersionId', 'trenchCategory','Feature', 'Feature_Type', 'Segment_Column', 'Segment_Value', 'Month',
       'Base_Month', 'Current_Month', 'Bin', 'Bin_Range', 'Bin_Min', 'Bin_Max', 'Base_Percentage',
       'Actual_Percentage', 'Bin_PSI']].copy()

# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.temp_csi_new_monitoring_data"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(psi_results, table_id, job_config=job_config)
job.result()  # Wait for the job to complete


# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.temp_csi_new_monitoring_data_feature_bin_level"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(bin_psi_results, table_id, job_config=job_config)
job.result()  # Wait for the job to complete

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=96aa43dc-1143-42b2-b42f-8b83da1a6c2a>

In [107]:
psi_results

,modelDisplayName,Model_Name,modelVersionId,trenchCategory,Feature,Feature_Type,Segment_Column,Segment_Value,Month,Base_Month,Current_Month,PSI
0,Beta - DemoScoreModel,SIL Beta - DemoScoreModel,v1,NA,sil_beta_demo_score,numerical,Overall,All,2025-03,2025-03,2025-03,0.000000
1,Beta - DemoScoreModel,SIL Beta - DemoScoreModel,v1,NA,sil_beta_demo_score,numerical,Overall,All,2025-04,2025-03,2025-04,0.103751
2,Beta - DemoScoreModel,SIL Beta - DemoScoreModel,v1,NA,sil_beta_demo_score,numerical,Overall,All,2025-05,2025-03,2025-05,0.083315
3,Beta - DemoScoreModel,SIL Beta - DemoScoreModel,v1,NA,sil_beta_demo_score,numerical,Overall,All,2025-06,2025-03,2025-06,0.523703
4,Beta - DemoScoreModel,SIL Beta - DemoScoreModel,v1,NA,sil_beta_demo_score,numerical,Overall,All,2025-07,2025-03,2025-07,0.612188
...,...,...,...,...,...,...,...,...,...,...,...,...
1003,Beta - DemoScoreModel,SIL Beta - DemoScoreModel,v1,NA,calc_beta_de_ln_email_primary_domain,categorical,product,SIL,2025-06,2025-03,2025-06,0.017841
1004,Beta - DemoScoreModel,SIL Beta - DemoScoreModel,v1,NA,calc_beta_de_ln_email_primary_domain,categorical,product,SIL,2025-07,2025-03,2025-07,0.018347
1005,Beta - DemoScoreModel,SIL Beta - DemoScoreModel,v1,NA,calc_beta_de_ln_email_primary_domain,categorical,product,SIL,2025-08,2025-03,2025-08,0.013797
1006,Beta - DemoScoreModel,SIL Beta - DemoScoreModel,v1,NA,calc_beta_de_ln_email_primary_domain,categorical,product,SIL,2025-09,2025-03,2025-09,0.014347


## SIL_Beta - StackScoreModel

In [108]:
sq = """ WITH cleaned AS (
  SELECT
    customerId,digitalLoanAccountId,prediction,start_time,end_time,modelDisplayName,modelVersionId,
    REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeature
  FROM `prj-prod-dataplatform.audit_balance.ml_model_run_details`
  WHERE modelDisplayName = 'Beta - StackScoreModel')
SELECT
  r.customerId,r.digitalLoanAccountId,prediction,start_time,end_time,modelDisplayName,modelVersionId,
 loanmaster.new_loan_type,
 loanmaster.gender,
 'SIL_Beta - StackScoreModel' Model_Name,
 'SIL' as product,
 'NA' trenchCategory,
  r.calcFeature calcFeatures,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  r.start_time) AS appln_submit_datetime,
  loanmaster.disbursementDateTime,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  r.start_time)) as Application_month,  
FROM cleaned r 
left join risk_credit_mis.loan_master_table loanmaster 
  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
qualify row_number() over (partition by r.customerId, r.digitalLoanAccountId order by start_time desc) = 1
;
"""
dfd= client.query(sq).to_dataframe()
dfd.head()


,customerId,digitalLoanAccountId,prediction,start_time,end_time,modelDisplayName,modelVersionId,new_loan_type,gender,Model_Name,product,trenchCategory,calcFeatures,appln_submit_datetime,disbursementDateTime,Application_month
0,3745349,a45d3ae3-4122-4763-ac40-81038bef8688,0.07187971544312756,2025-10-15 01:55:14.875356,2025-10-15 01:55:14.879744,Beta - StackScoreModel,v1,None,None,SIL_Beta - StackScoreModel,SIL,NA,"{""s_apps_score"": 0.519023091596331, ""s_credo_s...",2025-10-15 01:55:14.875356,NaT,2025-10
1,3745372,e1721406-22e2-4a8f-8475-61f0cc9f8f0e,0.08451742707808087,2025-10-15 02:07:41.799703,2025-10-15 02:07:41.804031,Beta - StackScoreModel,v1,None,None,SIL_Beta - StackScoreModel,SIL,NA,"{""s_apps_score"": 0.534947994566677, ""s_credo_s...",2025-10-15 02:07:41.799703,NaT,2025-10
2,3745415,fd38e2cf-7544-4e35-a2e6-bdd569f9a6e7,0.21844435463792397,2025-10-15 02:34:47.602987,2025-10-15 02:34:47.607057,Beta - StackScoreModel,v1,None,None,SIL_Beta - StackScoreModel,SIL,NA,"{""s_apps_score"": 0.5401976345319875, ""s_credo_...",2025-10-15 02:34:47.602987,NaT,2025-10
3,3746091,a066915f-fd86-4a8e-acc4-ad75293e28c1,0.225218798757672,2025-10-15 07:19:56.751477,2025-10-15 07:19:56.755751,Beta - StackScoreModel,v1,None,None,SIL_Beta - StackScoreModel,SIL,NA,"{""s_apps_score"": 0.6588851612192854, ""s_credo_...",2025-10-15 07:19:56.751477,NaT,2025-10
4,3746129,02297789-a7b6-4270-8aaa-b8e1cbd5367e,0.14252038078722618,2025-10-15 07:26:49.606389,2025-10-15 07:26:49.611004,Beta - StackScoreModel,v1,None,None,SIL_Beta - StackScoreModel,SIL,NA,"{""s_apps_score"": 0.5643299771903714, ""s_credo_...",2025-10-15 07:26:49.606389,NaT,2025-10


### Expand df

In [109]:
# Expand the calcFeatures column
expanded_df = expand_calc_features(dfd)

# Display the result
print(f"Original columns: {dfd.shape[1]}")
print(f"Expanded columns: {expanded_df.shape[1]}")
expanded_df.head()


Original columns: 16
Expanded columns: 19


,customerId,digitalLoanAccountId,prediction,start_time,end_time,modelDisplayName,modelVersionId,new_loan_type,gender,Model_Name,product,trenchCategory,calcFeatures,appln_submit_datetime,disbursementDateTime,Application_month,calc_s_apps_score,calc_s_credo_score,calc_sb_demo_score
0,3745349,a45d3ae3-4122-4763-ac40-81038bef8688,0.07187971544312756,2025-10-15 01:55:14.875356,2025-10-15 01:55:14.879744,Beta - StackScoreModel,v1,None,None,SIL_Beta - StackScoreModel,SIL,NA,"{""s_apps_score"": 0.519023091596331, ""s_credo_s...",2025-10-15 01:55:14.875356,NaT,2025-10,0.519023,0.1273,0.072608
1,3745372,e1721406-22e2-4a8f-8475-61f0cc9f8f0e,0.08451742707808087,2025-10-15 02:07:41.799703,2025-10-15 02:07:41.804031,Beta - StackScoreModel,v1,None,None,SIL_Beta - StackScoreModel,SIL,NA,"{""s_apps_score"": 0.534947994566677, ""s_credo_s...",2025-10-15 02:07:41.799703,NaT,2025-10,0.534948,0.1626,0.062735
2,3745415,fd38e2cf-7544-4e35-a2e6-bdd569f9a6e7,0.21844435463792397,2025-10-15 02:34:47.602987,2025-10-15 02:34:47.607057,Beta - StackScoreModel,v1,None,None,SIL_Beta - StackScoreModel,SIL,NA,"{""s_apps_score"": 0.5401976345319875, ""s_credo_...",2025-10-15 02:34:47.602987,NaT,2025-10,0.540198,0.1612,0.199881
3,3746091,a066915f-fd86-4a8e-acc4-ad75293e28c1,0.225218798757672,2025-10-15 07:19:56.751477,2025-10-15 07:19:56.755751,Beta - StackScoreModel,v1,None,None,SIL_Beta - StackScoreModel,SIL,NA,"{""s_apps_score"": 0.6588851612192854, ""s_credo_...",2025-10-15 07:19:56.751477,NaT,2025-10,0.658885,0.3656,0.070423
4,3746129,02297789-a7b6-4270-8aaa-b8e1cbd5367e,0.14252038078722618,2025-10-15 07:26:49.606389,2025-10-15 07:26:49.611004,Beta - StackScoreModel,v1,None,None,SIL_Beta - StackScoreModel,SIL,NA,"{""s_apps_score"": 0.5643299771903714, ""s_credo_...",2025-10-15 07:26:49.606389,NaT,2025-10,0.564330,0.1353,0.109921


In [110]:
df = expanded_df.drop(columns=['calcFeatures']).copy()
df.rename(columns={'prediction': 'sil_beta_stack_score'}, inplace=True)
df = df.drop_duplicates(keep='first')

In [111]:
df['sil_beta_stack_score'] = pd.to_numeric(df['sil_beta_stack_score'], errors='coerce')


### run psi calculation

In [112]:
df.columns

Index(['customerId', 'digitalLoanAccountId', 'sil_beta_stack_score',
       'start_time', 'end_time', 'modelDisplayName', 'modelVersionId',
       'new_loan_type', 'gender', 'Model_Name', 'product', 'trenchCategory',
       'appln_submit_datetime', 'disbursementDateTime', 'Application_month',
       'calc_s_apps_score', 'calc_s_credo_score', 'calc_sb_demo_score'],
      dtype='object')

In [113]:
df = df.copy()

# Define feature list
feature_list = ['sil_beta_stack_score',
        'calc_s_apps_score', 'calc_s_credo_score', 'calc_sb_demo_score']

# Define segment columns
segment_columns = ['new_loan_type', 'gender', 'osType', 'trenchCategory', 'product']
# Calculate month-on-month PSI
psi_results = calculate_month_on_month_psi(df, feature_list, segment_columns)
psi_results['modelDisplayName'] = df['modelDisplayName'].iloc[0]
psi_results['Model_Name'] = df['Model_Name'].iloc[0]
psi_results['modelVersionId'] = df['modelVersionId'].iloc[0]
psi_results['trenchCategory'] = df['trenchCategory'].iloc[0]
psi_results = psi_results[[ 'modelDisplayName', 'Model_Name',   'modelVersionId', 'trenchCategory','Feature', 'Feature_Type', 'Segment_Column', 'Segment_Value', 'Month',
       'Base_Month', 'Current_Month', 'PSI']].copy()

# Calculate bin-level PSI
bin_psi_results = calculate_bin_level_psi(df, feature_list, segment_columns)
bin_psi_results['modelDisplayName'] = df['modelDisplayName'].iloc[0]
bin_psi_results['Model_Name'] = df['Model_Name'].iloc[0]
bin_psi_results['modelVersionId'] = df['modelVersionId'].iloc[0]
bin_psi_results['trenchCategory'] = df['trenchCategory'].iloc[0]
bin_psi_results = bin_psi_results[[ 'modelDisplayName', 'Model_Name',  'modelVersionId', 'trenchCategory','Feature', 'Feature_Type', 'Segment_Column', 'Segment_Value', 'Month',
       'Base_Month', 'Current_Month', 'Bin', 'Bin_Range', 'Bin_Min', 'Bin_Max', 'Base_Percentage',
       'Actual_Percentage', 'Bin_PSI']].copy()

# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.temp_csi_new_monitoring_data"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(psi_results, table_id, job_config=job_config)
job.result()  # Wait for the job to complete


# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.temp_csi_new_monitoring_data_feature_bin_level"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(bin_psi_results, table_id, job_config=job_config)
job.result()  # Wait for the job to complete

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=cdbb1de8-4530-4c83-8cdd-357445ff6abe>

In [114]:
psi_results

,modelDisplayName,Model_Name,modelVersionId,trenchCategory,Feature,Feature_Type,Segment_Column,Segment_Value,Month,Base_Month,Current_Month,PSI
0,Beta - StackScoreModel,SIL_Beta - StackScoreModel,v1,NA,sil_beta_stack_score,numerical,Overall,All,2025-03,2025-03,2025-03,0.000000
1,Beta - StackScoreModel,SIL_Beta - StackScoreModel,v1,NA,sil_beta_stack_score,numerical,Overall,All,2025-04,2025-03,2025-04,0.266647
2,Beta - StackScoreModel,SIL_Beta - StackScoreModel,v1,NA,sil_beta_stack_score,numerical,Overall,All,2025-05,2025-03,2025-05,0.295325
3,Beta - StackScoreModel,SIL_Beta - StackScoreModel,v1,NA,sil_beta_stack_score,numerical,Overall,All,2025-06,2025-03,2025-06,0.085729
4,Beta - StackScoreModel,SIL_Beta - StackScoreModel,v1,NA,sil_beta_stack_score,numerical,Overall,All,2025-07,2025-03,2025-07,0.084792
...,...,...,...,...,...,...,...,...,...,...,...,...
219,Beta - StackScoreModel,SIL_Beta - StackScoreModel,v1,NA,calc_sb_demo_score,numerical,product,SIL,2025-06,2025-03,2025-06,0.523703
220,Beta - StackScoreModel,SIL_Beta - StackScoreModel,v1,NA,calc_sb_demo_score,numerical,product,SIL,2025-07,2025-03,2025-07,0.612188
221,Beta - StackScoreModel,SIL_Beta - StackScoreModel,v1,NA,calc_sb_demo_score,numerical,product,SIL,2025-08,2025-03,2025-08,0.605235
222,Beta - StackScoreModel,SIL_Beta - StackScoreModel,v1,NA,calc_sb_demo_score,numerical,product,SIL,2025-09,2025-03,2025-09,0.625418


## Beta - IncomeEstimationModel

In [115]:
sq = """ 
WITH cleaned AS (
  SELECT
    customerId,digitalLoanAccountId,prediction,start_time,end_time,modelDisplayName,modelVersionId,

    REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeature
  FROM `prj-prod-dataplatform.audit_balance.ml_model_run_details`
  WHERE modelDisplayName = 'Beta - IncomeEstimationModel')
SELECT
  r.customerId,r.digitalLoanAccountId,prediction sil_beta_income_estimation_score,start_time,end_time,modelDisplayName,modelVersionId,
  JSON_VALUE(calcFeature, "$.inc_beta_ln_loan_type") AS inc_beta_ln_loan_type,
  JSON_VALUE(calcFeature, "$.inc_beta_ln_education_level") AS inc_beta_ln_education_level,
  JSON_VALUE(calcFeature, "$.inc_beta_ln_employment_type_new") AS inc_beta_ln_employment_type_new,
  JSON_VALUE(calcFeature, "$.inc_beta_ln_industry_new") AS inc_beta_ln_industry_new,
  JSON_VALUE(calcFeature, "$.inc_beta_ln_age") AS inc_beta_ln_age,
  JSON_VALUE(calcFeature, "$.inc_beta_ln_brand") AS inc_beta_ln_brand,
  JSON_VALUE(calcFeature, "$.inc_beta_ln_city") AS inc_beta_ln_city,
  JSON_VALUE(calcFeature, "$.inc_beta_ln_purpose") AS inc_beta_ln_purpose,
  JSON_VALUE(calcFeature, "$.inc_beta_ln_osversion_bin") AS inc_beta_ln_osversion_bin,
  JSON_VALUE(calcFeature, "$.inc_beta_ln_postal_code") AS inc_beta_ln_postal_code,
  JSON_VALUE(calcFeature, "$.inc_beta_ln_gender") AS inc_beta_ln_gender,
  JSON_VALUE(calcFeature, "$.inc_beta_ln_doc_type_rolled") AS inc_beta_ln_doc_type_rolled,
  JSON_VALUE(calcFeature, "$.inc_beta_ln_cnt_dependents") AS inc_beta_ln_cnt_dependents,
  JSON_VALUE(calcFeature, "$.inc_beta_ln_source_of_funds_new") AS inc_beta_ln_source_of_funds_new,
  JSON_VALUE(calcFeature, "$.inc_beta_ln_marital_status_new") AS inc_beta_ln_marital_status_new,
  JSON_VALUE(calcFeature, "$.inc_beta_encoded_company_name_grouped") AS inc_beta_encoded_company_name_grouped,
   loanmaster.new_loan_type,
 loanmaster.gender,
 'SIL Beta - IncomeEstimationModel' Model_Name,
 'SIL' as product,
 'NA' trenchCategory,
  r.calcFeature calcFeatures,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  r.start_time) AS appln_submit_datetime,
  loanmaster.disbursementDateTime,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  r.start_time)) as Application_month,  
FROM cleaned r 
left join risk_credit_mis.loan_master_table loanmaster 
  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
qualify row_number() over (partition by r.customerId, r.digitalLoanAccountId order by start_time desc) = 1
  ;
"""
dfd = client.query(sq).to_dataframe()
dfd.head()

,customerId,digitalLoanAccountId,sil_beta_income_estimation_score,start_time,end_time,modelDisplayName,modelVersionId,inc_beta_ln_loan_type,inc_beta_ln_education_level,inc_beta_ln_employment_type_new,inc_beta_ln_industry_new,inc_beta_ln_age,inc_beta_ln_brand,inc_beta_ln_city,inc_beta_ln_purpose,inc_beta_ln_osversion_bin,inc_beta_ln_postal_code,inc_beta_ln_gender,inc_beta_ln_doc_type_rolled,inc_beta_ln_cnt_dependents,inc_beta_ln_source_of_funds_new,inc_beta_ln_marital_status_new,inc_beta_encoded_company_name_grouped,new_loan_type,gender,Model_Name,product,trenchCategory,calcFeatures,appln_submit_datetime,disbursementDateTime,Application_month
0,2445354,28328f4c-cc58-4fe5-b8da-c1e208fc9cdb,33275.18861534607,2025-04-01 09:17:07.432853,2025-04-01 09:17:07.496704,Beta - IncomeEstimationModel,v1,BNPL,College Graduate,Employed,Doctor/Dentist/Medical Professional,31,samsung,CF0063,"Washing machine, dryer or both",2-Voyager,1870,F,Other,0,1,Single,93731.2656120701,SIL-Instore,F,SIL Beta - IncomeEstimationModel,SIL,NA,"{""inc_beta_ln_loan_type"":""BNPL"",""inc_beta_ln_e...",2025-04-01 17:17:04,NaT,2025-04
1,2501206,24261d70-6039-4fe0-9315-ae63c615e2eb,8430.189771057514,2025-08-15 03:07:23.801484,2025-08-15 03:07:23.859948,Beta - IncomeEstimationModel,v1,SILCOM,Technical/Vocational Graduate,Employed - Private Employee,Architecture/Engineering,40,TECNO,CF1682,Refrigerators,5-Pioneer,1776,M,ID Card,0,Salary,Single,0.0,SIL Competitor,M,SIL Beta - IncomeEstimationModel,SIL,NA,"{""inc_beta_ln_loan_type"":""SILCOM"",""inc_beta_ln...",2025-08-15 11:07:20,NaT,2025-08
2,2571988,2fddd759-74ef-44aa-8daa-dd640b8745ac,8692.120754263524,2025-04-16 02:50:20.735779,2025-04-16 02:50:20.794555,Beta - IncomeEstimationModel,v1,BNPL,College Undergraduate,Business Owner,Business Owner (MSME),41,TECNO,CF0579,Air conditioners,2-Voyager,4103,M,Other,3,3,Married,0.0,SIL-Instore,M,SIL Beta - IncomeEstimationModel,SIL,NA,"{""inc_beta_ln_loan_type"":""BNPL"",""inc_beta_ln_e...",2025-04-16 10:50:17,NaT,2025-04
3,2659855,733dd15d-3de6-4f5a-9042-516952662e3d,8523.899774803616,2025-08-19 04:09:34.771898,2025-08-19 04:09:34.840998,Beta - IncomeEstimationModel,v1,SILCOM,College Graduate,Employed - Private Employee,Maritime/Shipping,31,Apple,CF1367,Air conditioners,5-Pioneer,4232,F,UMID Card,0,Salary,Single,0.0,SIL Competitor,F,SIL Beta - IncomeEstimationModel,SIL,NA,"{""inc_beta_ln_loan_type"":""SILCOM"",""inc_beta_ln...",2025-08-19 12:09:32,2025-08-19 12:11:56,2025-08
4,2790668,8ab7a0f0-91c1-4456-b25d-32053c6dce9e,9084.45501044467,2025-04-20 09:46:28.118712,2025-04-20 09:46:28.173127,Beta - IncomeEstimationModel,v1,BNPL0.1,High School Graduate,Employed,Other Professional services,56,vivo,CF1185,Air conditioners,2-Voyager,3023,M,Other,1,1,Married,0.0,SIL ZERO,M,SIL Beta - IncomeEstimationModel,SIL,NA,"{""inc_beta_ln_loan_type"":""BNPL0.1"",""inc_beta_l...",2025-04-20 17:46:24,NaT,2025-04


### Expand df


In [116]:
# Expand the calcFeatures column
expanded_df = dfd.copy()

# Display the result
print(f"Original columns: {dfd.shape[1]}")
print(f"Expanded columns: {expanded_df.shape[1]}")
expanded_df.head()


Original columns: 32
Expanded columns: 32


,customerId,digitalLoanAccountId,sil_beta_income_estimation_score,start_time,end_time,modelDisplayName,modelVersionId,inc_beta_ln_loan_type,inc_beta_ln_education_level,inc_beta_ln_employment_type_new,inc_beta_ln_industry_new,inc_beta_ln_age,inc_beta_ln_brand,inc_beta_ln_city,inc_beta_ln_purpose,inc_beta_ln_osversion_bin,inc_beta_ln_postal_code,inc_beta_ln_gender,inc_beta_ln_doc_type_rolled,inc_beta_ln_cnt_dependents,inc_beta_ln_source_of_funds_new,inc_beta_ln_marital_status_new,inc_beta_encoded_company_name_grouped,new_loan_type,gender,Model_Name,product,trenchCategory,calcFeatures,appln_submit_datetime,disbursementDateTime,Application_month
0,2445354,28328f4c-cc58-4fe5-b8da-c1e208fc9cdb,33275.18861534607,2025-04-01 09:17:07.432853,2025-04-01 09:17:07.496704,Beta - IncomeEstimationModel,v1,BNPL,College Graduate,Employed,Doctor/Dentist/Medical Professional,31,samsung,CF0063,"Washing machine, dryer or both",2-Voyager,1870,F,Other,0,1,Single,93731.2656120701,SIL-Instore,F,SIL Beta - IncomeEstimationModel,SIL,NA,"{""inc_beta_ln_loan_type"":""BNPL"",""inc_beta_ln_e...",2025-04-01 17:17:04,NaT,2025-04
1,2501206,24261d70-6039-4fe0-9315-ae63c615e2eb,8430.189771057514,2025-08-15 03:07:23.801484,2025-08-15 03:07:23.859948,Beta - IncomeEstimationModel,v1,SILCOM,Technical/Vocational Graduate,Employed - Private Employee,Architecture/Engineering,40,TECNO,CF1682,Refrigerators,5-Pioneer,1776,M,ID Card,0,Salary,Single,0.0,SIL Competitor,M,SIL Beta - IncomeEstimationModel,SIL,NA,"{""inc_beta_ln_loan_type"":""SILCOM"",""inc_beta_ln...",2025-08-15 11:07:20,NaT,2025-08
2,2571988,2fddd759-74ef-44aa-8daa-dd640b8745ac,8692.120754263524,2025-04-16 02:50:20.735779,2025-04-16 02:50:20.794555,Beta - IncomeEstimationModel,v1,BNPL,College Undergraduate,Business Owner,Business Owner (MSME),41,TECNO,CF0579,Air conditioners,2-Voyager,4103,M,Other,3,3,Married,0.0,SIL-Instore,M,SIL Beta - IncomeEstimationModel,SIL,NA,"{""inc_beta_ln_loan_type"":""BNPL"",""inc_beta_ln_e...",2025-04-16 10:50:17,NaT,2025-04
3,2659855,733dd15d-3de6-4f5a-9042-516952662e3d,8523.899774803616,2025-08-19 04:09:34.771898,2025-08-19 04:09:34.840998,Beta - IncomeEstimationModel,v1,SILCOM,College Graduate,Employed - Private Employee,Maritime/Shipping,31,Apple,CF1367,Air conditioners,5-Pioneer,4232,F,UMID Card,0,Salary,Single,0.0,SIL Competitor,F,SIL Beta - IncomeEstimationModel,SIL,NA,"{""inc_beta_ln_loan_type"":""SILCOM"",""inc_beta_ln...",2025-08-19 12:09:32,2025-08-19 12:11:56,2025-08
4,2790668,8ab7a0f0-91c1-4456-b25d-32053c6dce9e,9084.45501044467,2025-04-20 09:46:28.118712,2025-04-20 09:46:28.173127,Beta - IncomeEstimationModel,v1,BNPL0.1,High School Graduate,Employed,Other Professional services,56,vivo,CF1185,Air conditioners,2-Voyager,3023,M,Other,1,1,Married,0.0,SIL ZERO,M,SIL Beta - IncomeEstimationModel,SIL,NA,"{""inc_beta_ln_loan_type"":""BNPL0.1"",""inc_beta_l...",2025-04-20 17:46:24,NaT,2025-04


In [117]:
df = expanded_df.drop(columns=['calcFeatures']).copy()

In [118]:
df['sil_beta_income_estimation_score'] = pd.to_numeric(df['sil_beta_income_estimation_score'], errors='coerce')
df.columns

Index(['customerId', 'digitalLoanAccountId',
       'sil_beta_income_estimation_score', 'start_time', 'end_time',
       'modelDisplayName', 'modelVersionId', 'inc_beta_ln_loan_type',
       'inc_beta_ln_education_level', 'inc_beta_ln_employment_type_new',
       'inc_beta_ln_industry_new', 'inc_beta_ln_age', 'inc_beta_ln_brand',
       'inc_beta_ln_city', 'inc_beta_ln_purpose', 'inc_beta_ln_osversion_bin',
       'inc_beta_ln_postal_code', 'inc_beta_ln_gender',
       'inc_beta_ln_doc_type_rolled', 'inc_beta_ln_cnt_dependents',
       'inc_beta_ln_source_of_funds_new', 'inc_beta_ln_marital_status_new',
       'inc_beta_encoded_company_name_grouped', 'new_loan_type', 'gender',
       'Model_Name', 'product', 'trenchCategory', 'appln_submit_datetime',
       'disbursementDateTime', 'Application_month'],
      dtype='object')

### run psi calculation

In [119]:
df = df.copy()

# Define feature list
feature_list = ['sil_beta_income_estimation_score',  'inc_beta_ln_loan_type',
       'inc_beta_ln_education_level', 'inc_beta_ln_employment_type_new',
       'inc_beta_ln_industry_new', 'inc_beta_ln_age', 'inc_beta_ln_brand',
       'inc_beta_ln_city', 'inc_beta_ln_purpose', 'inc_beta_ln_osversion_bin',
       'inc_beta_ln_postal_code', 'inc_beta_ln_gender',
       'inc_beta_ln_doc_type_rolled', 'inc_beta_ln_cnt_dependents',
       'inc_beta_ln_source_of_funds_new', 'inc_beta_ln_marital_status_new',
       'inc_beta_encoded_company_name_grouped',]

# Define segment columns
segment_columns = ['new_loan_type', 'gender', 'osType', 'trenchCategory', 'product']
# Calculate month-on-month PSI
psi_results = calculate_month_on_month_psi(df, feature_list, segment_columns)
psi_results['modelDisplayName'] = df['modelDisplayName'].iloc[0]
psi_results['Model_Name'] = df['Model_Name'].iloc[0]
psi_results['modelVersionId'] = df['modelVersionId'].iloc[0]
psi_results['trenchCategory'] = df['trenchCategory'].iloc[0]
psi_results = psi_results[[ 'modelDisplayName', 'Model_Name',   'modelVersionId', 'trenchCategory','Feature', 'Feature_Type', 'Segment_Column', 'Segment_Value', 'Month',
       'Base_Month', 'Current_Month', 'PSI']].copy()

# Calculate bin-level PSI
bin_psi_results = calculate_bin_level_psi(df, feature_list, segment_columns)
bin_psi_results['modelDisplayName'] = df['modelDisplayName'].iloc[0]
bin_psi_results['Model_Name'] = df['Model_Name'].iloc[0]
bin_psi_results['modelVersionId'] = df['modelVersionId'].iloc[0]
bin_psi_results['trenchCategory'] = df['trenchCategory'].iloc[0]
bin_psi_results = bin_psi_results[[ 'modelDisplayName', 'Model_Name',  'modelVersionId', 'trenchCategory','Feature', 'Feature_Type', 'Segment_Column', 'Segment_Value', 'Month',
       'Base_Month', 'Current_Month', 'Bin', 'Bin_Range', 'Bin_Min', 'Bin_Max', 'Base_Percentage',
       'Actual_Percentage', 'Bin_PSI']].copy()

# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.temp_csi_new_monitoring_data"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(psi_results, table_id, job_config=job_config)
job.result()  # Wait for the job to complete


# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.temp_csi_new_monitoring_data_feature_bin_level"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(bin_psi_results, table_id, job_config=job_config)
job.result()  # Wait for the job to complete

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=eac7cad2-60bf-4808-b87b-012d8c73ff1d>

In [120]:
bin_psi_results

,modelDisplayName,Model_Name,modelVersionId,trenchCategory,Feature,Feature_Type,Segment_Column,Segment_Value,Month,Base_Month,Current_Month,Bin,Bin_Range,Bin_Min,Bin_Max,Base_Percentage,Actual_Percentage,Bin_PSI
0,Beta - IncomeEstimationModel,SIL Beta - IncomeEstimationModel,v1,NA,sil_beta_income_estimation_score,numerical,Overall,All,2025-03,2025-03,2025-03,Bin_7,"[9357.10, 25685.69]",9357.096274,25685.686113,10.144928,10.144928,0.000000
1,Beta - IncomeEstimationModel,SIL Beta - IncomeEstimationModel,v1,NA,sil_beta_income_estimation_score,numerical,Overall,All,2025-03,2025-03,2025-03,Bin_10,"[29863.25, 60822.05]",29863.253940,60822.050743,10.144928,10.144928,0.000000
2,Beta - IncomeEstimationModel,SIL Beta - IncomeEstimationModel,v1,NA,sil_beta_income_estimation_score,numerical,Overall,All,2025-03,2025-03,2025-03,Bin_4,"[8957.18, 9067.55]",8957.177895,9067.553655,10.144928,10.144928,0.000000
3,Beta - IncomeEstimationModel,SIL Beta - IncomeEstimationModel,v1,NA,sil_beta_income_estimation_score,numerical,Overall,All,2025-03,2025-03,2025-03,Bin_1,"[7724.89, 8539.50]",7724.886081,8539.503169,10.144928,10.144928,0.000000
4,Beta - IncomeEstimationModel,SIL Beta - IncomeEstimationModel,v1,NA,sil_beta_income_estimation_score,numerical,Overall,All,2025-03,2025-03,2025-03,Bin_2,"[8539.50, 8856.72]",8539.503169,8856.718048,9.903382,9.903382,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5599,Beta - IncomeEstimationModel,SIL Beta - IncomeEstimationModel,v1,NA,inc_beta_encoded_company_name_grouped,categorical,product,SIL,2025-10,2025-03,2025-10,30086.7246045263,30086.7246045263,NaN,NaN,0.724638,0.253209,0.004839
5600,Beta - IncomeEstimationModel,SIL Beta - IncomeEstimationModel,v1,NA,inc_beta_encoded_company_name_grouped,categorical,product,SIL,2025-10,2025-03,2025-10,30211.7246045263,30211.7246045263,NaN,NaN,0.483092,0.497686,0.000004
5601,Beta - IncomeEstimationModel,SIL Beta - IncomeEstimationModel,v1,NA,inc_beta_encoded_company_name_grouped,categorical,product,SIL,2025-10,2025-03,2025-10,30892.5062448281,30892.5062448281,NaN,NaN,23.188406,13.638348,0.050660
5602,Beta - IncomeEstimationModel,SIL Beta - IncomeEstimationModel,v1,NA,inc_beta_encoded_company_name_grouped,categorical,product,SIL,2025-10,2025-03,2025-10,480455.9091384736,480455.9091384736,NaN,NaN,0.724638,0.061119,0.015493


# End